<a href="https://colab.research.google.com/github/denliew/Chronicle-APIs-Colab/blob/main/Chronicle_APIs_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Start Here
To start we need to clone the git repo of the google API python cliet as well as the CBN Tool and CLI tool, and then install them on this Google Colab instance.

In [ ]:
#@title Run me first
#@markdown Run this cell to clone needed git repos and configure the environment. Should take about 1-3 minutes to complete.
from IPython.display import clear_output


!!rm -rf sample_data
!!git clone --depth 1 https://github.com/googleapis/google-api-python-client --single-branch --branch=main
!!git clone --depth 1 https://github.com/chronicle/cbn-tool --single-branch --branch=main
!!git clone --depth 1 https://github.com/chronicle/cli --single-branch --branch=main
!!pip install -r /content/cli/requirements.txt
!!python3 -m pip install --editable /content/cli/
!!mkdir ~/.chronicle_cli
!!pip install -r /content/cbn-tool/requirements.txt
!!pip install google-api-python-client
clear_output(wait=False)
print(f"Initialization completed. Please proceed to the Upload keyfile section.")

Initialization completed. Please proceed to the Upload keyfile section.


The next step is to upload the appropriate service key to the environment and initialize the web client.

> ⇒ Click the play button below, then select your keyfile.



In [ ]:
#@title Upload keyfile & Initialize web client
# Imports required for the sample - Google Auth and API Client Library Imports.
# Get these packages from https://pypi.org/project/google-api-python-client/ or run $ pip
# install google-api-python-client from your terminal
from google.oauth2 import service_account
from google.auth.transport.requests import AuthorizedSession
from googleapiclient import _auth
import urllib
import json
import base64
import http
import re
import os
import sys
# import subprocess
from pprint import pprint
from time import sleep
from urllib.parse import urlencode
from datetime import datetime
from datetime import timezone

SCOPES = [
    'https://www.googleapis.com/auth/chronicle-backstory',  #regular backstory API
    'https://www.googleapis.com/auth/malachite-ingestion',  #ingestion API
    'https://www.googleapis.com/auth/cloud-platform'        #dataplane API (experimenting)
    ]
#@markdown Please make sure you select the appopriate region below.
region = "North America" #@param  ["North America", "Europe", "Asia (Singapore)", "United Kingdom (London)", "Australia (Sydney)", "Tel Aviv", "Frankfurt", "Zurich", "Saudi Arabia", "Mumbai", "Canada", "Japan (Tokyo)", "Qatar", "Italy", "Brazil", "France", "Jakarta"]

#@markdown <font color=red>Warning:</font> Due to the coming deprecation of the CBN tool newer regions are not supported. Only the US (North America) and Europe regions are known to work without a rewrite of the CBN scripts.

if region == "North America":
  region_prefix = ""
  cbn_region = "US"
  cli_region = "US"
elif region == "Europe":
  region_prefix = "europe-"
  cbn_region = "EUROPE"
  cli_region = "EUROPE"
elif region == "United Kingdom (London)": #Brexit now supported
  region_prefix = "europe-west2-"
  cbn_region = "EUROPE" #This won't work since the CBN tool doesn't have the UK region in it
  cli_region = "EUROPE-WEST2" #This may not work since the CBN tool doesn't have the UK region in it
elif region == "Asia (Singapore)":
  region_prefix = "asia-southeast1-"
  cbn_region = "ASIA"
  cli_region = "ASIA-SOUTHEAST1"
elif region == "Asia (Indonesia)":
  region_prefix ="asia-southeast2-"
  cbn_region = "ASIA"
  cli_region = "ASIA-SOUTHEAST2"
elif region == "Australia (Sydney)":
  region_prefix = "australia-southeast1-"
  cbn_region = "AUSTRALIA"
  cli_region = "AUSTRALIA-SOUTHEAST1"
elif region == "Tel Aviv":
  region_prefix = "me-west1-"
  cli_region = "ME-WEST1"
elif region == "Mumbai":
  region_prefix = "asia-south1-"
  cli_region = "ASIA-SOUTH1"
elif region == "Frankfurt":
  region_prefix = "europe-west3-"
  cli_region = "EUROPE-WEST3"
elif region == "Zurich":
  region_prefix = "europe-west6-"
  cli_region = "EUROPE-WEST6"
elif region == "Saudi Arabia":
  region_prefix = "me-central2-"
  cli_region = "ME-CENTRAL2"
elif region == "Canada":
  region_prefix = "northamerica-northeast2-"
  cli_region = "NORTHAMERICA_NORTHEAST2"
elif region == "Japan (Tokyo)":
  region_prefix = "asia-northeast1-"
  cli_region = "ASIA-NORTHEAST1",
elif region == "Brazil":
  region_prefix = "southamerica-east1-"
  cli_region = "SOUTHAMERICA-EAST1"
elif region == "France":
  region_prefix = "europe-west9-"
  cli_region = "EUROPE-WEST9"
elif region == "Italy":
  region_prefix = "europe-west12-"
  cli_region = "EUROPE-WEST12"
elif region == "Qatar":
  region_prefix = "me-central1-"
  cli_region = "ME-CENTRAL1"


# The apikeys-demo.json file contains the customer's OAuth 2 credentials.
# SERVICE_ACCOUNT_FILE is the full path to the apikeys-demo.json file
# ToDo: Replace this with the full path to your OAuth2 credentials

os.system(f"rm supplied_key.json")

from google.colab import files
print("Please select and upload your keyfile")
uploaded = files.upload()

os.system(f"mv {list(uploaded.keys())[0]} supplied_key.json")

!cp supplied_key.json ~/.chronicle_cli/chronicle_credentials.json

key_filename = "supplied_key.json"

SERVICE_ACCOUNT_FILE = key_filename

# Create a credential using Google Developer Service Account Credential and Chronicle API
# Scope.
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# Build an HTTP client to make authorized OAuth requests.
http_client = _auth.authorized_http(credentials)
session = AuthorizedSession(credentials)

print("Webclient initialized")

Please select and upload your keyfile


Saving dmrsk-bk-1709044560.json to dmrsk-bk-1709044560.json
Webclient initialized


##Quick Jump Index
|                                                   |                                                     |                                               |
|:-------------------------------------------------:|:---------------------------------------------------:|:---------------------------------------------:|
|   [BigQuery Access API](#scrollTo=KCl0S9f9wWeu)   | [DataTap Configuration API](#scrollTo=MoDoZbCq8P7M) |    [Ingestion API](#scrollTo=6Ug4UflixBld)    |
|         [CBN Tool](#scrollTo=ACbfcv9MdcYk)        |    [Detection Engine API](#scrollTo=gOLIlkpBCpNM)   |      [Parser API](#scrollTo=jV2L4JJXdVxn)     |
|         [CLI tool](#scrollTo=Wx1pfTxOjv57)        |    [Feed Management API](#scrollTo=B1oCjyR52XG1)    | [Parser API v2 (dev)](#scrollTo=hJclWq5wb_1N) |
|      [Collector API](#scrollTo=bdE3Nw4Yd6Xr)      |      [Feed Schema API](#scrollTo=m3wpTMoXG3Fh)      |       [RBAC API](#scrollTo=hbI-zFGq6hI5)      |
| [Customer Management API](#scrollTo=ca5dXllqgGwQ) |       [Forwarder API](#scrollTo=gY3dJhKQRmKD)       |  [Reference List API](#scrollTo=NMvpcTZ3b-Ij) |
|     [Data Export API](#scrollTo=q5xxRkQL2OM-)     |          [GCTI API](#scrollTo=RzEhT4Gsy7jN)         |      [Search API](#scrollTo=-q_rBGSYzWD8)     |

#Parser API
[Jump to top](#scrollTo=v0jKbGj8ZZRB)

In [ ]:
#@title Get current custom parsers
uri = f'https://{region_prefix}backstory.googleapis.com/v1/tools/cbnParsers'

# Send the request, parse the response, save logs to a file
response = session.get(uri)

if response.status_code == 200:
  live_parsers = response.json()

  index = 0
  if "cbnParsers" in live_parsers:
    for parser in live_parsers['cbnParsers']:
      print("Type: {type}\nAuthor:{author}\nState:{state}\nIndex: {index}\n".format(author= parser['author'], state= parser["state"], type= parser["logType"], index=index))
      index += 1
  else:
    print("There don't appear to be any live custom parsers at the moment.")
else:
  # something went wrong
  print(response.text)

In [ ]:
#@title Get Sample Logs

uri = f'https://{region_prefix}backstory.googleapis.com/v1/tools:retrieveSampleLogs'

max_entries = 1000 #@param {type: "number"}
log_type = "IBM_SAM" #@param {type:"string"}

startDate = "2021-11-01" #@param {type: "date"}
endDate = "2023-07-19" #@param {type: "date"}

#@markdown Time should be in the format HH:MM:SS including leading zeros
startTime = "00:00:01" #@param {type: "string"}
endTime = "23:59:59" #@param {type: "string"}

startTime = f"{startDate}T{startTime}Z"
endTime = f"{endDate}T{endTime}Z"

# define the logType, ingestion start and end, and the maximum number or records to return.
data = {
  'log_type': log_type,
  'start_time': startTime,
  'end_time': endTime,
  'max_entries': max_entries
}

headers= {
  "Content-type": "application/x-www-form-urlencoded"
}

# Send the request, parse the response, save logs to a file
response = session.post(uri, params=data, headers=headers)

# ToDo: replace this value with you own
file_name = 'PAN_PRISMA_CA.logs' #@param {type: "string"}

#check if file already exists, if it does delete it to prevent confusion
if file_name in os.listdir():
  os.remove(file_name)

download_file = False #@param {"type": "boolean"}
if response.status_code == 200:
  sample_logs = response.json()
  sample_logs_data = sample_logs.get('data', [])
  if len(sample_logs_data) == 0:
    print("No logs returned for selected time.")
  else:
    for sample_log in sample_logs_data:
      f = open(file_name, 'a')
      f.write(base64.b64decode(sample_log).decode())
      f.write('\n')
      f.close()
      # print(base64.b64decode(sample_log).decode())
    print("{} entries have been writen to filename: {} ".format(len(sample_logs_data), file_name))
    if download_file:
      files.download(file_name)
else:
  # something went wrong
  print(response.text)


In [ ]:
#@title Get parser history

# logType identifying the parser
log_type = 'DNS_SINKHOLE' #@param {type: "string"}

uri = f'https://{region_prefix}backstory.googleapis.com/v1/tools/cbnParsers:listCbnParserHistory?log_type={log_type}'
# send the request and process the response
resp = session.get(uri)
json_resp = resp.json()
# show the parser history
if resp.status_code > 299:
  pprint(json_resp['error'])
else:
  print("{:36s} {:15s} {:10s} {:25s}".format("configId","logType","state","stateLastChangedTime"))
  for item in json_resp['cbnParsers']:
    print("{:36s} {:15s} {:10s} {:25s}".format(item['configId'], item['logType'], item['state'], item['stateLastChangedTime']))

In [ ]:
#@title Test parser
#@markdown Enter the filename of the CBN to test
parser_conf_file = "parser.conf" #@param {type: "string"}
#@markdown Enter the filename of the testdata
test_data_file = "sample.log" #@param {type: "string"}

#@markdown Choose whether to write the output data to a file and then whether to download that file.
write_to_file = False #@param {type: "boolean"}
download_file = False #@param {type: "boolean"}

file_name = "parser_output.blah" #@param {type: "string"}
also_write_to_screen = False #@param {type: "boolean"}

uri = f'https://{region_prefix}backstory.googleapis.com/v1/tools:validateCbnParser'

conf_file = open(parser_conf_file, 'rb')
conf_data = conf_file.read()
conf_file.close()

log_file = open(test_data_file, 'rb')
log_data = log_file.read()
log_file.close()

data = {
    'config' : base64.urlsafe_b64encode(conf_data).decode(),
    'logs' : base64.urlsafe_b64encode(log_data).decode()
}

r_headers = {}
r_headers.update({'Content-type' : 'application/x-www-form-urlencoded'})

resp = session.post(uri,  data=data, headers=r_headers)

if resp.status_code > 299:
  print(resp.text)
else:
  if write_to_file:
    with open(file_name, "w") as output_file:
      output_file.write(resp.text)
      if also_write_to_screen:
        pprint(resp.json())
    if download_file:
      files.download(file_name)
  else:
    pprint(resp.json())

In [ ]:
#@title Get a single CBN parser
# Unique configuration identifier assigned to the parser when it was submitted
# ToDo: Replace this value with your own.

#@markdown Config_id can either be an ID of a custom parser or the name of a default parser (for example: OKTA)
# config ID list taken from ingest API get support log types endpoint
config_id = "IBM_CLOUD_ACTIVITY_TRACKER" #@param ['A10_LOAD_BALANCER', 'ABNORMAL_SECURITY', 'ABSOLUTE', 'ACALVIO', 'ACCELLION', 'ACCOPS_HYSECURE_VPN', 'ACQUIA_CLOUD_PLATFORM', 'ACRONIS', 'ACTIVE_SYNC', 'ADAUDIT_PLUS', 'ADAXES', 'ADFS', 'ADMANAGER_PLUS', 'ADMIN_BY_REQUEST', 'ADOBE_COMMERCE', 'ADOBE_EXPERIENCE_MANAGER', 'ADSELFSERVICE_PLUS', 'ADTRAN_NETVANTA', 'ADVA_FSP', 'AGARI_PHISHING_DEFENSE', 'AIDE', 'AIRDEFENSE', 'AIRLOCK_DIGITAL', 'AIRWATCH', 'AIR_TABLE', 'AIX_SYSTEM', 'AI_HUNTER', 'AKAMAI_CLOUD_MONITOR', 'AKAMAI_DDOS', 'AKAMAI_DHCP', 'AKAMAI_DNS', 'AKAMAI_EAA', 'AKAMAI_ETP', 'AKAMAI_GUARDICORE', 'AKAMAI_SIEM_CONNECTOR', 'AKAMAI_WAF', 'AKEYLESS_VAULT', 'ALCATEL_SWITCH', 'ALERTLOGIC_NOTIFICATIONS', 'ALERT_GUARDIAN', 'ALGOSEC', 'ALICLOUD_ANTI_DDOS', 'ALICLOUD_WAF', 'ALIENVAULT_OTX', 'ALLOT_NETENFORCER', 'ALVEO_RDM', 'AMAVIS', 'AMD_DSS_FIREWALL', 'ANALYST1_IOC', 'ANOMALI_IOC', 'ANSIBLE_AWX', 'APACHE', 'APACHE_KAFKA_AUDIT', 'APACHE_SPAMASSASSIN', 'APC_ATS', 'APC_NETBOTZ', 'APC_PDU', 'APC_SMART_UPS', 'APC_STRUXUREWARE', 'APIIRO', 'APPGATE_SDP', 'APPIAN_CLOUD', 'APPOMNI', 'APPVIEWX', 'APTOS_EOM', 'AQUA_SECURITY', 'ARBOR_EDGE_DEFENSE', 'ARBOR_SIGHTLINE', 'ARCHER_IRM', 'ARCSIGHT_CEF', 'AREA1', 'ARGO_CD', 'ARGO_WORKFLOWS', 'ARISTA_AGNI', 'ARISTA_CVP', 'ARISTA_NDR', 'ARISTA_SWITCH', 'ARKIME_PCAP', 'ARMIS', 'ARMIS_ACTIVITIES', 'ARMIS_ALERTS', 'ARMIS_DEVICES', 'ARMIS_VULNERABILITIES', 'ARMORBLOX_ESC', 'ARMOR_ANYWHERE', 'ARRAYNETWORKS_VPN', 'ARRAY_NETWORKS_WAF', 'ARUBA_AIRWAVE', 'ARUBA_CENTRAL', 'ARUBA_EDGECONNECT_SDWAN', 'ARUBA_IPS', 'ARUBA_ORCHESTRATOR', 'ARUBA_SWITCH', 'ARUBA_WIRELESS', 'ARXAN_THREAT_ANALYTICS', 'ASANA', 'ASCERTIA', 'ASIMILY', 'ASOC_ALERT', 'ASSETNOTE', 'ASSET_STATIC_IP', 'ATLASSIAN_AUDIT', 'ATLASSIAN_BEACON', 'ATLASSIAN_BITBUCKET', 'ATLASSIAN_CONFLUENCE', 'ATLASSIAN_CONFLUENCE_JSON', 'ATLASSIAN_JIRA', 'ATLASSIAN_JIRA_JSON', 'ATTIVO', 'ATT_NETBOND', 'AUDITD', 'AUTHENTIC8_SILO', 'AUTHX', 'AUTHX_USER_CONTEXT', 'AUTH_ZERO', 'AUTOMATION_ANYWHERE', 'AUTOMOX_EPM', 'AVANAN_EMAIL', 'AVAST_HUB', 'AVATIER', 'AVAYA_AURA', 'AVAYA_BORDER', 'AVAYA_IVR', 'AVAYA_VSP', 'AVAYA_WIRELESS', 'AVIATRIX', 'AWAKE_NDR', 'AWS_AURORA', 'AWS_CLOUDFRONT', 'AWS_CLOUDTRAIL', 'AWS_CLOUDWATCH', 'AWS_CONFIG', 'AWS_CONTROL_TOWER', 'AWS_DYNAMO_DB', 'AWS_EC2_HOSTS', 'AWS_EC2_INSTANCES', 'AWS_EC2_VPCS', 'AWS_ELASTI_CACHE', 'AWS_ELB', 'AWS_EMR', 'AWS_FSX', 'AWS_IAM', 'AWS_INSPECTOR', 'AWS_INSPECTOR2', 'AWS_KMS', 'AWS_MACIE', 'AWS_NETWORK_FIREWALL', 'AWS_NGINX', 'AWS_RDS', 'AWS_REDSHIFT', 'AWS_ROUTE_53', 'AWS_S3_SERVER_ACCESS', 'AWS_SECURITY_HUB', 'AWS_SES', 'AWS_SESSION_MANAGER', 'AWS_SHIELD', 'AWS_VPC_FLOW', 'AWS_VPN', 'AWS_WAF', 'AXIS_ATMOS', 'AXIS_OS', 'AXONIUS', 'AZION', 'AZURE', 'AZURE_ACTIVITY', 'AZURE_AD', 'AZURE_AD_AUDIT', 'AZURE_AD_CONTEXT', 'AZURE_AD_PASSWORD_PROTECTION', 'AZURE_AD_PROVISIONING', 'AZURE_AD_SIGNIN', 'AZURE_API_MANAGEMENT', 'AZURE_APP_SERVICE', 'AZURE_ATP', 'AZURE_BASTION', 'AZURE_COSMOS_DB', 'AZURE_DEVOPS', 'AZURE_DNS', 'AZURE_FIREWALL', 'AZURE_FRONT_DOOR', 'AZURE_GATEWAY', 'AZURE_KEYVAULT_AUDIT', 'AZURE_MDM_INTUNE', 'AZURE_MDM_INTUNE_CONTEXT', 'AZURE_NSG_FLOW', 'AZURE_RESOURCE_LOGS', 'AZURE_SECURITY_CENTER', 'AZURE_SQL', 'AZURE_STORAGE_AUDIT', 'AZURE_VPN', 'AZURE_WAF', 'BABELFORCE', 'BACKBOX', 'BALABIT', 'BAMBOO_HR', 'BANNER_DD', 'BARRACUDA_CLOUDGEN_ACCESS', 'BARRACUDA_CLOUDGEN_FIREWALL', 'BARRACUDA_EMAIL', 'BARRACUDA_FIREWALL', 'BARRACUDA_IMPERSONATION', 'BARRACUDA_SHIELD', 'BARRACUDA_WAF', 'BARRACUDA_WEBFILTER', 'BETTERCLOUD', 'BEYONDTRUST_BEYONDINSIGHT', 'BEYONDTRUST_CPB', 'BEYONDTRUST_ENDPOINT', 'BEYONDTRUST_MC', 'BEYONDTRUST_PI', 'BEYONDTRUST_REMOTE_ACCESS', 'BEYOND_IDENTITY', 'BIGSWITCH_BCF', 'BIND_DNS', 'BITDEFENDER', 'BITVISE_SSHD', 'BITWARDEN_EVENTS', 'BLACKBERRY_WORKSPACES', 'BLOXONE', 'BLUECAT_AM', 'BLUECAT_DDI', 'BLUECAT_EDGE', 'BLUECOAT_WEBPROXY', 'BLUE_PRISM', 'BMC_AMI_DEFENDER', 'BMC_CLIENT_MANAGEMENT', 'BMC_CONTROL_M', 'BMC_HELIX_DISCOVERY', 'BOKS', 'BOMGAR', 'BOX', 'BRICATA_NDR', 'BRITIVE_AUDIT_API', 'BRIVO', 'BROADCOM_CA_PAM', 'BROADCOM_CEM', 'BROADCOM_SSL_VA', 'BROADCOM_SUPPORT_PORTAL', 'BROCADE_FOS', 'BROCADE_SANNAV', 'BROCADE_SERVERIRON', 'BROCADE_SWITCH', 'BRO_DHCP', 'BRO_HTTP', 'BRO_JSON', 'BRO_TSV', 'BT_IPCONTROL', 'BURPSUITE', 'CAMBIUM_NETWORKS', 'CAMEYO_BYO_CLOUD', 'CANARY_AUDIT_TRAIL', 'CASSANDRA', 'CATO_NETWORKS', 'CATO_SDWAN', 'CA_ACCESS_CONTROL', 'CA_ACF2', 'CA_LDAP', 'CA_SSO_WEB', 'CB_APP_CONTROL', 'CB_EDR', 'CENSORNET_CASB', 'CENSYS', 'CENTRIFY_SSO', 'CENTRIPETAL_IOC', 'CEQUENCE_BOT_DEFENSE', 'CERBERUS_FTP', 'CHECKPOINT_CLOUDGUARD', 'CHECKPOINT_EDR', 'CHECKPOINT_EMAIL', 'CHECKPOINT_FIREWALL', 'CHECKPOINT_GAIA', 'CHECKPOINT_HARMONY', 'CHECKPOINT_SMARTDEFENSE', 'CHRONICLE_SOAR_AUDIT', 'CILIUM', 'CIPHERTRUST_MANAGER', 'CIRCLECI', 'CISCO_ACE', 'CISCO_ACI', 'CISCO_ACS', 'CISCO_AIRONET', 'CISCO_AMP', 'CISCO_APIC', 'CISCO_ASA_FIREWALL', 'CISCO_CALL_MANAGER', 'CISCO_CLOUDLOCK_CASB', 'CISCO_CTS', 'CISCO_CYBER_VISION', 'CISCO_DHCP', 'CISCO_DNAC', 'CISCO_DNS', 'CISCO_EMAIL_SECURITY', 'CISCO_ESTREAMER', 'CISCO_FIREPOWER_FIREWALL', 'CISCO_FIRESIGHT', 'CISCO_FWSM', 'CISCO_IOS', 'CISCO_IRONPORT', 'CISCO_ISE', 'CISCO_MERAKI', 'CISCO_MERAKI_CAMERA', 'CISCO_NX_OS', 'CISCO_PIX_FIREWALL', 'CISCO_PRIME', 'CISCO_ROUTER', 'CISCO_SDWAN', 'CISCO_SECURE_MALWARE_ANALYTICS', 'CISCO_SECURE_WORKLOAD', 'CISCO_SMA', 'CISCO_SNMP', 'CISCO_STADIUMVISION', 'CISCO_STEALTHWATCH', 'CISCO_SWITCH', 'CISCO_TACACS', 'CISCO_UCM', 'CISCO_UCS', 'CISCO_UMBRELLA_AUDIT', 'CISCO_UNITY_CONNECTION', 'CISCO_VCS', 'CISCO_VPN', 'CISCO_WIPS', 'CISCO_WIRELESS', 'CISCO_WSA', 'CISCO_WSM', 'CISCO_XDR', 'CIS_ALBERT_ALERT', 'CITRIX_ANALYTICS', 'CITRIX_MONITOR', 'CITRIX_NETSCALER', 'CITRIX_NETSCALER_WEB_LOGS', 'CITRIX_SDWAN', 'CITRIX_SESSION_METADATA', 'CITRIX_STOREFRONT', 'CITRIX_VDI', 'CITRIX_WAF', 'CITRIX_WEB_GATEWAY', 'CITRIX_WORKSPACE', 'CITRIX_XENCENTER', 'CLAM_AV', 'CLAROTY_CTD', 'CLAROTY_EMC', 'CLAROTY_XDOME', 'CLEARPASS', 'CLEARSENSE', 'CLEARSWIFT', 'CLICK_STUDIOS_PASSWORDSTATE', 'CLOUDAWARE', 'CLOUDBEES', 'CLOUDBOLT', 'CLOUDFLARE', 'CLOUDFLARE_AUDIT', 'CLOUDFLARE_BOT_MANAGEMENT', 'CLOUDFLARE_WAF', 'CLOUDGENIX_SDWAN', 'CLOUDIAN_HYPERSTORE', 'CLOUDM', 'CLOUDPASSAGE_CSM', 'CLOUDPASSAGE_FIM', 'CLOUDPASSAGE_LIDS', 'CLOUDPASSAGE_SVM', 'CLOUD_IDENTITY_CONTEXT', 'CLOUD_PASSAGE', 'CMD', 'COALITION', 'COCKROACH_DB', 'CODE42', 'CODE42_INCYDR', 'CODE_WORLDWIDE', 'COFENSE_TRIAGE', 'COFENSE_VISION', 'COHESITY', 'COHESITY_HELIOS', 'COHESITY_SMARTFILES', 'COMMVAULT', 'COMMVAULT_COMMCELL', 'COMMVAULT_METALLIC', 'COMODO_AV', 'CONFLUENT_AUDIT', 'CONNECTWISE_AUTOMATE', 'CONNECTWISE_CONTROL', 'CORELIGHT', 'CORRATA', 'CORTEX_XDR', 'COVID_CTC_IOC', 'CRADLEPOINT_NETCLOUD', 'CRIBL_APPSCOPE', 'CRIBL_CLOUD', 'CRIBL_EDGE', 'CRIBL_SEARCH', 'CRIBL_STREAM', 'CROWDSTRIKE_IOC', 'CRYPTOSPIKE', 'CSG_CUSTOMENGINE', 'CSG_SINGLEVIEW', 'CSV_CUSTOM_CMDB', 'CSV_CUSTOM_IOC', 'CS_CEF_EDR', 'CS_DETECTS', 'CS_EDR', 'CS_IDP', 'CS_STREAM', 'CTERA_DRIVE', 'CULTURE_AI', 'CUSTOMER_ALERT', 'CUSTOM_APPLICATION_ACCESS', 'CUSTOM_DNS', 'CUSTOM_HOST_FORENSICS', 'CUSTOM_SECURITY_DATA_ANALYTICS', 'CYBERARK', 'CYBERARK_EPM', 'CYBERARK_PAM', 'CYBERARK_PRIVILEGE_CLOUD', 'CYBERARK_SSO', 'CYBERCNS', 'CYBEREASON_EDR', 'CYBERGATEKEEPER_NAC', 'CYBERHAVEN_DDR', 'CYBERHAVEN_EVENTS', 'CYBERINT', 'CYBERX', 'CYBER_2_IDS', 'CYCODE', 'CYDERES_INSIDER', 'CYDERES_IOC', 'CYLANCE', 'CYLANCE_PROTECT', 'CYLERA_IOT', 'CYMULATE', 'CYNET_360_AUTOXDR', 'CYOLO_ZTNA', 'D3_BANKING', 'D3_SECURITY', 'DARKTRACE', 'DATABRICKS', 'DATADOG', 'DATAIKU_DSS_LOGS', 'DATALOCKER_SAFECONSOLE', 'DATALUST', 'DATAMINR_ALERT', 'DATASUNRISE_DAM', 'DATAWATCH', 'DATTO_FILE_PROTECTION', 'DB2_DB', 'DEAL_CLOUD', 'DEEPFENCE', 'DEEP_INSTINCT_EDR', 'DELINEA_PAM', 'DELINEA_PRIVILEGE_MANAGER', 'DELINEA_SECRET_SERVER', 'DELINEA_SERVER_SUITE', 'DELL_COMPELLENT', 'DELL_CRM', 'DELL_CYBERSENSE', 'DELL_ECS', 'DELL_EMC_AVAMAR', 'DELL_EMC_CLOUDLINK', 'DELL_EMC_DATA_DOMAIN', 'DELL_EMC_NAS', 'DELL_EMC_POWERSTORE', 'DELL_EMC_UNITY', 'DELL_OPENMANAGE', 'DELL_SWITCH', 'DELL_WAF', 'DESIGN_PROFIT_CENTRAL_SERVER', 'DESYNOVA_CONTIDO', 'DEVICE_42', 'DEVOLUTIONS_RDM', 'DHS_IOC', 'DIGITALARTS_IFILTER', 'DIGITALGUARDIAN_DLP', 'DIGITALGUARDIAN_EDR', 'DIGITAL_SHADOWS_IOC', 'DIGITAL_SHADOWS_SEARCHLIGHT', 'DIGI_MODEMS', 'DIVVY_CLOUD', 'DMARCIAN', 'DMP_ENTRE', 'DNSFILTER', 'DOCKER', 'DOCUSIGN', 'DOMAINTOOLS_THREATINTEL', 'DOMO', 'DOPE_SWG', 'DRAGOS', 'DRAYTEK', 'DREMIO_DATA_LAKEHOUSE', 'DROPBOX', 'DRUPAL', 'DRUVA_BACKUP', 'DSP_AUDIT', 'DTEX_INTERCEPT', 'DUO_ACTIVITY', 'DUO_ADMIN', 'DUO_AUTH', 'DUO_CASB', 'DUO_CONTEXT', 'DUO_NETWORK_GATEWAY', 'DUO_TELEPHONY', 'DUO_TRUST_MONITOR', 'DUO_USER_CONTEXT', 'DYNATRACE', 'E2_GUARDIAN', 'E2_SOLUTIONS', 'EATON_UPS', 'ECAR', 'ECAR_BRO', 'EDGECORE_NETWORKS', 'EDGIO_CDN', 'EDGIO_RL', 'EDGIO_WAF', 'EFAX', 'EFFICIENTIP_DDI', 'EGNYTE', 'EGRESS_DEFEND', 'EGRESS_PREVENT', 'EIQ_EDR', 'ELASTIC_AUDITBEAT', 'ELASTIC_FILEBEAT', 'ELASTIC_METRICBEAT', 'ELASTIC_PACKETBEATS', 'ELASTIC_SEARCH', 'ELASTIC_WINLOGBEAT', 'EMERSON_FIREWALL', 'EMSISOFT_ANTIVIRUS', 'ENDGAME_EDR', 'ENDPOINT_PROTECTOR_DLP', 'ENSONO', 'ENTRUST_HSM', 'ENTRUST_NTP_SERVER', 'ENTRUST_SECRETS_VAULT', 'EPIC', 'ERLANG_SHELL', 'ERMES', 'ERMETIC', 'ESET_AV', 'ESET_EDR', 'ESET_IOC', 'ESHARE_PLATFORM', 'ESTAR', 'ETQ_RELIANCE', 'ET_PRO_IOC', 'EVIDOS_FIREWALL', 'EVISION_FIRCOSOFT', 'EXABEAM_FUSION_XDR', 'EXCHANGE_MAIL', 'EXTRAHOP', 'EXTRAHOP_DHCP', 'EXTRAHOP_DNS', 'EXTREMEWARE_NETWORKS', 'EXTREME_CONTROL', 'EXTREME_MANAGEMENT', 'EXTREME_SWITCH', 'EXTREME_WIRELESS', 'EZPROXY', 'F5_AFM', 'F5_ASM', 'F5_BIGIP_APM', 'F5_BIGIP_LTM', 'F5_BOT', 'F5_DCS', 'F5_DNS', 'F5_IP_INTELLIGENCE', 'F5_SHAPE', 'F5_SILVERLINE', 'F5_VPN', 'FAIL2BAN', 'FALCO_IDS', 'FARSIGHT_DNSDB', 'FASTLY_WAF', 'FEENICS_ACCESS_CONTROL', 'FIDELIS_ENDPOINT', 'FIDELIS_NETWORK', 'FILEMAGE_SFTP', 'FILEZILLA_FTP', 'FILE_SCANNING_FRAMEWORK', 'FIREBASE', 'FIREEYE_ALERT', 'FIREEYE_CMS', 'FIREEYE_EMPS', 'FIREEYE_ETP', 'FIREEYE_HELIX', 'FIREEYE_HX', 'FIREEYE_HX_AUDIT', 'FIREEYE_NX', 'FIREEYE_PX', 'FIREMON_FIREWALL', 'FISGLOBAL_QUANTUM', 'FIVETRAN', 'FLASHPOINT_IOC', 'FLEET_DM', 'FLUENTD', 'FORCEPOINT_CASB', 'FORCEPOINT_DLP', 'FORCEPOINT_EMAILSECURITY', 'FORCEPOINT_FIREWALL', 'FORCEPOINT_FIT', 'FORCEPOINT_MAIL_RELAY', 'FORCEPOINT_VSERIES', 'FORCEPOINT_WEBPROXY', 'FORESCOUT_NAC', 'FORGEROCK_IDENTITY_CLOUD', 'FORGEROCK_OPENIDM', 'FORSETI', 'FORTANIX_DSM', 'FORTINET_AP', 'FORTINET_DHCP', 'FORTINET_FIREWALL', 'FORTINET_FORTIANALYZER', 'FORTINET_FORTIAUTHENTICATOR', 'FORTINET_FORTICLIENT', 'FORTINET_FORTIEDR', 'FORTINET_FORTIMAIL', 'FORTINET_FORTINAC', 'FORTINET_FORTIWEB', 'FORTINET_SANDBOX', 'FORTINET_SWITCH', 'FORTINET_WEBPROXY', 'FORTRA_POWERTECH_SIEM_AGENT', 'FOUNDRY_FASTIRON', 'FOX_IT_STIX', 'FREEIPA', 'FREERADIUS', 'FRONTLINE_VM', 'FS_ISAC_IOC', 'FUTUREX_HSM', 'GCP_APIGEE', 'GCP_APIGEE_X', 'GCP_APP_ENGINE', 'GCP_ARTIFACT_REGISTRY', 'GCP_BIGQUERY_CONTEXT', 'GCP_CLOUDAUDIT', 'GCP_CLOUDIDENTITY_DEVICES', 'GCP_CLOUDIDENTITY_DEVICEUSERS', 'GCP_CLOUDIOT', 'GCP_CLOUDSQL', 'GCP_CLOUD_FUNCTIONS_CONTEXT', 'GCP_CLOUD_NAT', 'GCP_COMPUTE', 'GCP_COMPUTE_CONTEXT', 'GCP_DNS', 'GCP_FIREWALL', 'GCP_IAM_ANALYSIS', 'GCP_IAM_CONTEXT', 'GCP_IDS', 'GCP_KUBERNETES_CONTAINER_SECURITY', 'GCP_KUBERNETES_CONTEXT', 'GCP_LOADBALANCING', 'GCP_NETWORK_CONNECTIVITY_CONTEXT', 'GCP_NGFW_ENTERPRISE', 'GCP_RECAPTCHA_ENTERPRISE', 'GCP_RESOURCE_MANAGER_CONTEXT', 'GCP_RUN', 'GCP_SECURITYCENTER_POSTURE_VIOLATION', 'GCP_SECURITYCENTER_TOXIC_COMBINATION', 'GCP_SQL_CONTEXT', 'GCP_STORAGE_CONTEXT', 'GCP_SWP', 'GCP_THREAT_DETECTION', 'GCP_VPC_FLOW', 'GENETEC_AUDIT', 'GIGAMON', 'GIGYA_CIAM', 'GITGUARDIAN_ENTERPRISE', 'GITHUB', 'GITHUB_EVENTS', 'GITLAB', 'GLEAN', 'GLOBALSCAPE_SFTP', 'GLUSTER_FS', 'GMAIL_LOGS', 'GMV_CHECKER', 'GMV_CHECKER_CONTEXT', 'GOANYWHERE_MFT', 'GODADDY_DNS', 'GOLDILOCK', 'GOOGLE_ADS', 'GRAYHATWARFARE', 'GRAYLOG', 'GREATHORN', 'GREYNOISE', 'GTB_DLP', 'GUARDDUTY', 'GUARDICORE_CENTRA', 'GUARDIUM', 'GURUCUL', 'H3C_SWITCH', 'HADOOP', 'HALCYON', 'HALO', 'HAPROXY', 'HAPROXY_LOADBALANCER', 'HARBOR', 'HARFANGLAB_EDR', 'HASHICORP', 'HCL_BIGFIX', 'HCNET_ACCOUNT_ADAPTER', 'HIBOB', 'HIBP', 'HIRSCHMANN_SWITCH', 'HITACHI_CLOUD_PLATFORM', 'HITACHI_ID_PAM', 'HONEYD', 'HORNET_SECURITY', 'HPE_BLADESYSTEM_C7000', 'HPE_ILO', 'HPE_SAN', 'HP_ONEVIEW', 'HP_POLY', 'HP_PRINTER', 'HP_PROCURVE', 'HP_WOLF', 'HUAWEI_NAC', 'HUBSPOT_ACTIVITY', 'HUBSPOT_CRM', 'HUBSPOT_LOGIN', 'HYPR_MFA', 'IBM_3COM', 'IBM_AS400', 'IBM_CICS', 'IBM_CLEVERSAFE', 'IBM_DATAPOWER', 'IBM_DS8000', 'IBM_I', 'IBM_KNS', 'IBM_LTO', 'IBM_MAAS360', 'IBM_MAINFRAME_STORAGE', 'IBM_MQ_FILE_TRANSFER', 'IBM_QRADAR', 'IBM_SAFENET', 'IBM_SAM', 'IBM_SECURITY_VERIFY', 'IBM_SECURITY_VERIFY_SAAS', 'IBM_SIM', 'IBM_SOAR', 'IBM_SPECTRUM_PROTECT', 'IBM_SWITCH', 'IBM_TIVOLI', 'IBM_TRIRIGA', 'IBM_WEBSEAL', 'IBM_WEBSPHERE_APP_SERVER', 'IBM_WINCOLLECT', 'IBM_ZOS', 'IBM_ZSECURE_ALERT', 'IBOSS_WEBPROXY', 'IDECSI', 'IDRAC', 'IIS', 'ILLUMIO_CORE', 'IMAGENOW', 'IMANAGE_CLOUD', 'IMPERVA_ABP', 'IMPERVA_AUDIT_TRAIL', 'IMPERVA_CEF', 'IMPERVA_DB', 'IMPERVA_FLEXPROTECT', 'IMPERVA_SECURESPHERE', 'IMPERVA_SONAR', 'IMPERVA_WAF', 'IMPRIVATA_CONFIRM_ID', 'IMPRIVATA_IDG', 'IMPRIVATA_ONESIGN', 'INFINICO_NETWYVERN', 'INFINIDAT', 'INFOBLOX', 'INFOBLOX_DHCP', 'INFOBLOX_DNS', 'INFOBLOX_LOADBALANCER', 'INFOBLOX_NETMRI', 'INFOBLOX_RPZ', 'INFORMIX', 'INKY', 'INTERSYSTEMS_CACHE', 'INTRUDER_IO', 'INWEBO_MFA', 'IONIX', 'ION_SPECTRUM', 'IPSWITCH_MOVEIT_AUTOMATION', 'IPSWITCH_MOVEIT_TRANSFER', 'IPSWITCH_SFTP', 'IRONSCALES', 'ISC_DHCP', 'ISLAND_BROWSER', 'IVANTI_APP_CONTROL', 'IVANTI_CONNECT_SECURE', 'IVANTI_DEVICE_CONTROL', 'IVANTI_XTRACTION', 'JAMF', 'JAMF_COMPLIANCE_REPORTER', 'JAMF_NETWORK_TRAFFIC', 'JAMF_PRO', 'JAMF_PROTECT', 'JAMF_PRO_CONTEXT', 'JAMF_PRO_MDM', 'JAMF_TELEMETRY', 'JAMF_THREAT_EVENTS', 'JDE', 'JENKINS', 'JOURNALD', 'JUMPCLOUD_DAAS', 'JUMPCLOUD_DESKTOP', 'JUMPCLOUD_DIRECTORY_INSIGHTS', 'JUNIPER_FIREWALL', 'JUNIPER_IPS', 'JUNIPER_JUNOS', 'JUNIPER_MIST', 'JUNIPER_MX', 'JUNIPER_SDWAN', 'JUNIPER_VPN', 'JUPITER_ONE', 'KACE_SERVICE_DESK', 'KACE_SMA', 'KAMAILIO', 'KANDJI', 'KASEYA', 'KASPERSKY_AV', 'KASPERSKY_ENDPOINT', 'KEA_DHCP', 'KEEPALIVED', 'KEEPER', 'KEMP_LOADBALANCER', 'KERIOCONTROL', 'KEYCLOAK', 'KEYFACTOR', 'KEYSIGHT', 'KIBANA', 'KION', 'KISI', 'KITEWORKS', 'KNOWBE4_PHISHER', 'KOLIDE', 'KONG_GATEWAY', 'KUBERNETES_AUDIT', 'KUBERNETES_AUDIT_AZURE', 'KUBERNETES_AUTH_PROXY', 'KUBERNETES_NODE', 'KUSTOMER_CRM', 'KYRIBA', 'LACEWORK', 'LANSWEEPER', 'LASTPASS', 'LAUNCH_DARKLY', 'LB_ADC', 'LEANIX', 'LEANIX_CMDB', 'LENEL_ONGUARD', 'LEPIDE', 'LEXMARK_PRINTER', 'LIAISON_NUBRIDGES', 'LIBRAESVA_EMAIL', 'LIMACHARLIE_EDR', 'LINUX_DHCP', 'LINUX_SYSMON', 'LIRA', 'LOGICMONITOR', 'LOGONBOX', 'LOOKER_AUDIT', 'LOOKINGGLASS_IPS', 'LOOKING_GLASS_IOC', 'LOOKOUT_MOBILE_ENDPOINT_SECURITY', 'LSI_BMS', 'LUMEN_DDOS_HYPER', 'LUMETA', 'LUMOS', 'LXC_ORCHESTRATOR', 'MACOS', 'MACOS_ENDPOINT_SECURITY', 'MAILMARSHAL', 'MAILSCANNER', 'MALWAREBYTES_EDR', 'MAMBU', 'MANAGEENGINE_ENDPOINT', 'MANAGEENGINE_RAP', 'MANAGE_ENGINE_AD360', 'MANAGE_ENGINE_PAM360', 'MANAGE_ENGINE_PASSWORD_MANAGER', 'MANAGE_ENGINE_REPORTER_PLUS', 'MANDIANT_ASM_ENTITY', 'MANDIANT_ASM_ISSUE', 'MANDIANT_ASM_TECHNOLOGY', 'MANDIANT_CUSTOM_IOC', 'MANGOAPPS', 'MANHATTAN_WMS', 'MARIA_DB', 'MATERIAL_SECURITY', 'MATRIX_FRONTIER', 'MATTERMOST', 'MCAFEE_APP_CONTROL', 'MCAFEE_ATD', 'MCAFEE_DLP', 'MCAFEE_EDR', 'MCAFEE_EPO', 'MCAFEE_ESM', 'MCAFEE_IPS', 'MCAFEE_MVISION_CASB', 'MCAFEE_SKYHIGH_CASB', 'MCAFEE_SOLID_CORE', 'MCAFEE_UCE', 'MCAFEE_WEBPROXY', 'MCAFEE_WEB_PROTECTION', 'MEDIGATE_CMDB', 'MEDIGATE_IOT', 'MELISSA', 'MENANDMICE_DNS', 'MEND_IO', 'MENLO_SECURITY', 'META_MARKETING', 'MIASMA_SECRETSCANNER', 'MICROFOCUS_IMANAGER', 'MICROSEMI_NTP', 'MICROSOFT_ADS', 'MICROSOFT_ATA', 'MICROSOFT_CASB', 'MICROSOFT_CASB_CONTEXT', 'MICROSOFT_DEFENDER_CLOUD_ALERTS', 'MICROSOFT_DEFENDER_ENDPOINT', 'MICROSOFT_DEFENDER_IDENTITY', 'MICROSOFT_DYNAMICS_365', 'MICROSOFT_EASM', 'MICROSOFT_GRAPH_ACTIVITY_LOGS', 'MICROSOFT_GRAPH_ALERT', 'MICROSOFT_IAS', 'MICROSOFT_IDENTITY_PROTECTION', 'MICROSOFT_LAPS', 'MICROSOFT_NETLOGON', 'MICROSOFT_NPS', 'MICROSOFT_POWERBI_ACTIVITY_LOG', 'MICROSOFT_RISK_DETECTIONS', 'MICROSOFT_SCEP', 'MICROSOFT_SECURITY_ACTIONS', 'MICROSOFT_SECURITY_ALERTS', 'MICROSOFT_SENTINEL', 'MICROSOFT_SQL', 'MICROSOFT_SSTP', 'MICROSOFT_THREAT_INDICATORS', 'MIKROTIK_ROUTER', 'MIMECAST_ATTACHMENT_LOGS', 'MIMECAST_AUDIT_LOGS', 'MIMECAST_DLP_LOGS', 'MIMECAST_IMPERSONATION_LOGS', 'MIMECAST_MAIL', 'MIMECAST_URL_LOGS', 'MIMECAST_WEBPROXY', 'MINERVA_AV', 'MIRTH_NEXTGEN', 'MISP_IOC', 'MITEL_MCD', 'MOBILEIRON', 'MODE_ANALYTICS', 'MODSECURITY', 'MONDAY', 'MONGO_ATLAS_AUDIT', 'MONGO_DB', 'MOSYLE', 'MULESOFT', 'MULTICOM_SWITCH', 'MULTIPAY', 'MYSQL', 'NAGIOS', 'NASUNI_FILE_SERVICES', 'NCC_SCOUTSUITE', 'NCR_DIGITAL_INSIGHT_FSG', 'NCR_DIGITAL_INSIGHT_GL', 'NEO4J', 'NEOSEC', 'NESSUS', 'NETAPP_ONTAP', 'NETAPP_SAN', 'NETDISCO', 'NETDOCUMENTS', 'NETENRICH_ENTITY_BEHAVIOR', 'NETFILTER_IPTABLES', 'NETGEAR_SWITCH', 'NETIQ_ACCESS_MANAGER', 'NETIQ_EDIRECTORY', 'NETMOTION', 'NETSCOUT_OCI', 'NETSKOPE_ALERT', 'NETSKOPE_CASB', 'NETSKOPE_CLIENT', 'NETSKOPE_WEBPROXY', 'NETSURION_PROTECTWISE', 'NETWRIX', 'NET_SUITE', 'NEUSTAR_SITEPROTECT', 'NEW_RELIC', 'NEXTCLOUD_HUB', 'NEXTTHINK_FINDER', 'NEXUS_SONATYPE', 'NE_SILENT_LOG', 'NGINX', 'NIMBLE_OS', 'NINJAONE', 'NIST_NVD', 'NIX_SYSTEM', 'NNT_FIM', 'NOKIA_ROUTER', 'NONAME_API_SECURITY', 'NORD_LAYER', 'NORTEL_SWITCH', 'NOZOMI_GUARDIAN', 'NTOPNG', 'NUCLEUS_ASSET', 'NUCLEUS_VULNERABILITY', 'NUCLEUS_VULNERABILITY_DELTA', 'NUTANIX_FRAME', 'NUTANIX_PRISM', 'NXLOG_AGENT', 'NXLOG_FIM', 'NXLOG_MANAGER', 'NYANSA_EVENTS', 'OBSERVEIT', 'OBSIDIAN', 'OCI_AUDIT', 'OCI_FLOW', 'OCSF', 'OFFICE_365', 'OFFICE_365_MESSAGETRACE', 'OKERA_DAP', 'OKTA', 'OKTA_ACCESS_GATEWAY', 'OKTA_RADIUS', 'OKTA_USER_CONTEXT', 'ONAPSIS', 'ONBASE_CMS', 'ONEIDENTITY_ARS', 'ONEIDENTITY_CHANGE_AUDITOR', 'ONEIDENTITY_DEFENDER', 'ONEIDENTITY_TPAM', 'ONELOGIN_SSO', 'ONELOGIN_USER_CONTEXT', 'ONEPASSWORD', 'ONEPASSWORD_AUDIT_EVENTS', 'ONFIDO', 'OPENAM', 'OPENCANARY', 'OPENDJ', 'OPENGEAR', 'OPENLDAP', 'OPENPATH', 'OPENSSH', 'OPENTELEMETRY', 'OPENTEXT_FAX2MAIL', 'OPENVAS', 'OPEN_VPN', 'OPNSENSE', 'OPSWAT_KIOSK', 'OPSWAT_METADEFENDER', 'OPUS', 'ORACLE_CLOUD_AUDIT', 'ORACLE_DB', 'ORACLE_FUSION', 'ORACLE_HCM', 'ORACLE_NETSUITE', 'ORACLE_OUD', 'ORACLE_SSO_AUDIT', 'ORACLE_WEBLOGIC', 'ORCA', 'ORDR_IOT', 'OSCAR_CLAIMS', 'OSINT_IOC', 'OSIRIUM_PAM', 'OSQUERY_EDR', 'OSSEC', 'OUTPOST24', 'PACKETLIGHT_DWDM', 'PACKET_VIPER', 'PACOM_SYSTEMS', 'PAGERDUTY', 'PAGERDUTY_AUDIT', 'PALANTIR', 'PAN_CASB', 'PAN_CORTEX_XDR_EVENTS', 'PAN_DNS_SECURITY', 'PAN_EDR', 'PAN_FIREWALL', 'PAN_GLOBAL_PROTECT', 'PAN_IOC', 'PAN_IOT', 'PAN_PANORAMA', 'PAN_PRISMA_CA', 'PAN_PRISMA_CLOUD', 'PAN_XDR_MGMT_AUDIT', 'PAN_XSOAR', 'PAPER_CUT', 'PASSFORT', 'PASSIVE_DNS', 'PASSWORDSTATE', 'PAXTON_ACS', 'PCAP_SSL_CLIENT_HELLO', 'PEGA', 'PENTERA', 'PENTERA_ASV', 'PENTERA_LEEF', 'PEOPLESOFT', 'PEPLINK_FW', 'PEPLINK_LOADBALANCER', 'PEPLINK_ROUTER', 'PEPLINK_SWITCH', 'PERIMETERX_BOT_PROTECTION', 'PERIMETER_81', 'PFSENSE', 'PHISHEYE_ALERT', 'PHISHLABS', 'PING', 'PINGSAFE', 'PING_ACCESS', 'PING_DIRECTORY', 'PING_FEDERATE', 'PING_ONE', 'PING_SDK', 'PIVOTAL', 'PLASO', 'PLIXER_SCRUTINIZER', 'POMERIUM', 'PORTNOX_AUDIT', 'PORTNOX_CEF', 'POSTFIX_MAIL', 'POSTGRESQL', 'POWERSHELL', 'POWERSHELL_TRANSCRIPT', 'POWER_DNS', 'PREEMPT', 'PREEMPT_AUTH', 'PREVEIL_ENTERPRISE', 'PRISMA_SD_WAN', 'PROOFID', 'PROOFPOINT_CASB', 'PROOFPOINT_DLP', 'PROOFPOINT_MAIL', 'PROOFPOINT_MAIL_FILTER', 'PROOFPOINT_META', 'PROOFPOINT_ON_DEMAND', 'PROOFPOINT_SECURE_SHARE', 'PROOFPOINT_SECURITY_AWARENESS_TRAINING', 'PROOFPOINT_SENDMAIL_SENTRION', 'PROOFPOINT_SER', 'PROOFPOINT_TRAP', 'PROOFPOINT_WEB_BROWSER_ISOLATION', 'PROTEGRITY_DEFIANCE', 'PROWATCH', 'PROXMAX', 'PRTG_NETWORKMONITOR', 'PULSE_SECURE_VPN', 'PULSE_SECURE_VTM', 'PUPPET', 'PURE_STORAGE', 'QLIK_AUDIT', 'QNAP_NAS', 'QUALYS_ACTIVITY', 'QUALYS_ASSET_CONTEXT', 'QUALYS_CONTINUOUS_MONITORING', 'QUALYS_KNOWLEDGEBASE', 'QUALYS_SCAN', 'QUALYS_VIRTUAL_SCANNER', 'QUALYS_VM', 'QUEST_AD', 'QUEST_CA_AUDIT', 'QUEST_CHANGE_AUDITOR_EMC', 'QUEST_FILE_AUDIT', 'QUMULO_FS', 'RABBITMQ', 'RADIFLOW_IDS', 'RADIUS', 'RADWARE_ALTEON', 'RADWARE_DDOS', 'RADWARE_FIREWALL', 'RAD_ETX', 'RAPID7_INSIGHT', 'RAPID7_NEXPOSE', 'RAPID7_SECURITY_ONION', 'RARITAN_DOMINION', 'REALITEQ', 'RECORDED_FUTURE_IOC', 'RECORDIA', 'REDCANARY_CLOUD_PROTECTION_RAW', 'REDCANARY_EDR', 'REDHAT_DIRECTORY_SERVER', 'REDHAT_IM', 'REDHAT_JBOSS', 'REDHAT_KEYCLOAK', 'REDHAT_OPENSHIFT', 'REDHAT_SATELLITE', 'REDHAT_STACKROX', 'REDIS', 'REMEDIANT_SECUREONE', 'RH_ISAC_IOC', 'RIBBON_ANALYTICS_PLATFORM', 'RIBBON_SBC', 'RING_CENTRAL', 'RISKIQ_DIGITAL_FOOTPRINT', 'RSA_AUTH_MANAGER', 'RSA_NETWITNESS', 'RSA_SECURID', 'RUBRIK', 'RUBRIK_POLARIS', 'RUCKUS_WIRELESS', 'RUMBLE_NETWORK_DISCOVERY', 'SAFEBREACH', 'SAFECONNECT_NAC', 'SAILPOINT_IAM', 'SAIWALL_VPN', 'SALESFORCE', 'SALESFORCE_COMMERCE_CLOUD', 'SALESFORCE_CONTEXT', 'SANGFOR_NGAF', 'SAP_BTP', 'SAP_C4C', 'SAP_HANA', 'SAP_IDM', 'SAP_INSURANCE', 'SAP_NETWEAVER', 'SAP_SAST', 'SAP_SM20', 'SAP_SUCCESSFACTORS', 'SAP_WEBDISP', 'SAVIYNT_EIP', 'SCALITY_RING_AUDIT', 'SCCM', 'SECBERUS', 'SECUREAUTH_SSO', 'SECURELINK', 'SECURITYSCORECARD', 'SEMPERIS_ADFR', 'SEMPERIS_DSP', 'SENDGRID', 'SENDMAIL', 'SENDSAFELY', 'SENHASEGURA_PAM', 'SENTINELONE_ACTIVITY', 'SENTINELONE_ALERT', 'SENTINELONE_CF', 'SENTINEL_DV', 'SENTINEL_EDR', 'SENTRIGO', 'SEP', 'SEPPMAIL', 'SEQRITE_ENDPOINT', 'SERPICO', 'SERVICENOW_AUDIT', 'SERVICENOW_CMDB', 'SERVICENOW_ROLES', 'SERVICENOW_SECURITY', 'SEVCO_CMDB', 'SHAREPOINT', 'SHAREPOINT_ULS', 'SHIBBOLETH_IDP', 'SHODAN_IO', 'SHRUBBERY_TACACS', 'SIEBEL', 'SIEMENS_SIPASS', 'SIERRA_WIRELESS', 'SIGNAL_SCIENCES_WAF', 'SILVERFORT', 'SILVERPEAK_FIREWALL', 'SINGLE_STORE', 'SITE24X7', 'SITEMINDER_SSO', 'SKYBOX_FIREWALL_ASSURANCE', 'SKYSEA', 'SLACK_AUDIT', 'SMART_SIMPLE', 'SMBD', 'SNAPATTACK', 'SNARE_SOLUTIONS', 'SNIPE_IT', 'SNOOPY_LOGGER', 'SNORT_IDS', 'SNOWFLAKE', 'SNYK_SDLC', 'SOCOMEC_UPS', 'SOFTWARE_HOUSE_ACS', 'SOFTWARE_HOUSE_CCURE9000', 'SOLACE_AUDIT', 'SOLARIS_SYSTEM', 'SOLARWINDS_KSS', 'SOLARWINDS_SERV_U', 'SOLAR_SYSTEM', 'SONARQUBE', 'SONICWALL_SMA', 'SONIC_FIREWALL', 'SONRAI', 'SOPHOS_AV', 'SOPHOS_CAPSULE8', 'SOPHOS_CENTRAL', 'SOPHOS_DHCP', 'SOPHOS_EDR', 'SOPHOS_EMAIL', 'SOPHOS_FIREWALL', 'SOPHOS_URL', 'SOPHOS_UTM', 'SOTI_MOBICONTROL', 'SOURCEFIRE_IDS', 'SPAMHAUS', 'SPE', 'SPECTERX', 'SPIRION', 'SPLASHTOP', 'SPLUNK', 'SPLUNK_ATTACK_ANALYZER', 'SPLUNK_DNS', 'SPLUNK_PHANTOM', 'SPLUNK_TRUSTAR', 'SPUR_FEEDS', 'SPYCLOUD', 'SQUID_WEBPROXY', 'STAIRWELL_INCEPTION', 'STEALTHBITS_AUDIT', 'STEALTHBITS_DEFEND', 'STEALTHBITS_DLP', 'STEALTHBITS_PAM', 'STEELHEAD', 'STELLAR_CYBER', 'STIX', 'STORMSHIELD_FIREWALL', 'STREAMALERT', 'STRONGDM', 'STRONGSWAN_VPN', 'SUBLIMESECURITY', 'SUPERMICRO_IPMI', 'SUPERNA_EYEGLASS', 'SUREVIEW_SYSTEMS', 'SURICATA_EVE', 'SURICATA_IDS', 'SWIFT', 'SWIFT_AMH', 'SWIMLANE', 'SYMANTEC_CASB', 'SYMANTEC_DLP', 'SYMANTEC_EDR', 'SYMANTEC_EVENT_EXPORT', 'SYMANTEC_MAIL', 'SYMANTEC_VIP', 'SYMANTEC_WEB_ISOLATION', 'SYMANTEC_WSS', 'SYMPHONYAI', 'SYNOLOGY', 'SYSDIG', 'SYXSENSE', 'TABLEAU', 'TAILSCALE', 'TALON', 'TANIUM_ASSET', 'TANIUM_AUDIT', 'TANIUM_COMPLY', 'TANIUM_DEPLOY', 'TANIUM_DISCOVER', 'TANIUM_INSIGHT', 'TANIUM_INTEGRITY_MONITOR', 'TANIUM_PATCH', 'TANIUM_QUESTION', 'TANIUM_REVEAL', 'TANIUM_TANOS', 'TANIUM_TH', 'TANIUM_THREAT_RESPONSE', 'TCPWAVE_DDI', 'TEAMVIEWER', 'TECHNITIUM_DNS', 'TELEPORT_ACCESS_PLANE', 'TEMENOS_MANAGER_SYSTEMEVENT', 'TENABLE_ADS', 'TENABLE_IO', 'TENABLE_OT', 'TENABLE_SC', 'TENABLE_WAS', 'TERADICI_PCOIP', 'TERAMIND', 'TERRAFORM_ENTERPRISE', 'TESSIAN_PLATFORM', 'TETRAGON_EBPF_AUDIT_LOGS', 'TGDETECT', 'THALES_DIS', 'THALES_LUNA_HSM', 'THALES_MFA', 'THINKST_CANARY', 'THREATCONNECT_IOC', 'THREATLOCKER', 'THREATQ_IOC', 'THREATX_WAF', 'THYCOTIC', 'THYCOTIC_DEVOPS_SECRETVAULT', 'TIKTOK', 'TIPPING_POINT', 'TOMCAT', 'TRACEABLE_PLATFORM', 'TRAEFIK', 'TRELLIX_HX_ES', 'TRENDMICRO_APEX_CENTRAL', 'TRENDMICRO_APEX_ONE', 'TRENDMICRO_AV', 'TRENDMICRO_CLOUDAPPSECURITY', 'TRENDMICRO_CLOUDONE', 'TRENDMICRO_DDI', 'TRENDMICRO_DEEP_SECURITY', 'TRENDMICRO_EDGEIPS', 'TRENDMICRO_EDR', 'TRENDMICRO_EMAIL_SECURITY', 'TRENDMICRO_STELLAR', 'TRENDMICRO_VISION_ONE', 'TRENDMICRO_WEBPROXY', 'TRENDMICRO_WEBPROXY_DSM', 'TRIDIUM_NIAGARA_FRAMEWORK', 'TRIPP_LITE', 'TRIPWIRE_FIM', 'TRUEFORT', 'TRUENAS', 'TWILIO_AUDIT', 'TWILIO_AUTHY', 'TWINGATE', 'TYK_IO', 'UBIQUITI_ACCESSPOINT', 'UBIQUITI_FIREWALL', 'UBIQUITI_SWITCH', 'UDM', 'UIPATH', 'ULTRADNS', 'ULTRA_CYBERFENCE', 'UMBRELLA_DNS', 'UMBRELLA_FIREWALL', 'UMBRELLA_IP', 'UMBRELLA_WEBPROXY', 'UNBOUND_DNS', 'UNIFI_AP', 'UNIFI_SWITCH', 'UNIT21', 'UPGUARD', 'UPTYCS_EDR', 'VANDYKE_SFTP', 'VANGUARD', 'VARONIS', 'VECTOR_DEV', 'VECTRA_DETECT', 'VECTRA_PROTECT', 'VECTRA_STREAM', 'VEEAM', 'VELO_FIREWALL', 'VENAFI', 'VERCARA', 'VERIDIUM_ID', 'VERITAS_NETBACKUP', 'VERIZON_NDR', 'VERKADA', 'VERSA_FIREWALL', 'VIACONTROL', 'VIRSEC_EVENT', 'VIRSEC_THREAT', 'VIRTRU_EMAIL_ENCRYPTION', 'VIRUSTOTAL_THREAT_HUNTER', 'VITALQIP', 'VMRAY_FLOG_XML', 'VMWARE_ARIA_LOGS', 'VMWARE_AVINETWORKS_IWAF', 'VMWARE_AVI_VANTAGE', 'VMWARE_CD', 'VMWARE_ESX', 'VMWARE_HCX', 'VMWARE_HORIZON', 'VMWARE_NSX', 'VMWARE_NSX_AVI', 'VMWARE_SDDC', 'VMWARE_SDWN_EVENTS', 'VMWARE_TANZU', 'VMWARE_UNIFIED_ACCESS_GATEWAY', 'VMWARE_VCENTER', 'VMWARE_VREALIZE', 'VMWARE_VSHIELD', 'VMWARE_WORKSPACE_ONE', 'VOLTAGE', 'VONAGE', 'VORMETRIC', 'VSFTPD', 'VSFTPD_AUDIT', 'VYOS', 'WALLARM_NOTIFICATIONS', 'WALLIX_BASTION', 'WALLIX_EPM', 'WALLIX_PAM', 'WATCHGUARD', 'WATCHGUARD_EDR', 'WATERFALL_DSM', 'WAZUH', 'WEBEX_SAAS', 'WEBMARSHAL', 'WHITECLOUD_EDR', 'WINDCHILL', 'WINDOWS_AD', 'WINDOWS_APPLOCKER', 'WINDOWS_DEFENDER_ATP', 'WINDOWS_DEFENDER_AV', 'WINDOWS_DHCP', 'WINDOWS_DNS', 'WINDOWS_FIREWALL', 'WINDOWS_HYPERV', 'WINDOWS_NET_POLICY_SERVER', 'WINDOWS_SYSMON', 'WINDOWS_WFP', 'WINEVTLOG', 'WINEVTLOG_XML', 'WINSCP', 'WITHSECURE_CLOUD', 'WITHSECURE_ELEMENTS', 'WIZ_IO', 'WORDPRESS_CMS', 'WORDPRESS_SIMPLE_HISTORY', 'WORKATO', 'WORKDAY', 'WORKDAY_AUDIT', 'WORKDAY_USER_ACTIVITY', 'WORKSPACE_ACTIVITY', 'WORKSPACE_ALERTS', 'WORKSPACE_CHROMEOS', 'WORKSPACE_GROUPS', 'WORKSPACE_MOBILE', 'WORKSPACE_PRIVILEGES', 'WORKSPACE_USERS', 'WORKSPOT_CONTROL', 'WP_ENGINE', 'WS_FTP', 'WTI_CONSOLE_SERVERS', 'XITING_XAMS', 'YAMAHA_ROUTER', 'YSOFT_DSM', 'YUBICO_OTP', 'ZABBIX', 'ZENDESK_CRM', 'ZEROFOX_PLATFORM', 'ZIMPERIUM', 'ZIX_EMAIL_ENCRYPTION', 'ZOHO_AUDIT', 'ZOOM_OPERATION_LOGS', 'ZSCALER_CASB', 'ZSCALER_DECEPTION', 'ZSCALER_DIGITAL_EXPERIENCE', 'ZSCALER_DLP', 'ZSCALER_DNS', 'ZSCALER_FIREWALL', 'ZSCALER_INTERNET_ACCESS', 'ZSCALER_NSS_FEEDS', 'ZSCALER_TUNNEL', 'ZSCALER_VPN', 'ZSCALER_WEBPROXY', 'ZSCALER_ZCC', 'ZSCALER_ZDX', 'ZSCALER_ZPA', 'ZSCALER_ZPA_AUDIT', 'ZUORA_APP_LOGS'] {allow-input: true}

#@markdown Choose show status to just see the status of the parser (live, validating, rejected, etc), otherwise the parser body will be saved (or downloaded) instead.
show_status = False #@param {type: "boolean"}
download_file = False #@param {type: "boolean"}

# Construct the URL
uri = f"https://{region_prefix}backstory.googleapis.com/v1/tools/cbnParsers/{config_id.upper()}"

# send the request and process the response
resp = session.get(uri)
json_resp = resp.json()
if resp.status_code > 299:
  pprint(json_resp.get('error').get('message'))
else:
  if not show_status:
    requested_parser = base64.b64decode(json_resp['config'])
    f = open(config_id + ".conf", "wb")
    f.write(requested_parser)
    f.close()
    print("Parser written to {}.conf".format(config_id))
    if download_file:
      files.download("{}.conf".format(config_id))
  else:
    pprint(json_resp)

Parser written to IBM_CLOUD_ACTIVITY_TRACKER.conf


In [ ]:
#@title Submit a parser
#@markdown This endpoint enables you to submit a customer-specific parser configuration for a specific parser. The validation process requires at least 10,000 raw logs for each LogType and that event timestamps are no older than 30 days, otherwise it will fail. Make sure there are at least 10,000 raw logs for the LogType with event timestamps no older than 30 days from the time you submit the parser configuration.
log_type = 'ASSET_STATIC_IP' #@param {type: "string"}
author_name = "Eugene Dimarsky" #@param {type: "string"}
#@markdown Check the below box to upload a parser or leave unchecked to specify a filename in the `parser_filename` variable below that is already in the colab instance.
upload_file = False #@param {type: "boolean"}
parser_filename = "parser.conf" #@param {type:"string"}
if upload_file:
  print("Please select the parser file to upload")
  uploaded = files.upload()

  parser_filename = list(uploaded.keys())[0]

with open(parser_filename, "rb") as parser_file:
  parser = parser_file.read()

skipValidationOnNoLogs = True #@param {type: "boolean"}

uri = f"https://{region_prefix}backstory.googleapis.com/v1/tools/cbnParsers"

# conf_file = '/_data/nginx.conf'


# create the body with the parser configuration, logType identifier, and author
request_body = {
'config': base64.urlsafe_b64encode(parser).decode("ascii"),
'log_type': log_type,
'author': author_name
}

if skipValidationOnNoLogs:
  request_body['skipValidationOnNoLogs'] = True


# encode the body and set the Content-Type
body = urllib.parse.urlencode(request_body)
r_headers = {}
r_headers.update({'Content-type' : 'application/x-www-form-urlencoded'})

# send the request and process the response
resp = session.post(uri, data=body, headers=r_headers)
json_resp = resp.json()
if resp.status_code > 299:
  pprint(json_resp['error'])
else:
  pprint(json_resp)

In [ ]:
#@title Archive a parser
#@markdown Sets the state of a specified parser configuration to ARCHIVED and returns information about the parser configuration before changing the state.
config_id = "0b598c6e-9e91-4fb9-a52d-481e17a7f13c" #@param {type: "string"}

uri = f"https://{region_prefix}backstory.googleapis.com/v1/tools/cbnParsers/{config_id}:archive"

# send the request and process the response
resp = session.post(uri)
json_resp = resp.json()
if resp.status_code > 299:
  pprint(json_resp.get('error').get('message'))
else:
  pprint(json_resp)

In [ ]:
#@title List CBN Parser Errors

#@markdown When customer-specific and default parsers encounter errors, they are captured and saved. This endpoint retrieves the errors generated by a specific logType over a defined time range. It returns a maximum of 1000 errors with each request.

#@markdown Config_id can either be an ID of a custom parser or the name of a default parser (for example: OKTA)
# config ID list taken from ingest API get support log types endpoint
logtype = "CLOUDFLARE" #@param ['ABNORMAL_SECURITY', 'ABSOLUTE', 'ACALVIO', 'ACCELLION', 'ACQUIA_CLOUD_PLATFORM', 'ACTIVE_SYNC', 'ADAUDIT_PLUS', 'ADFS', 'ADMANAGER_PLUS', 'ADOBE_COMMERCE', 'ADSELFSERVICE_PLUS', 'ADTRAN_NETVANTA', 'AIRDEFENSE', 'AIRLOCK_DIGITAL', 'AIRWATCH', 'AIX_SYSTEM', 'AI_HUNTER', 'AKAMAI_CLOUD_MONITOR', 'AKAMAI_DDOS', 'AKAMAI_DHCP', 'AKAMAI_DNS', 'AKAMAI_EAA', 'AKAMAI_ETP', 'AKAMAI_SIEM_CONNECTOR', 'AKAMAI_WAF', 'AKEYLESS_VAULT', 'ALERTLOGIC_NOTIFICATIONS', 'ALGOSEC', 'ALLOT_NETENFORCER', 'ALVEO_RDM', 'ANOMALI_IOC', 'ANSIBLE_AWX', 'APACHE', 'APACHE_KAFKA_AUDIT', 'APACHE_SPAMASSASSIN', 'APC_NETBOTZ', 'APC_SMART_UPS', 'APC_STRUXUREWARE', 'APPGATE_SDP', 'APPIAN_CLOUD', 'APPOMNI', 'APTOS_EOM', 'AQUA_SECURITY', 'ARBOR_EDGE_DEFENSE', 'ARBOR_SIGHTLINE', 'ARCHER_IRM', 'ARCSIGHT_CEF', 'AREA1', 'ARGO_CD', 'ARISTA_SWITCH', 'ARKIME_PCAP', 'ARMIS', 'ARMORBLOX_ESC', 'ARMOR_ANYWHERE', 'ARRAYNETWORKS_VPN', 'ARRAY_NETWORKS_WAF', 'ARUBA_AIRWAVE', 'ARUBA_IPS', 'ARUBA_WIRELESS', 'ARXAN_THREAT_ANALYTICS', 'ASANA', 'ASOC_ALERT', 'ASSET_STATIC_IP', 'ATLASSIAN_CONFLUENCE', 'ATLASSIAN_JIRA', 'ATTIVO', 'ATT_NETBOND', 'AUDITD', 'AUTHENTIC8_SILO', 'AUTHX', 'AUTHX_USER_CONTEXT', 'AUTH_ZERO', 'AUTOMATION_ANYWHERE', 'AUTOMOX_EPM', 'AVANAN_EMAIL', 'AVATIER', 'AVAYA_AURA', 'AVAYA_IVR', 'AVAYA_WIRELESS', 'AVIATRIX', 'AWAKE_NDR', 'AWS_AURORA', 'AWS_CLOUDFRONT', 'AWS_CLOUDTRAIL', 'AWS_CLOUDWATCH', 'AWS_CONFIG', 'AWS_CONTROL_TOWER', 'AWS_ELB', 'AWS_KMS', 'AWS_MACIE', 'AWS_REDSHIFT', 'AWS_ROUTE_53', 'AWS_S3_SERVER_ACCESS', 'AWS_SECURITY_HUB', 'AWS_SES', 'AWS_SESSION_MANAGER', 'AWS_VPC_FLOW', 'AWS_WAF', 'AXIS_OS', 'AXONIUS', 'AZURE', 'AZURE_ACTIVITY', 'AZURE_AD', 'AZURE_AD_AUDIT', 'AZURE_AD_CONTEXT', 'AZURE_ATP', 'AZURE_COSMOS_DB', 'AZURE_DEVOPS', 'AZURE_FIREWALL', 'AZURE_MDM_INTUNE', 'AZURE_NSG_FLOW', 'AZURE_RESOURCE_LOGS', 'AZURE_SECURITY_CENTER', 'AZURE_SQL', 'AZURE_WAF', 'BALABIT', 'BARRACUDA_CLOUDGEN_ACCESS', 'BARRACUDA_EMAIL', 'BARRACUDA_FIREWALL', 'BARRACUDA_WAF', 'BETTERCLOUD', 'BEYONDTRUST_BEYONDINSIGHT', 'BEYONDTRUST_CPB', 'BEYONDTRUST_ENDPOINT', 'BEYONDTRUST_PI', 'BEYONDTRUST_REMOTE_ACCESS', 'BIGSWITCH_BCF', 'BIND_DNS', 'BITDEFENDER', 'BLUECAT_DDI', 'BLUECAT_EDGE', 'BLUECOAT_WEBPROXY', 'BLUE_PRISM', 'BMC_AMI_DEFENDER', 'BMC_CONTROL_M', 'BMC_HELIX_DISCOVERY', 'BOMGAR', 'BOX', 'BRICATA_NDR', 'BRIVO', 'BROADCOM_CEM', 'BROADCOM_SSL_VA', 'BROCADE_SERVERIRON', 'BROCADE_SWITCH', 'BRO_DHCP', 'BRO_HTTP', 'BRO_JSON', 'BRO_TSV', 'BT_IPCONTROL', 'CAMEYO_BYO_CLOUD', 'CASSANDRA', 'CATO_NETWORKS', 'CATO_SDWAN', 'CA_ACCESS_CONTROL', 'CA_ACF2', 'CA_LDAP', 'CA_SSO_WEB', 'CB_APP_CONTROL', 'CB_EDR', 'CENTRIFY_SSO', 'CENTRIPETAL_IOC', 'CEQUENCE_BOT_DEFENSE', 'CERBERUS_FTP', 'CHECKPOINT_CLOUDGUARD', 'CHECKPOINT_EDR', 'CHECKPOINT_EMAIL', 'CHECKPOINT_FIREWALL', 'CHECKPOINT_HARMONY', 'CIRCLECI', 'CISCO_ACE', 'CISCO_ACI', 'CISCO_ACS', 'CISCO_AMP', 'CISCO_APIC', 'CISCO_ASA_FIREWALL', 'CISCO_CLOUDLOCK_CASB', 'CISCO_CTS', 'CISCO_DHCP', 'CISCO_DNAC', 'CISCO_DNS', 'CISCO_EMAIL_SECURITY', 'CISCO_FIREPOWER_FIREWALL', 'CISCO_FIRESIGHT', 'CISCO_IOS', 'CISCO_ISE', 'CISCO_MERAKI', 'CISCO_NX_OS', 'CISCO_PRIME', 'CISCO_ROUTER', 'CISCO_SDWAN', 'CISCO_SECURE_MALWARE_ANALYTICS', 'CISCO_SECURE_WORKLOAD', 'CISCO_SMA', 'CISCO_STEALTHWATCH', 'CISCO_SWITCH', 'CISCO_TACACS', 'CISCO_UCM', 'CISCO_UCS', 'CISCO_VCS', 'CISCO_VPN', 'CISCO_WIPS', 'CISCO_WIRELESS', 'CISCO_WSA', 'CIS_ALBERT_ALERT', 'CITRIX_ANALYTICS', 'CITRIX_MONITOR', 'CITRIX_NETSCALER', 'CITRIX_NETSCALER_WEB_LOGS', 'CITRIX_SDWAN', 'CITRIX_SESSION_METADATA', 'CITRIX_STOREFRONT', 'CITRIX_WAF', 'CITRIX_WEB_GATEWAY', 'CITRIX_WORKSPACE', 'CITRIX_XENCENTER', 'CLAM_AV', 'CLEARPASS', 'CLEARSENSE', 'CLICK_STUDIOS_PASSWORDSTATE', 'CLOUDFLARE', 'CLOUDFLARE_AUDIT', 'CLOUDFLARE_BOT_MANAGEMENT', 'CLOUDFLARE_WAF', 'CLOUDGENIX_SDWAN', 'CLOUDIAN_HYPERSTORE', 'CLOUDM', 'CLOUDPASSAGE_CSM', 'CLOUDPASSAGE_FIM', 'CLOUDPASSAGE_LIDS', 'CLOUDPASSAGE_SVM', 'CLOUD_IDENTITY_CONTEXT', 'CLOUD_PASSAGE', 'CMD', 'CODE42', 'CODE42_INCYDR', 'CODE_WORLDWIDE', 'COFENSE_TRIAGE', 'COFENSE_VISION', 'COHESITY', 'COMMVAULT', 'COMODO_AV', 'CONFLUENT_AUDIT', 'CONNECTWISE_CONTROL', 'CORELIGHT', 'CORTEX_XDR', 'COVID_CTC_IOC', 'CRADLEPOINT_NETCLOUD', 'CROWDSTRIKE_IOC', 'CSV_CUSTOM_CMDB', 'CSV_CUSTOM_IOC', 'CS_CEF_EDR', 'CS_DETECTS', 'CS_EDR', 'CS_STREAM', 'CTERA_DRIVE', 'CUSTOMER_ALERT', 'CUSTOM_APPLICATION_ACCESS', 'CUSTOM_DNS', 'CUSTOM_HOST_FORENSICS', 'CUSTOM_SECURITY_DATA_ANALYTICS', 'CYBERARK', 'CYBERARK_PRIVILEGE_CLOUD', 'CYBEREASON_EDR', 'CYBERHAVEN_DDR', 'CYCODE', 'CYLANCE', 'CYLANCE_PROTECT', 'CYOLO_ZTNA', 'D3_BANKING', 'D3_SECURITY', 'DARKTRACE', 'DATADOG', 'DATALOCKER_SAFECONSOLE', 'DATAWATCH', 'DATTO_FILE_PROTECTION', 'DB2_DB', 'DEEPFENCE', 'DEEP_INSTINCT_EDR', 'DELINEA_PRIVILEGE_MANAGER', 'DELINEA_SECRET_SERVER', 'DELINEA_SERVER_SUITE', 'DELL_EMC_AVAMAR', 'DELL_EMC_CLOUDLINK', 'DELL_EMC_DATA_DOMAIN', 'DELL_EMC_NAS', 'DELL_EMC_UNITY', 'DELL_OPENMANAGE', 'DELL_SWITCH', 'DESIGN_PROFIT_CENTRAL_SERVER', 'DEVOLUTIONS_RDM', 'DHS_IOC', 'DIGITALARTS_IFILTER', 'DIGITALGUARDIAN_DLP', 'DIGITALGUARDIAN_EDR', 'DIGITAL_SHADOWS_IOC', 'DIGITAL_SHADOWS_SEARCHLIGHT', 'DMP_ENTRE', 'DOCKER', 'DOMO', 'DREMIO_DATA_LAKEHOUSE', 'DROPBOX', 'DRUPAL', 'DUO_ADMIN', 'DUO_AUTH', 'DUO_CASB', 'DUO_CONTEXT', 'DUO_NETWORK_GATEWAY', 'DUO_TELEPHONY', 'DUO_USER_CONTEXT', 'DYNATRACE', 'E2_SOLUTIONS', 'EATON_UPS', 'ECAR', 'ECAR_BRO', 'EDGIO_CDN', 'EFFICIENTIP_DDI', 'EIQ_EDR', 'ELASTIC_AUDITBEAT', 'ELASTIC_FILEBEAT', 'ELASTIC_METRICBEAT', 'ELASTIC_PACKETBEATS', 'ELASTIC_SEARCH', 'ELASTIC_WINLOGBEAT', 'EMERSON_FIREWALL', 'ENDGAME_EDR', 'ENDPOINT_PROTECTOR_DLP', 'ENTRUST_HSM', 'ENTRUST_NTP_SERVER', 'ENTRUST_SECRETS_VAULT', 'EPIC', 'ESET_AV', 'ESET_EDR', 'ESET_IOC', 'ESHARE_PLATFORM', 'ESTAR', 'ETQ_RELIANCE', 'ET_PRO_IOC', 'EXABEAM_FUSION_XDR', 'EXCHANGE_MAIL', 'EXTRAHOP', 'EXTRAHOP_DHCP', 'EXTRAHOP_DNS', 'EXTREME_MANAGEMENT', 'EXTREME_SWITCH', 'F5_ASM', 'F5_BIGIP_LTM', 'F5_BOT', 'F5_DNS', 'F5_SHAPE', 'F5_VPN', 'FALCO_IDS', 'FASTLY_WAF', 'FIDELIS_ENDPOINT', 'FIDELIS_NETWORK', 'FILEZILLA_FTP', 'FILE_SCANNING_FRAMEWORK', 'FIREEYE_ALERT', 'FIREEYE_EMPS', 'FIREEYE_ETP', 'FIREEYE_HX', 'FIREEYE_HX_AUDIT', 'FIREEYE_NX', 'FIREMON_FIREWALL', 'FLASHPOINT_IOC', 'FLUENTD', 'FORCEPOINT_CASB', 'FORCEPOINT_DLP', 'FORCEPOINT_FIREWALL', 'FORCEPOINT_WEBPROXY', 'FORESCOUT_NAC', 'FORSETI', 'FORTANIX_DSM', 'FORTINET_DHCP', 'FORTINET_FIREWALL', 'FORTINET_FORTIANALYZER', 'FORTINET_FORTIAUTHENTICATOR', 'FORTINET_FORTICLIENT', 'FORTINET_FORTIEDR', 'FORTINET_FORTINAC', 'FORTINET_SANDBOX', 'FORTINET_WEBPROXY', 'FOX_IT_STIX', 'FREEIPA', 'FREERADIUS', 'FRONTLINE_VM', 'FUTUREX_HSM', 'GCP_APIGEE', 'GCP_CLOUDIDENTITY_DEVICES', 'GCP_CLOUDIDENTITY_DEVICEUSERS', 'GCP_CLOUDIOT', 'GCP_CLOUDSQL', 'GCP_COMPUTE', 'GCP_IDS', 'GCP_LOADBALANCING', 'GCP_RECAPTCHA_ENTERPRISE', 'GCP_RUN', 'GCP_THREAT_DETECTION', 'GCP_VPC_FLOW', 'GIGAMON', 'GITHUB', 'GITLAB', 'GLOBALSCAPE_SFTP', 'GLUSTER_FS', 'GMAIL_LOGS', 'GMV_CHECKER', 'GMV_CHECKER_CONTEXT', 'GODADDY_DNS', 'GREATHORN', 'GTB_DLP', 'GUARDDUTY', 'GUARDICORE_CENTRA', 'GUARDIUM', 'H3C_SWITCH', 'HADOOP', 'HAPROXY', 'HAPROXY_LOADBALANCER', 'HASHICORP', 'HCL_BIGFIX', 'HCNET_ACCOUNT_ADAPTER', 'HIRSCHMANN_SWITCH', 'HITACHI_CLOUD_PLATFORM', 'HITACHI_ID_PAM', 'HONEYD', 'HPE_ILO', 'HPE_SAN', 'HP_PRINTER', 'HP_PROCURVE', 'HYPR_MFA', 'IBM_AS400', 'IBM_CICS', 'IBM_DATAPOWER', 'IBM_MAAS360', 'IBM_MQ_FILE_TRANSFER', 'IBM_QRADAR', 'IBM_SAFENET', 'IBM_SECURITY_VERIFY', 'IBM_SOAR', 'IBM_SPECTRUM_PROTECT', 'IBM_SWITCH', 'IBM_TIVOLI', 'IBM_WEBSEAL', 'IBM_WEBSPHERE_APP_SERVER', 'IBM_ZOS', 'IBM_ZSECURE_ALERT', 'IBOSS_WEBPROXY', 'IDRAC', 'IIS', 'ILLUMIO_CORE', 'IMANAGE_CLOUD', 'IMPERVA_DB', 'IMPERVA_FLEXPROTECT', 'IMPERVA_SECURESPHERE', 'IMPERVA_SONAR', 'IMPERVA_WAF', 'IMPRIVATA_CONFIRM_ID', 'IMPRIVATA_IDG', 'IMPRIVATA_ONESIGN', 'INFOBLOX', 'INFOBLOX_DHCP', 'INFOBLOX_DNS', 'INFOBLOX_LOADBALANCER', 'INFOBLOX_NETMRI', 'INFOBLOX_RPZ', 'INFORMIX', 'INTERSYSTEMS_CACHE', 'INWEBO_MFA', 'IPSWITCH_MOVEIT_AUTOMATION', 'IPSWITCH_MOVEIT_TRANSFER', 'IPSWITCH_SFTP', 'ISC_DHCP', 'JAMF', 'JAMF_COMPLIANCE_REPORTER', 'JAMF_PROTECT', 'JDE', 'JENKINS', 'JUMPCLOUD_DAAS', 'JUNIPER_FIREWALL', 'JUNIPER_IPS', 'JUNIPER_JUNOS', 'JUNIPER_MX', 'KAMAILIO', 'KANDJI', 'KASEYA', 'KASPERSKY_AV', 'KEA_DHCP', 'KEEPER', 'KEMP_LOADBALANCER', 'KIBANA', 'KISI', 'KNOWBE4_PHISHER', 'KOLIDE', 'KONG_GATEWAY', 'KUBERNETES_AUDIT', 'KUBERNETES_AUTH_PROXY', 'KUBERNETES_NODE', 'KYRIBA', 'LACEWORK', 'LASTPASS', 'LENEL_ONGUARD', 'LEXMARK_PRINTER', 'LIAISON_NUBRIDGES', 'LIBRAESVA_EMAIL', 'LIMACHARLIE_EDR', 'LINUX_DHCP', 'LINUX_SYSMON', 'LOGICMONITOR', 'LOOKINGGLASS_IPS', 'LOOKING_GLASS_IOC', 'LSI_BMS', 'LUMEN_DDOS_HYPER', 'LXC_ORCHESTRATOR', 'MACOS', 'MAILSCANNER', 'MALWAREBYTES_EDR', 'MANAGE_ENGINE_AD360', 'MANAGE_ENGINE_PASSWORD_MANAGER', 'MANAGE_ENGINE_REPORTER_PLUS', 'MANDIANT_IOC', 'MANGOAPPS', 'MARIA_DB', 'MATERIAL_SECURITY', 'MATRIX_FRONTIER', 'MCAFEE_ATD', 'MCAFEE_DLP', 'MCAFEE_EDR', 'MCAFEE_EPO', 'MCAFEE_ESM', 'MCAFEE_IPS', 'MCAFEE_MVISION_CASB', 'MCAFEE_SKYHIGH_CASB', 'MCAFEE_UCE', 'MCAFEE_WEBPROXY', 'MCAFEE_WEB_PROTECTION', 'MEDIGATE_CMDB', 'MEDIGATE_IOT', 'MENANDMICE_DNS', 'MENLO_SECURITY', 'MICROFOCUS_IMANAGER', 'MICROSEMI_NTP', 'MICROSOFT_ATA', 'MICROSOFT_CASB', 'MICROSOFT_DEFENDER_ENDPOINT', 'MICROSOFT_DEFENDER_IDENTITY', 'MICROSOFT_GRAPH_ALERT', 'MICROSOFT_NETLOGON', 'MICROSOFT_SCEP', 'MICROSOFT_SENTINEL', 'MICROSOFT_SQL', 'MICROSOFT_SSTP', 'MIMECAST_MAIL', 'MIMECAST_WEBPROXY', 'MINERVA_AV', 'MISP_IOC', 'MOBILEIRON', 'MONGO_DB', 'MULESOFT', 'MYSQL', 'NAGIOS', 'NASUNI_FILE_SERVICES', 'NCC_SCOUTSUITE', 'NCR_DIGITAL_INSIGHT_FSG', 'NCR_DIGITAL_INSIGHT_GL', 'NETAPP_ONTAP', 'NETAPP_SAN', 'NETDISCO', 'NETFILTER_IPTABLES', 'NETIQ_ACCESS_MANAGER', 'NETIQ_EDIRECTORY', 'NETMOTION', 'NETSKOPE_ALERT', 'NETSKOPE_CASB', 'NETSKOPE_WEBPROXY', 'NETSURION_PROTECTWISE', 'NEW_RELIC', 'NEXTCLOUD_HUB', 'NGINX', 'NIMBLE_OS', 'NIST_NVD', 'NIX_SYSTEM', 'NUCLEUS_ASSET', 'NUCLEUS_VULNERABILITY', 'NUCLEUS_VULNERABILITY_DELTA', 'NUTANIX_FRAME', 'NUTANIX_PRISM', 'NXLOG_MANAGER', 'OBSERVEIT', 'OBSIDIAN', 'OFFICE_365', 'OKERA_DAP', 'OKTA', 'OKTA_ACCESS_GATEWAY', 'OKTA_RADIUS', 'OKTA_USER_CONTEXT', 'ONBASE_CMS', 'ONEIDENTITY_ARS', 'ONEIDENTITY_CHANGE_AUDITOR', 'ONEIDENTITY_DEFENDER', 'ONEIDENTITY_TPAM', 'ONELOGIN_SSO', 'ONELOGIN_USER_CONTEXT', 'ONEPASSWORD', 'OPENAM', 'OPENDJ', 'OPENGEAR', 'OPENLDAP', 'OPENPATH', 'OPENSSH', 'OPENTEXT_FAX2MAIL', 'OPEN_VPN', 'ORACLE_CLOUD_AUDIT', 'ORACLE_DB', 'ORACLE_WEBLOGIC', 'ORCA', 'ORDR_IOT', 'OSCAR_CLAIMS', 'OSINT_IOC', 'OSIRIUM_PAM', 'OSQUERY_EDR', 'OSSEC', 'PACOM_SYSTEMS', 'PAGERDUTY', 'PAN_CASB', 'PAN_CORTEX_XDR_EVENTS', 'PAN_EDR', 'PAN_FIREWALL', 'PAN_GLOBAL_PROTECT', 'PAN_IOC', 'PAN_PRISMA_CLOUD', 'PAN_XSOAR', 'PASSIVE_DNS', 'PCAP_SSL_CLIENT_HELLO', 'PENTERA_ASV', 'PEOPLESOFT', 'PEPLINK_LOADBALANCER', 'PEPLINK_ROUTER', 'PEPLINK_SWITCH', 'PERIMETERX_BOT_PROTECTION', 'PFSENSE', 'PHISHEYE_ALERT', 'PING', 'PING_FEDERATE', 'PIVOTAL', 'PLASO', 'PLIXER_SCRUTINIZER', 'POSTFIX_MAIL', 'POWERSHELL', 'POWER_DNS', 'PREEMPT', 'PREEMPT_AUTH', 'PREVEIL_ENTERPRISE', 'PROOFID', 'PROOFPOINT_CASB', 'PROOFPOINT_MAIL', 'PROOFPOINT_MAIL_FILTER', 'PROOFPOINT_ON_DEMAND', 'PROOFPOINT_TRAP', 'PROOFPOINT_WEB_BROWSER_ISOLATION', 'PROTEGRITY_DEFIANCE', 'PROWATCH', 'PULSE_SECURE_VPN', 'PULSE_SECURE_VTM', 'PUPPET', 'PURE_STORAGE', 'QNAP_NAS', 'QUALYS_CONTINUOUS_MONITORING', 'QUALYS_VM', 'QUEST_AD', 'RADIUS', 'RADWARE_DDOS', 'RADWARE_FIREWALL', 'RAPID7_INSIGHT', 'RAPID7_NEXPOSE', 'RECORDED_FUTURE_IOC', 'REDCANARY_CLOUD_PROTECTION_RAW', 'REDCANARY_EDR', 'REDHAT_DIRECTORY_SERVER', 'REDHAT_IM', 'REDHAT_KEYCLOAK', 'REDHAT_OPENSHIFT', 'REDHAT_STACKROX', 'REMEDIANT_SECUREONE', 'RH_ISAC_IOC', 'RIBBON_ANALYTICS_PLATFORM', 'RIBBON_SBC', 'RING_CENTRAL', 'RISKIQ_DIGITAL_FOOTPRINT', 'RSA_AUTH_MANAGER', 'RSA_NETWITNESS', 'RSA_SECURID', 'RUBRIK', 'RUBRIK_POLARIS', 'RUCKUS_WIRELESS', 'RUMBLE_NETWORK_DISCOVERY', 'SAFECONNECT_NAC', 'SAILPOINT_IAM', 'SALESFORCE', 'SALESFORCE_CONTEXT', 'SAP_C4C', 'SAP_HANA', 'SAP_IDM', 'SAP_INSURANCE', 'SAP_SUCCESSFACTORS', 'SAVIYNT_EIP', 'SECBERUS', 'SECUREAUTH_SSO', 'SECURELINK', 'SECURITYSCORECARD', 'SEMPERIS_ADFR', 'SEMPERIS_DSP', 'SENDMAIL', 'SENTINELONE_ALERT', 'SENTINEL_DV', 'SENTINEL_EDR', 'SEP', 'SERVICENOW_AUDIT', 'SERVICENOW_CMDB', 'SERVICENOW_ROLES', 'SERVICENOW_SECURITY', 'SEVCO_CMDB', 'SHAREPOINT', 'SHIBBOLETH_IDP', 'SHODAN_IO', 'SHRUBBERY_TACACS', 'SIEMENS_SIPASS', 'SIGNAL_SCIENCES_WAF', 'SILVERFORT', 'SILVERPEAK_FIREWALL', 'SITEMINDER_SSO', 'SLACK_AUDIT', 'SNARE_SOLUTIONS', 'SNIPE_IT', 'SNOOPY_LOGGER', 'SNORT_IDS', 'SNOWFLAKE', 'SOFTWARE_HOUSE_ACS', 'SOLARIS_SYSTEM', 'SOLARWINDS_KSS', 'SOLARWINDS_SERV_U', 'SONARQUBE', 'SONIC_FIREWALL', 'SOPHOS_AV', 'SOPHOS_CAPSULE8', 'SOPHOS_CENTRAL', 'SOPHOS_DHCP', 'SOPHOS_EDR', 'SOPHOS_FIREWALL', 'SOPHOS_UTM', 'SOTI_MOBICONTROL', 'SOURCEFIRE_IDS', 'SPLUNK', 'SPLUNK_DNS', 'SPLUNK_PHANTOM', 'SPYCLOUD', 'SQUID_WEBPROXY', 'STAIRWELL_INCEPTION', 'STEALTHBITS_AUDIT', 'STEALTHBITS_DEFEND', 'STEELHEAD', 'STORMSHIELD_FIREWALL', 'STREAMALERT', 'STRONGSWAN_VPN', 'SUPERNA_EYEGLASS', 'SUREVIEW_SYSTEMS', 'SURICATA_EVE', 'SURICATA_IDS', 'SWIFT_AMH', 'SWIMLANE', 'SYMANTEC_CASB', 'SYMANTEC_DLP', 'SYMANTEC_EDR', 'SYMANTEC_EVENT_EXPORT', 'SYMANTEC_MAIL', 'SYMANTEC_VIP', 'SYMANTEC_WEB_ISOLATION', 'SYMANTEC_WSS', 'SYSDIG', 'TANIUM_ASSET', 'TANIUM_AUDIT', 'TANIUM_COMPLY', 'TANIUM_DEPLOY', 'TANIUM_DISCOVER', 'TANIUM_INSIGHT', 'TANIUM_INTEGRITY_MONITOR', 'TANIUM_PATCH', 'TANIUM_QUESTION', 'TANIUM_REVEAL', 'TANIUM_TH', 'TANIUM_THREAT_RESPONSE', 'TCPWAVE_DDI', 'TEAMVIEWER', 'TELEPORT_ACCESS_PLANE', 'TENABLE_IO', 'TENABLE_SC', 'TERRAFORM_ENTERPRISE', 'TESSIAN_PLATFORM', 'TGDETECT', 'THALES_DIS', 'THALES_LUNA_HSM', 'THALES_MFA', 'THINKST_CANARY', 'THREATCONNECT_IOC', 'THYCOTIC', 'THYCOTIC_DEVOPS_SECRETVAULT', 'TIPPING_POINT', 'TOMCAT', 'TRACEABLE_PLATFORM', 'TRENDMICRO_AV', 'TRENDMICRO_CLOUDAPPSECURITY', 'TRENDMICRO_DEEP_SECURITY', 'TRENDMICRO_EDR', 'TRENDMICRO_WEBPROXY', 'TRIPWIRE_FIM', 'TWILIO_AUDIT', 'UBIQUITI_SWITCH', 'UDM', 'ULTRADNS', 'ULTRA_CYBERFENCE', 'UMBRELLA_DNS', 'UMBRELLA_FIREWALL', 'UMBRELLA_IP', 'UMBRELLA_WEBPROXY', 'UNBOUND_DNS', 'UNIFI_AP', 'UNIFI_SWITCH', 'UNIT21', 'UPTYCS_EDR', 'VANDYKE_SFTP', 'VARONIS', 'VECTRA_DETECT', 'VECTRA_PROTECT', 'VECTRA_STREAM', 'VEEAM', 'VENAFI', 'VERIZON_NDR', 'VERSA_FIREWALL', 'VIRUSTOTAL_THREAT_HUNTER', 'VITALQIP', 'VMRAY_FLOG_XML', 'VMWARE_AVINETWORKS_IWAF', 'VMWARE_AVI_VANTAGE', 'VMWARE_ESX', 'VMWARE_HCX', 'VMWARE_HORIZON', 'VMWARE_NSX', 'VMWARE_SDWN_EVENTS', 'VMWARE_TANZU', 'VMWARE_VCENTER', 'VMWARE_VREALIZE', 'VMWARE_VSHIELD', 'VMWARE_WORKSPACE_ONE', 'VOLTAGE', 'VORMETRIC', 'VSFTPD_AUDIT', 'VYOS', 'WATCHGUARD', 'WAZUH', 'WHITECLOUD_EDR', 'WINDOWS_AD', 'WINDOWS_APPLOCKER', 'WINDOWS_DEFENDER_ATP', 'WINDOWS_DEFENDER_AV', 'WINDOWS_DHCP', 'WINDOWS_DNS', 'WINDOWS_FIREWALL', 'WINDOWS_NET_POLICY_SERVER', 'WINDOWS_SYSMON', 'WINEVTLOG', 'WINEVTLOG_XML', 'WIZ_IO', 'WORDPRESS_CMS', 'WORKDAY', 'WORKDAY_AUDIT', 'WORKSPACE_ACTIVITY', 'WORKSPACE_ALERTS', 'WORKSPACE_CHROMEOS', 'WORKSPACE_GROUPS', 'WORKSPACE_MOBILE', 'WORKSPACE_PRIVILEGES', 'WORKSPACE_USERS', 'WORKSPOT_CONTROL', 'WP_ENGINE', 'WTI_CONSOLE_SERVERS', 'YUBICO_OTP', 'ZENDESK_CRM', 'ZEROFOX_PLATFORM', 'ZIMPERIUM', 'ZIX_EMAIL_ENCRYPTION', 'ZOOM_OPERATION_LOGS', 'ZSCALER_CASB', 'ZSCALER_DECEPTION', 'ZSCALER_DIGITAL_EXPERIENCE', 'ZSCALER_DLP', 'ZSCALER_DNS', 'ZSCALER_FIREWALL', 'ZSCALER_VPN', 'ZSCALER_WEBPROXY'] {allow-input: true}

#@markdown Time and date are in UTC
start_date = "2023-07-01" #@param {type : "date"}
end_date = "2023-07-13" #@param {type : "date"}

#@markdown Time should be in the format HH:MM:SS including leading zeros
startTime = "00:00:00" #@param {type: "string"}
endTime = "23:59:59" #@param {type: "string"}

startTime = f"{start_date}T{startTime}Z"
endTime = f"{end_date}T{endTime}Z"

decode_base64 = True #@param {type: "boolean"}

#@markdown Check the below checkbox to download the result to your computer, otherwise the results will only be saved to the colab instnace.
download_file = True #@param {type: "boolean"}
# Construct the URL
uri = f"https://{region_prefix}backstory.googleapis.com/v1/tools/cbnParsers:listCbnParserErrors?log_type={logtype}&start_time={startTime}&end_time={endTime}"
# send the request and process the response
# timeout set to ten minutes because some complaints of timeouts for long calls were received.
resp = session.get(uri, timeout=600)
json_resp = resp.json()
if resp.status_code > 299:
  pprint(json_resp.get('error').get('message'))
else:
  if f"{logtype}-{startTime}-{endTime}.json" in os.listdir():
    os.remove(f"{logtype}-{startTime}-{endTime}.json")

  if decode_base64:
    for error_index in range(len(json_resp['errors'])):
      for log_index in range(len(json_resp['errors'][error_index]["logs"])):
        json_resp['errors'][error_index]["logs"][log_index] = base64.b64decode(json_resp['errors'][error_index]["logs"][log_index]).decode()
  f = open( f"{logtype}-{startTime}-{endTime}.json", "w")
  f.write(json.dumps(json_resp))
  f.close()
  print(f"Errors written to filename: {logtype}-{startTime}-{endTime}.json ")
  if download_file:
    files.download(f"{logtype}-{startTime}-{endTime}.json")

In [ ]:
#@title Get all CBN parsers
#@markdown This will take many  minutes to run. Please prepare to twiddle your thumbs, or better yet just use the above cell to get the specific parser(s) you need one at a time.

# config ID list taken from ingest API get support log types endpoint
parser_list = ["A10_LOAD_BALANCER","ABNORMAL_SECURITY","ABSOLUTE","ACALVIO","ACCELLION","ADAUDIT_PLUS","ADFS","ADVA_FSP","AI_HUNTER","AIRLOCK_DIGITAL","AIRWATCH","AIX_SYSTEM","AKAMAI_CLOUD_MONITOR","AKAMAI_DNS","AKAMAI_EAA","AKAMAI_SIEM_CONNECTOR","AKAMAI_WAF","AKEYLESS_VAULT","ALCATEL_SWITCH","ALGOSEC","AMD_DSS_FIREWALL","ANOMALI_IOC","ANSIBLE_AWX","APACHE","APPOMNI","AQUA_SECURITY","ARBOR_EDGE_DEFENSE","ARBOR_SIGHTLINE","ARCHER_IRM","ARCSIGHT_CEF","AREA1","ARISTA_SWITCH","ARMIS_ACTIVITIES","ARMIS_ALERTS","ARMIS_DEVICES","ARMIS_VULNERABILITIES","ARRAYNETWORKS_VPN","ARUBA_AIRWAVE","ARUBA_EDGECONNECT_SDWAN","ARUBA_IPS","ARUBA_SWITCH","ARUBA_WIRELESS","ASOC_ALERT","ASSET_STATIC_IP","ATLASSIAN_BITBUCKET","ATLASSIAN_CONFLUENCE","ATLASSIAN_JIRA","ATTIVO","AUDITD","AUTH_ZERO","AUTOMATION_ANYWHERE","AVANAN_EMAIL","AVATIER","AVAYA_AURA","AWAKE_NDR","AWS_API_GATEWAY","AWS_AURORA","AWS_CLOUDFRONT","AWS_CLOUDTRAIL","AWS_CLOUDWATCH","AWS_CONFIG","AWS_CONTROL_TOWER","AWS_EC2_HOSTS","AWS_EC2_INSTANCES","AWS_EC2_VPCS","AWS_ELB","AWS_EMR","AWS_IAM","AWS_KMS","AWS_MACIE","AWS_NETWORK_FIREWALL","AWS_RDS","AWS_ROUTE_53","AWS_S3_SERVER_ACCESS","AWS_SECURITY_HUB","AWS_SESSION_MANAGER","AWS_VPC_FLOW","AWS_VPN","AWS_WAF","AZION","AZURE_ACTIVITY","AZURE_AD","AZURE_AD_AUDIT","AZURE_AD_CONTEXT","AZURE_AD_SIGNIN","AZURE_APP_SERVICE","AZURE_COSMOS_DB","AZURE_DEVOPS","AZURE_FIREWALL","AZURE_GATEWAY","AZURE_KEYVAULT_AUDIT","AZURE_MDM_INTUNE","AZURE_NSG_FLOW","AZURE_RESOURCE_LOGS","AZURE_SQL","AZURE_STORAGE_AUDIT","AZURE_VPN","AZURE_WAF","BARRACUDA_EMAIL","BARRACUDA_FIREWALL","BARRACUDA_WAF","BARRACUDA_WEBFILTER","BEYONDTRUST_ENDPOINT","BEYONDTRUST_PI","BEYONDTRUST_REMOTE_ACCESS","BIGSWITCH_BCF","BIND_DNS","BITDEFENDER","BITWARDEN_EVENTS","BLOXONE","BLUECAT_DDI","BLUECAT_EDGE","BLUECOAT_WEBPROXY","BMC_AMI_DEFENDER","BMC_HELIX_DISCOVERY","BOMGAR","BOX","BRO_JSON","BRO_TSV","BROADCOM_SSL_VA","BROCADE_SERVERIRON","BROCADE_SWITCH","CA_ACCESS_CONTROL","CA_ACF2","CA_LDAP","CA_SSO_WEB","CAMBIUM_NETWORKS","CASSANDRA","CATO_NETWORKS","CB_APP_CONTROL","CB_EDR","CENSYS","CENTRIFY_SSO","CENTRIPETAL_IOC","CEQUENCE_BOT_DEFENSE","CHECKPOINT_AUDIT","CHECKPOINT_EDR","CHECKPOINT_FIREWALL","CHECKPOINT_HARMONY","CHECKPOINT_SMARTDEFENSE","CHRONICLE_SOAR_AUDIT","CIMCOR","SYSLOG","CIRCLECI","CIS_ALBERT_ALERT","CISCO_ACE","CISCO_ACI","CISCO_ACS","CISCO_AMP","CISCO_ASA_FIREWALL","CISCO_CLOUDLOCK_CASB","CISCO_CTS","CISCO_DHCP","CISCO_DNAC","CISCO_EMAIL_SECURITY","CISCO_ESTREAMER","CISCO_FIREPOWER_FIREWALL","CISCO_FIRESIGHT","CISCO_FWSM","CISCO_IOS","CISCO_IRONPORT","CISCO_ISE","CISCO_MERAKI","CISCO_NX_OS","CISCO_PIX_FIREWALL","CISCO_PRIME","CISCO_ROUTER","CISCO_SECURE_WORKLOAD","CISCO_STADIUMVISION","CISCO_STEALTHWATCH","CISCO_SWITCH","CISCO_TACACS","CISCO_UCM","CISCO_UCS","CISCO_UMBRELLA_AUDIT","CISCO_VCS","CISCO_VPN","CISCO_WIPS","CISCO_WIRELESS","CISCO_WSA","CISCO_WSM","CITRIX_ANALYTICS","CITRIX_MONITOR","CITRIX_NETSCALER","CITRIX_STOREFRONT","CLAM_AV","CLAROTY_EMC","CLEARPASS","CLEARSWIFT","CLOUD_IDENTITY_CONTEXT","CLOUD_PASSAGE","CLOUDFLARE","CLOUDFLARE_AUDIT","CLOUDFLARE_WAF","CLOUDGENIX_SDWAN","CLOUDIAN_HYPERSTORE","CLOUDM","COFENSE_TRIAGE","COHESITY","COMFORTE_SECURDPS","COMMVAULT","COMMVAULT_COMMCELL","COMODO_AV","CORELIGHT","CORTEX_XDR","COVID_CTC_IOC","CRIBL_STREAM","CROWDSTRIKE_IOC","CS_DETECTS","CS_EDR","CS_STREAM","CSV_CUSTOM_IOC","CUSTOM_APPLICATION_ACCESS","CUSTOM_DNS","CUSTOM_SECURITY_DATA_ANALYTICS","CYBER_2_IDS","CYBERARK","CYBERARK_EPM","CYBERARK_PAM","CYBERARK_PRIVILEGE_CLOUD","CYBERARK_PTA","CYBEREASON_EDR","CYBERGATEKEEPER_NAC","CYBERX","CYLANCE_PROTECT","CYNET_360_AUTOXDR","D3_BANKING","DARKTRACE","DATADOG","DATAMINR_ALERT","DATTO_FILE_PROTECTION","DB2_DB","DEEP_INSTINCT_EDR","DELINEA_PAM","DELL_ECS","DELL_EMC_DATA_DOMAIN","DELL_EMC_NAS","DELL_OPENMANAGE","DELL_SWITCH","DESYNOVA_CONTIDO","DHS_IOC","DIGI_MODEMS","DIGITAL_SHADOWS_IOC","DIGITAL_SHADOWS_SEARCHLIGHT","DIGITALARTS_IFILTER","DIGITALGUARDIAN_DLP","DIGITALGUARDIAN_EDR","DMP_ENTRE","DNSFILTER","DOMAINTOOLS_THREATINTEL","DOPE_SWG","DUO_ADMIN","DUO_AUTH","DUO_CONTEXT","DUO_TELEPHONY","DUO_USER_CONTEXT","EFFICIENTIP_DDI","ELASTIC_AUDITBEAT","ELASTIC_PACKETBEATS","ELASTIC_SEARCH","ELASTIC_WINLOGBEAT","ENDPOINT_PROTECTOR_DLP","EPIC","ERGON_INFORMATIK_AIRLOCK_IAM","ESET_AV","ESET_EDR","ESET_IOC","ET_PRO_IOC","EVISION_FIRCOSOFT","EXCHANGE_MAIL","EXTRAHOP","EXTRAHOP_DNS","EXTREME_SWITCH","EXTREME_WIRELESS","F5_AFM","F5_ASM","F5_BIGIP_APM","F5_BIGIP_LTM","F5_DNS","F5_SHAPE","F5_SILVERLINE","F5_VPN","FALCO_IDS","FASTLY_WAF","FIDELIS_NETWORK","FILE_SCANNING_FRAMEWORK","FILEZILLA_FTP","FIREEYE_ALERT","FIREEYE_ETP","FIREEYE_HX","FIREEYE_HX_AUDIT","FIREEYE_NX","FIREEYE_NX_AUDIT","FIREEYE_PX","FIVETRAN","FLUENTD","FORCEPOINT_CASB","FORCEPOINT_DLP","FORCEPOINT_EMAILSECURITY","FORCEPOINT_FIREWALL","FORCEPOINT_MAIL_RELAY","FORCEPOINT_WEBPROXY","FORESCOUT_NAC","FORGEROCK_IDENTITY_CLOUD","FORGEROCK_OPENIDM","FORSETI","FORTINET_DHCP","FORTINET_FIREWALL","FORTINET_FORTIANALYZER","FORTINET_FORTIAUTHENTICATOR","FORTINET_FORTICLIENT","FORTINET_FORTIEDR","FORTINET_FORTIMAIL","FORTINET_FORTIMANAGER","FORTINET_FORTINAC","FORTINET_FORTIWEB","FORTRA_POWERTECH_SIEM_AGENT","GCP_APIGEE","GCP_APIGEE_X","GCP_APP_ENGINE","GCP_CLOUD_FUNCTIONS_CONTEXT","GCP_CLOUDIDENTITY_DEVICES","GCP_CLOUDIDENTITY_DEVICEUSERS","GCP_CLOUDIOT","GCP_CLOUDSQL","GCP_COMPUTE","GCP_IDS","GCP_KUBERNETES_CONTEXT","GCP_LOADBALANCING","GCP_NETWORK_CONNECTIVITY_CONTEXT","GCP_NGFW_ENTERPRISE","GCP_RECAPTCHA_ENTERPRISE","GCP_RESOURCE_MANAGER_CONTEXT","GCP_RUN","GCP_SECURITYCENTER_POSTURE_VIOLATION","GCP_SECURITYCENTER_TOXIC_COMBINATION","GCP_SQL_CONTEXT","GCP_SWP","GCP_VPC_FLOW","GITHUB","GITLAB","GMAIL_LOGS","GMV_CHECKER","GUARDDUTY","GUARDICORE_CENTRA","GUARDIUM","HADOOP","HAPROXY","HASHICORP","HCL_BIGFIX","HCNET_ACCOUNT_ADAPTER","HID_DIGITALPERSONA","HITACHI_CLOUD_PLATFORM","HONEYD","HP_PROCURVE","HPE_BLADESYSTEM_C7000","HPE_ILO","HYPR_MFA","IBM_AS400","IBM_CICS","IBM_DATAPOWER","IBM_DS8000","IBM_I","IBM_LTO","IBM_MAINFRAME_STORAGE","IBM_OPENPAGES","IBM_QRADAR","IBM_SAFENET","IBM_SAM","IBM_SECURITY_VERIFY","IBM_SECURITY_VERIFY_SAAS","IBM_SIM","IBM_TIVOLI","IBM_WEBSEAL","IBM_WEBSPHERE_APP_SERVER","IBM_ZOS","IBOSS_WEBPROXY","IIS","ILLUMIO_CORE","IMPERVA_ABP","IMPERVA_AUDIT_TRAIL","IMPERVA_CEF","IMPERVA_DB","IMPERVA_FLEXPROTECT","IMPERVA_SECURESPHERE","IMPERVA_WAF","INFOBLOX","INFOBLOX_DHCP","INFOBLOX_DNS","INFOBLOX_RPZ","INFORMIX","INGRIAN_NETWORKS_DATASECURE_APPLIANCE","INTERSYSTEMS_CACHE","ION_SPECTRUM","IONIX","IPSWITCH_MOVEIT_TRANSFER","IPSWITCH_SFTP","IRONSTREAM_ZOS","ISC_DHCP","ISLAND_BROWSER","JAMF","JAMF_PRO","JAMF_PROTECT","JAMF_TELEMETRY","JENKINS","JFROG_ARTIFACTORY","JUMPCLOUD_DIRECTORY_INSIGHTS","JUNIPER_FIREWALL","JUNIPER_IPS","JUNIPER_JUNOS","JUNIPER_MIST","JUNIPER_MX","JUNIPER_SDWAN","KASPERSKY_AV","KEA_DHCP","KEEPER","KEMP_LOADBALANCER","KERIOCONTROL","KEYCLOAK","KISI","KITEWORKS","KOLIDE","KONG_GATEWAY","KUBERNETES_AUDIT","KUBERNETES_AUDIT_AZURE","KUBERNETES_AUTH_PROXY","KUBERNETES_NODE","KYRIBA","LACEWORK","LASTPASS","LENEL_ONGUARD","LIMACHARLIE_EDR","LINUX_DHCP","LINUX_SYSMON","LOGONBOX","LOOKOUT_MOBILE_ENDPOINT_SECURITY","LUCID","MACOS","MACOS_ENDPOINT_SECURITY","MAILMARSHAL","MALWAREBYTES_EDR","MANAGE_ENGINE_AD360","MANAGE_ENGINE_REPORTER_PLUS","MANDIANT_CUSTOM_IOC","MARIA_DB","MATTERMOST","MCAFEE_DLP","MCAFEE_EPO","MCAFEE_ESM","MCAFEE_IPS","MCAFEE_MVISION_CASB","MCAFEE_SKYHIGH_CASB","MCAFEE_UCE","MCAFEE_WEB_PROTECTION","MCAFEE_WEBPROXY","MEDIGATE_IOT","MENANDMICE_DNS","MENLO_SECURITY","MICROFOCUS_IMANAGER","MICROSOFT_ATA","MICROSOFT_CASB","MICROSOFT_DEFENDER_CLOUD_ALERTS","MICROSOFT_DEFENDER_ENDPOINT","MICROSOFT_DEFENDER_IDENTITY","MICROSOFT_DEFENDER_MAIL","MICROSOFT_GRAPH_ACTIVITY_LOGS","MICROSOFT_GRAPH_ALERT","MICROSOFT_IAS","MICROSOFT_LAPS","MICROSOFT_NPS","MICROSOFT_SCEP","MICROSOFT_SENTINEL","MICROSOFT_SQL","MIKROTIK_ROUTER","MIMECAST_MAIL","MIMECAST_URL_LOGS","MISP_IOC","MOBILEIRON","MONGO_DB","MYSQL","N","N","N","N","N","N","N","N","N","N","N","N","NAGIOS","NASUNI_FILE_SERVICES","NEO4J","NEOSEC","NET_SUITE","NETAPP_ONTAP","NETAPP_SAN","NETAPP_STORAGEGRID","NETDOCUMENTS","NETFILTER_IPTABLES","NETIQ_EDIRECTORY","NETSCOUT_OCI","NETSKOPE_ALERT","NETSKOPE_CASB","NETSKOPE_WEBPROXY","NETWRIX","NGINX","NIMBLE_OS","NIX_SYSTEM","NOKIA_ROUTER","NONAME_API_SECURITY","NTOPNG","NUCLEUS_ASSET","NUCLEUS_VULNERABILITY","NUTANIX_PRISM","NXLOG_MANAGER","NYANSA_EVENTS","OBSERVEIT","OCI_AUDIT","OCI_FLOW","OCSF","OFFICE_365","OFFICE_365_MESSAGETRACE","OKERA_DAP","OKTA","OKTA_ACCESS_GATEWAY","OKTA_USER_CONTEXT","ONAPSIS","ONELOGIN_SSO","ONEPASSWORD","ONFIDO","OPEN_VPN","OPENAM","OPENCANARY","OPENDJ","OPENGEAR","OPENLDAP","OPENPATH","OPENSSH","OPNSENSE","ORACLE_CLOUD_AUDIT","ORACLE_DB","ORACLE_OUD","ORDR_IOT","OSQUERY_EDR","OSSEC","PAN_CASB","PAN_CORTEX_XDR_EVENTS","PAN_EDR","PAN_FIREWALL","PAN_IOC","PAN_PANORAMA","PAN_PRISMA_CA","PAN_PRISMA_CLOUD","PASSIVE_DNS","PASSWORDSTATE","PEPLINK_FW","PERIMETERX_BOT_PROTECTION","PFSENSE","PHISHLABS","PING","PING_FEDERATE","PIVOTAL","PORTNOX_CEF","POSTFIX_MAIL","POSTGRESQL","POWERSHELL","PREEMPT","PREEMPT_AUTH","PROOFPOINT_MAIL","PROOFPOINT_MAIL_FILTER","PROOFPOINT_ON_DEMAND","PROOFPOINT_SENDMAIL_SENTRION","PROOFPOINT_SER","PROOFPOINT_TRAP","PROOFPOINT_WEB_BROWSER_ISOLATION","PULSE_SECURE_VPN","PULSE_SECURE_VTM","QUALYS_ASSET_CONTEXT","QUALYS_CONTINUOUS_MONITORING","QUALYS_SCAN","QUALYS_VIRTUAL_SCANNER","QUALYS_VM","QUEST_AD","QUEST_CHANGE_AUDITOR_EMC","QUEST_FILE_AUDIT","QUMULO_FS","RADWARE_ALTEON","RADWARE_FIREWALL","RAPID7_INSIGHT","RAPID7_NEXPOSE","RECORDED_FUTURE_IOC","RECORDIA","REDCANARY_EDR","REDHAT_DIRECTORY_SERVER","REDHAT_OPENSHIFT","REMEDIANT_SECUREONE","RH_ISAC_IOC","RIBBON_ANALYTICS_PLATFORM","RIPPLING_ACTIVITYLOGS","RSA_AUTH_MANAGER","RSA_NETWITNESS","RSA_SECURID","RUBRIK","RUBRIK_POLARIS","RUCKUS_WIRELESS","SAILPOINT_IAM","SAIWALL_VPN","SALESFORCE","SANGFOR_NGAF","SAP_BTP","SAP_NETWEAVER","SAP_SAST","SAP_SM20","SAP_SUCCESSFACTORS","SAP_WEBDISP","SAVIYNT_EIP","SECUREAUTH_SSO","SECURELINK","SEMPERIS_DSP","SENDMAIL","SENTINEL_DV","SENTINEL_EDR","SENTINELONE_ALERT","SENTINELONE_CF","SEP","SEPPMAIL","SEQRITE_ENDPOINT","SERVICENOW_CMDB","SERVICENOW_SECURITY","SHIBBOLETH_IDP","SHRUBBERY_TACACS","SIERRA_WIRELESS","SIGNAL_SCIENCES_WAF","SILVERFORT","SITEMINDER_SSO","SKYBOX_FIREWALL_ASSURANCE","SLACK_AUDIT","SMBD","SNARE_SOLUTIONS","SNOOPY_LOGGER","SNORT_IDS","SNOWFLAKE","SNYK_SDLC","SOLARIS_SYSTEM","SOLARWINDS_KSS","SONIC_FIREWALL","SONICWALL_SMA","SONRAI","SOPHOS_AV","SOPHOS_CAPSULE8","SOPHOS_CENTRAL","SOPHOS_DHCP","SOPHOS_EDR","SOPHOS_FIREWALL","SOPHOS_UTM","SOTI_MOBICONTROL","SOURCEFIRE_IDS","SPLUNK","SPLUNK_ATTACK_ANALYZER","SPUR_FEEDS","SPYCLOUD","SQUID_WEBPROXY","STEALTHBITS_AUDIT","STEALTHBITS_DEFEND","STEALTHBITS_PAM","STEELHEAD","STIX","STORMSHIELD_FIREWALL","STRONGSWAN_VPN","SURICATA_EVE","SURICATA_IDS","SWIFT_AMH","SYMANTEC_CASB","SYMANTEC_DLP","SYMANTEC_EDR","SYMANTEC_EVENT_EXPORT","SYMANTEC_VIP","SYMANTEC_VIP_AUTHHUB","SYMANTEC_WEB_ISOLATION","SYMANTEC_WSS","SYNOLOGY","SYSDIG","TABLEAU","TALON","TANIUM_ASSET","TANIUM_AUDIT","TANIUM_COMPLY","TANIUM_DISCOVER","TANIUM_INSIGHT","TANIUM_INTEGRITY_MONITOR","TANIUM_PATCH","TANIUM_REVEAL","TANIUM_TH","TANIUM_THREAT_RESPONSE","TCPWAVE_DDI","TEAMVIEWER","TELEPORT_ACCESS_PLANE","TENABLE_ADS","TENABLE_AUDIT","TENABLE_IO","TENABLE_OT","TENABLE_SC","TERRAFORM_ENTERPRISE","TETRAGON_EBPF_AUDIT_LOGS","THALES_DIS","THALES_LUNA_HSM","THALES_MFA","THINKST_CANARY","THREATCONNECT_IOC","THREATLOCKER","THYCOTIC","TIPPING_POINT","TOMCAT","TRELLIX_HX_ES","TRENDMICRO_APEX_CENTRAL","TRENDMICRO_APEX_ONE","TRENDMICRO_AV","TRENDMICRO_CLOUDONE","TRENDMICRO_DEEP_SECURITY","TRENDMICRO_VISION_ONE","TRENDMICRO_WEBPROXY","TRIPWIRE_FIM","TWINGATE","UBIKA_WAAP","UBIKA_WAF","UBIQUITI_SWITCH","UMBRELLA_DNS","UMBRELLA_FIREWALL","UMBRELLA_IP","UMBRELLA_WEBPROXY","UNBOUND_DNS","UNIFI_AP","UPTYCS_EDR","UPX_ANTIDDOS","VANDYKE_SFTP","VARONIS","VECTRA_DETECT","VECTRA_STREAM","VEEAM","VELO_FIREWALL","VERBA_REC","VERIDIUM_ID","VERITAS_NETBACKUP","VERSA_FIREWALL","VIRTRU_EMAIL_ENCRYPTION","VITALQIP","VMWARE_ESX","VMWARE_HORIZON","VMWARE_NSX","VMWARE_TANZU","VMWARE_VCENTER","VMWARE_VREALIZE","VMWARE_WORKSPACE_ONE","VOLTAGE","VORMETRIC","VSFTPD","VYOS","WALLIX_BASTION","WATCHGUARD","WATCHGUARD_EDR","WAZUH","WEBMARSHAL","WINDCHILL","WINDOWS_AD","WINDOWS_APPLOCKER","WINDOWS_DEFENDER_ATP","WINDOWS_DEFENDER_AV","WINDOWS_DHCP","WINDOWS_DNS","WINDOWS_FIREWALL","WINDOWS_HYPERV","WINDOWS_NET_POLICY_SERVER","WINDOWS_SYSMON","WINEVTLOG","WINEVTLOG_XML","WINSCP","WIZ_IO","WORDPRESS_CMS","WORKDAY","WORKDAY_AUDIT","WORKSPACE_ACTIVITY","WORKSPACE_ALERTS","WORKSPACE_CHROMEOS","WORKSPACE_GROUPS","WORKSPACE_MOBILE","WORKSPACE_PRIVILEGES","WORKSPACE_USERS","XITING_XAMS","YAMAHA_ROUTER","YUBICO_OTP","ZIMPERIUM","ZIX_EMAIL_ENCRYPTION","ZOOM_OPERATION_LOGS","ZSCALER_CASB","ZSCALER_DECEPTION","ZSCALER_DLP","ZSCALER_DNS","ZSCALER_FIREWALL","ZSCALER_INTERNET_ACCESS","ZSCALER_TUNNEL","ZSCALER_VPN","ZSCALER_WEBPROXY","ZSCALER_ZPA","ZSCALER_ZPA_AUDIT"]

download_file = True #@param {type:"boolean"}

os.mkdir("cbn_files")

for logtype in parser_list:
  uri  = 'https://{}backstory.googleapis.com/v1/tools/cbnParsers/{}'.format(region_prefix, logtype)
  resp = http_client.request(uri, "GET")
  if resp[0].status == http.HTTPStatus.OK:
    result = json.loads(resp[1])
    cbn = base64.b64decode(result['config'])

    f = open("cbn_files/" + logtype + ".conf", "wb")
    f.write(cbn)
    f.close()
    #print(type(cbn))

    print(logtype + ': cbn exported')
    sleep(1)
  else:
    print('{}: no cbn found - '.format(logtype) + str(resp[0].status) )
    sleep(1)

!zip -r cbn_files.zip cbn_files


if download_file:
  files.download("cbn_files.zip")

In [ ]:
#@title Get all custom parsers
#@markdown This *should* pull all the current custom parsrers in the instnace a key has been provided for.

#@markdown Once complete a zip file containing the conf files will be downloaded by your browser.

#@markdown If you'd like to browse the CBNs in the colab UI switch to the files view and look at the `custom_parsers` directory.

download_file = True #@param {type: "boolean"}
from pprint import pprint

CHRONICLE_API_V1_URL = 'https://{}backstory.googleapis.com/v1'.format(region_prefix)
GET_CBN_PARSERS = '{}/tools/cbnParsers'.format(CHRONICLE_API_V1_URL)

print("Fetching parsers for Chronicle instance...")
# Send the request, parse the response, save logs to a file
response = http_client.request(GET_CBN_PARSERS , 'GET')

current_log_types = []
os.mkdir("custom_parsers")

if response[0].status == 200:
  live_parsers = json.loads(response[1])
  # pprint(live_parsers)

  index = 0
  print(f"Found {len(live_parsers['cbnParsers'])} parsers.")
  if "cbnParsers" in live_parsers:
    for parser in live_parsers['cbnParsers']:
      # print("Type: {type}\nAuthor:{author}\nState:{state}\nIndex: {index}\n".format(author= parser['author'], state= parser["state"], type= parser["logType"], index=index))
      current_log_types.append(parser["logType"])
      index += 1
    for log_type in current_log_types:
      print(f"Pausing for one second for API rate limit reasons, then fetching the parser for logtype: {log_type}")
      sleep(1)
      uri = f'https://{region_prefix}backstory.googleapis.com/v1/tools/cbnParsers:listCbnParserHistory?log_type={log_type}'
      # send the request and process the response
      resp = http_client.request(uri, "GET")
      json_resp = json.loads(resp[1])
      for item in json_resp['cbnParsers']:
        if item['state'] == "LIVE":
          with open(f"custom_parsers/{item['logType']}.conf", 'wb') as conf_file:
            conf_file.write(base64.b64decode(item['config']))
    print("Finished fetching parers.")
    if download_file:
      !zip -r custom_parsers.zip custom_parsers
      files.download("custom_parsers.zip")
  else:
    print("There don't appear to be any live parsers at the moment.")
else:
  # something went wrong
  err = response[1]
  print(err)


#<font color=red>Work in progess</font> Parser API V2 (BYOP)
This functionality requires two things

1. A Chronicle instance with BYOP setup  
1. A service account key from the GCP project tied to the Chronicle instance with the appropriate IAM roles granted
  1. Of the default roles `Chronicle API Admin` is the one to choose.

[Jump to top](#scrollTo=v0jKbGj8ZZRB)

In [ ]:
#@title Initialize BYOP info
#@markdown The below information is required to work with this API. Subsequent commands will not work without this.
PROJECT_ID = "chronicle-byop" #@param {type:"string"}
CUSTOMER_ID = "34871035-2752-4075-aed6-2e7f261be67d" #@param {type:"string"}
#@markdown <font color="red">Don't forget to click the play button on this cell after filling out the required fields!</font>
BASE_URI = f"https://{cli_region.lower()}-chronicle.googleapis.com/v1alpha/projects/{PROJECT_ID}/locations/{cli_region.lower()}/instances/{CUSTOMER_ID}/"

In [ ]:
#@title List Parsers
#@markdown Optional, leave as `-` to list all logtypes
logtype = "-" #@param ["-"] {allow-input: true}
decode_base64 = True #@param {type: "boolean"}
custom_only = False #@param {type: "boolean"}
#@markdown Filename required for all active parsers (not custom only). Too much data to display
filename = "parsers.json" #@param {type: "string"}
download_file = True #@param {type: "boolean"}
URL = f"https://{cli_region.lower()}-chronicle.googleapis.com/v1alpha/projects/{PROJECT_ID}/locations/{cli_region.lower()}/instances/{CUSTOMER_ID}/logTypes/{logtype.upper()}/parsers"
if custom_only:
  URL += "?filter=TYPE+%3D+CUSTOM"
resp = session.get(URL)
if resp.status_code > 299:
  print(resp.text)
  sys.exit()

json_resp = resp.json()
if decode_base64 and "parsers" in json_resp.keys():
  for parser_index in range(len(json_resp["parsers"])):
    json_resp["parsers"][parser_index]["cbn"] = base64.b64decode(json_resp["parsers"][parser_index]["cbn"]).decode()

if (not custom_only and filename) or len(resp.text) > 100000:
  if not filename:
    print(f"Please specify a filename. The response is too large to print on screen.")
    sys.exit(1)
  with open(filename, "w") as file_to_write:
    json.dump(json_resp, file_to_write, indent=4)
  if download_file:
    files.download(filename)
  print(f"Wrote data to {filename}")
elif not custom_only and len(resp.text) > 100000:
  print(f"Filename must be specified for listing all parsers. Data is too large to display on screen. Response body is {len(resp.raw)} bytes")
else:
  print(json.dumps(json_resp,indent=2))

In [ ]:
#@title List Extensions
#@markdown Optional, leave as `-` to list all logtypes
logtype = "-" #@param ["-"] {allow-input: true}
decode_base64 = True #@param {type: "boolean"}
#@markdown Filename required for all parsers (not custom only). Too much data to display
filename = "" #@param {type: "string"}
download_file = False #@param {type: "boolean"}
URL = f"{BASE_URI}logTypes/{logtype.upper()}/parserExtensions"
resp = session.get(URL)
if resp.status_code > 299:
  print(resp.text)
  sys.exit()

json_resp = resp.json()
if decode_base64:
  for parserExtension_index in range(len(json_resp["parserExtensions"])):
    if "log" in json_resp["parserExtensions"][parserExtension_index].keys():
      json_resp["parserExtensions"][parserExtension_index]["log"] = base64.b64decode(json_resp["parserExtensions"][parserExtension_index]["log"]).decode()
    if "fieldExtractors" in json_resp["parserExtensions"][parserExtension_index].keys() and \
    "transformedCbnSnippet" in json_resp["parserExtensions"][parserExtension_index]["fieldExtractors"].keys():
      json_resp["parserExtensions"][parserExtension_index]["fieldExtractors"]["transformedCbnSnippet"] = base64.b64decode(json_resp["parserExtensions"][parserExtension_index]["fieldExtractors"]["transformedCbnSnippet"]).decode()

if filename or len(resp.text) > 100000:
  if not filename:
    print(f"Please specify a filename. The response is too large to print on screen.")
    sys.exit(1)
  with open(filename, "w") as file_to_write:
    json.dump(json_resp, file_to_write, indent=4)
  if download_file:
    files.download(filename)
  print(f"Wrote data to {filename}")
else:
  print(json.dumps(json_resp,indent=2))

In [ ]:
#@title Run parser
#@markdown Enter the filename of the CBN to test
parser_conf_file = "VMWARE_TANZU.conf" #@param {type: "string"}
#@markdown Enter the filename of the testdata
test_data_file = "sample.log" #@param {type: "string"}

#@markdown Enter a valid logtype. This does not have the be the logtype tested, just any valid logtype.
LOG_TYPE = "OKTA" #@param {type: "string"}

#@markdown Choose whether to write the output data to a file and then whether to download that file.
write_to_file = True #@param {type: "boolean"}
download_file = False #@param {type: "boolean"}

file_name = "parser_output.blah" #@param {type: "string"}
also_write_to_screen = True #@param {type: "boolean"}

uri = f"{BASE_URI}logTypes/{logtype.upper()}:runParser"

conf_file = open(parser_conf_file, 'rb')
conf_data = conf_file.read()
conf_file.close()

log_file = open(test_data_file, 'rb')
log_data = log_file.readlines()
log_file.close()



data = {
  "parser": {
    "cbn": base64.urlsafe_b64encode(conf_data).decode()
  },
  "log": [base64.urlsafe_b64encode(x).decode() for x in log_data],
  "statedump_allowed": True
}

resp = session.post(uri,  json=data)

if resp.status_code > 299:
  print(resp.text)
else:
  if write_to_file:
    with open(file_name, "w") as output_file:
      output_file.write(resp.text)
      if also_write_to_screen:
        pprint(resp.json())
    if download_file:
      files.download(file_name)
  else:
    pprint(resp.json())

In [ ]:
#@title (Not ready) Submit a parser
#@markdown This endpoint enables you to submit a customer-specific parser configuration for a specific parser. The validation process requires at least 10,000 raw logs for each LogType and that event timestamps are no older than 30 days, otherwise it will fail. Make sure there are at least 10,000 raw logs for the LogType with event timestamps no older than 30 days from the time you submit the parser configuration.
log_type = 'PAN_EDR' #@param {type: "string"}
author_name = "Eugene Dimarsky" #@param {type: "string"}
#@markdown Check the below box to upload a parser or leave unchecked to specify a filename in the `parser_filename` variable below that is already in the colab instance.
upload_file = True #@param {type: "boolean"}
parser_filename = "ZSCALER_WEBPROXY.conf" #@param {type:"string"}
#@markdown Optional and experimental, add a changelog string to your parser
changelog_entry = "" #@param {type: "string"}
if upload_file:
  print("Please select the parser file to upload")
  uploaded = files.upload()

  parser_filename = list(uploaded.keys())[0]

with open(parser_filename, "rb") as parser_file:
  parser = parser_file.read()

skipValidationOnNoLogs = True #@param {type: "boolean"}

uri = f"{BASE_URI}logTypes/{logtype.upper()}/parsers"

# create the body with the parser configuration, logType identifier, and author
request_body = {
'cbn': base64.urlsafe_b64encode(parser).decode("ascii"),
'type' : "CUSTOM",
'changelogs' : {
    'entries' : [] if not changelog_entry else [changelog_entry]
},
'creator' : {
    'author' : author_name
},
'validatedOnEmptyLogs' : skipValidationOnNoLogs
}


# encode the body and set the Content-Type
body = urllib.parse.urlencode(request_body)
r_headers = {}
r_headers.update({'Content-type' : 'application/x-www-form-urlencoded'})

# send the request and process the response
resp = session.post(uri, data=body, headers=r_headers)
json_resp = resp.json()
if resp.status_code > 299:
  pprint(json_resp['error'])
else:
  pprint(json_resp)

#RBAC API
Documentation [link](https://cloud.google.com/chronicle/docs/reference/rbac-api)

[Jump to top](#scrollTo=v0jKbGj8ZZRB)

In [ ]:
#@title List Roles
#@markdown Retrieves all roles.
ENDPOINT_BASE = 'https://{}backstory.googleapis.com/v1'.format(region_prefix)
LIST_ROLES_URL = '{}/roles'.format(ENDPOINT_BASE)
# send the request and process the response
resp = session.get(LIST_ROLES_URL)
json_resp = resp.json()
if resp.status_code > 299:
  pprint(json_resp.get('error').get('message'))
else:
  pprint(json_resp)

In [ ]:
#@title Update Role
#@markdown Updates a role to specify whether it is the default role.

name = "Administrator" #@param {type: "string"}
isDefault = True #@param {type: "boolean"}

ENDPOINT_BASE = 'https://{}backstory.googleapis.com/v1'.format(region_prefix)
UPDATE_ROLE_URL = '{}/roles/{}'.format(ENDPOINT_BASE, name)

req_body = {
        "isDefault": isDefault
}

# send the request and process the response
resp = session.patch(UPDATE_ROLE_URL, json=req_body)
json_resp = resp.json()
if resp.status_code > 299:
  pprint(json_resp.get('error').get('message'))
else:
  pprint(json_resp)

In [ ]:
#@title Create Subject
#@markdown Creates a subject and assigns the given role.

#@markdown The name of the IdP group to grant the below role(s)
name = "i_like_cheese" #@param {type: "string"}
subject_type = "SUBJECT_TYPE_IDP_GROUP" #@param ["SUBJECT_TYPE_ANALYST", "SUBJECT_TYPE_IDP_GROUP"]
#@markdown Please enter a comma separated list of desired roles
roles = "ViewerWithNoDetectAccess, Viewer" #@param {type: "string"}

uri = f'https://{region_prefix}backstory.googleapis.com/v1/subjects'

req_body = {
  "name": name,
  "type": subject_type,
  "roles": [{"name": role.strip()} for role in roles.split(",")]
}

# send the request and process the response
resp = http_client.request(uri, "POST", body=json.dumps(req_body))
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  pprint(json_resp)

In [ ]:
#@title Get Subject
#@markdown Retrieves a subject.

subject_id = "i_like_cheese" #@param {type: "string"}

uri = f"https://{region_prefix}backstory.googleapis.com/v1/subjects/{subject_id}"
# send the request and process the response
resp = http_client.request(uri, "GET")
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  pprint(json_resp)

In [ ]:
#@title List Subjects
#@markdown Retrieves all subjects.

#@markdown Specify the maximum number of subjects to return (range is 1 through 1,000). The default is 100.
page_size = 101 #@param {type:"slider", min:1, max:1000, step:1}
#@markdown Page token received from a previous call. Use to retrieve the next page.
page_token = "" #@param {type: "string"}
url_params = {'page_size': page_size}
if page_token:
  url_params["page_token"] = page_token
encoded_url_params = urlencode(url_params)

ENDPOINT_BASE = f'https://{region_prefix}backstory.googleapis.com/v1'
LIST_SUBJECTS_URL = f'{ENDPOINT_BASE}/subjects?{encoded_url_params}'
# send the request and process the response
resp = http_client.request(LIST_SUBJECTS_URL, "GET")
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  pprint(json_resp)

In [ ]:
#@title Update Subject
#@markdown Updates the role of the specified subject.

subject_id = "i_like_cheese" #@param {type: "string"}
name = "" #@param {type: "string"}
subject_type = "SUBJECT_TYPE_IDP_GROUP" #@param ["SUBJECT_TYPE_ANALYST", "SUBJECT_TYPE_IDP_GROUP"]
#@markdown Please enter a comma separated list of desired roles
roles = "Administrator" #@param {type: "string"}

req_payload = {
  "subject": {
    "name": name,
    "type": subject_type,
    "roles": []
  }
}

roles = roles.split(",")
for role in roles:
  req_payload['subject']['roles'].append({"name": role.strip()})

uri = f"https://{region_prefix}backstory.googleapis.com/v1/subjects/{subject_id}"
# send the request and process the response
resp = http_client.request(uri, "PATCH", body=json.dumps(req_body))
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  pprint(json_resp)

In [ ]:
#@title Delete Subject
#@markdown Deletes a subject.

subject_id = "i_like_cheese" #@param {type: "string"}

uri = f"https://{region_prefix}backstory.googleapis.com/v1/subjects/{subject_id}"
# send the request and process the response
resp = http_client.request(uri, "DELETE")
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  pprint(json_resp)

#Ingestion API
Documentation [link](https://cloud.google.com/chronicle/docs/reference/ingestion-api)

[Jump to top](#scrollTo=v0jKbGj8ZZRB)

In [ ]:
#@title udmevents
#@markdown Use this method to forward UDM events to Chronicle in batches.

customer_id = "bd54dc26-3d54-44fb-88dc-8fd323df46f0" #@param {type: "string"}
#@markdown Choose whether to upload a file of UDM data or to specify a file already in the colab filesystem.
upload_log_file = True #@param {type: "boolean"}
log_file = "udm.json" #@param {type: "string"}

#@markdown **The logfile should be a JSON style array of UDM events.**

if upload_log_file:
  print("Please select the file with UDM data (in JSON format)")
  uploaded = files.upload()
  log_file = list(uploaded.keys())[0]
with open(log_file, "r") as log_data:
  log_entries = json.loads(log_data.read())

# delete logfile after upload to prevent naming issues as covered in the notes at the bottom
if upload_log_file:
  os.system(f"rm {log_file}")


payload = {
    "customer_id" : customer_id,
    "events" : log_entries
}

uri = f"https://{region_prefix}malachiteingestion-pa.googleapis.com/v2/udmevents:batchCreate"

resp = http_client.request(uri, "POST", body=json.dumps(payload))
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  print(f"Logs uploaded succesfully. Returned body is:\n{pprint(json_resp)}")


In [ ]:
#@title Unstructured Log Entries
#@markdown Use this method to forward unstructured log entries to Chronicle one batch at a time.

#@markdown Each batch of log data has a maximum size limit of 1 MB. Do not send multiple types of timestamps (use either ts_epoch_microseconds or ts_rfc3339 but not both). If you do not provide a separate timestamp, be sure to include one within the log_text field.

#@markdown  **Note:** The 1 MB maximum size limit refers to an uncompressed payload. In case you send a compressed payload, the batch size should be limited to 1 MB after decompression.

#@markdown <font color=red>**NO BATCHING IS DONE FOR THIS EXAMPLE CELL**</font>

#@markdown Please only specify a file that will keep the batch under 1MB. One log entry per line is assumed.

from time import sleep
customer_id = "d6cfdfe2-4a3c-4f73-8e92-d43c6ce1e554" #@param {type: "string"}
log_type = "CROWDSTRIKE_IOC" #@param {type: "string"}
namespace = "" #@param {type: "string"}
upload_log_file = True #@param {type: "boolean"}
log_file = "data.log" #@param {type: "string"}
#@markdown Optional:
add_current_time_as_log_timestamp = False #@param {type:"boolean"}
#@markdown Labels (also optional). Please enter as a comma seperated list of key=value pairs

#@markdown Example: **key1=value1, key2=value2, key3=value3, etc...**
labels = "" #@param {type: "string"}

if upload_log_file:
  print("Please select the file with UDM data (in JSON format)")
  uploaded = files.upload()
  log_file = list(uploaded.keys())[0]
with open(log_file, "r") as log_data:
  log_entries = []
  # lines_read = 0
  # log_data.seek(offset)
  # print("Position: {}\nOffset: {}\n{}".format(positon,offset, log_data.readline()))
  log_entries = [x.strip() for x in log_data.readlines()]

# delete logfile after upload to prevent naming issues as covered in the notes at the bottom
if upload_log_file:
  os.system(f"rm {log_file}")

payload = {
    "customer_id" : customer_id,
    "log_type" : log_type,
    "entries" : [],
    # "labels" : [
    #     {
    #         "key": "ingestion_test_label",
    #         "value": "successfully_applied"
    #     },
    #     {
    #         "key": "ingestion_test_label",
    #         "value": "also_applied"
    #     }
    # ]
}

if labels:
  #create a labels key
  if "labels" not in payload.keys():
    payload["labels"] = []

  labels = labels.split(",")
  for label in labels:
    key, value = label.split("=")
    payload["labels"].append({"key" : key.strip(), "value" : value.strip()})

if namespace:
  payload['namespace'] = namespace

for log_entry in log_entries:
  proto_log = {"log_text" : log_entry}
  if add_current_time_as_log_timestamp:
    now = datetime.now(timezone.utc)
    proto_log["ts_rfc3339"] = now.isoformat()
  payload["entries"].append(proto_log)

payload = json.dumps(payload)

ENDPOINT_BASE = 'https://{}malachiteingestion-pa.googleapis.com/v2'.format(region_prefix)
LIST_CBN_PARSER_STATUS_URL = '{}/unstructuredlogentries:batchCreate'.format(ENDPOINT_BASE)
# send the request and process the response
resp = http_client.request(LIST_CBN_PARSER_STATUS_URL, "POST", body=payload)
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  print("Logs uploaded successfully. Total logs sent: {}".format(len(log_entries)))
  # pprint(json_resp)

Please select the file with UDM data (in JSON format)


Saving FinalCSIOC2.json to FinalCSIOC2.json
Logs uploaded succesfully. Total logs sent: 1


In [ ]:
#@title createentities
#@markdown Creates entities. You are limited to 1MB of data per request.

customer_id = "9cd07b80-f66e-42c3-acd2-0ac803b63337" #@param {type: "string"}
#@markdown Choose whether to upload a file of UDM data or to specify a file already in the colab filesystem.
upload_log_file = False #@param {type: "boolean"}
log_file = "udm.json" #@param {type: "string"}
log_type = "QUALYS_ASSET_CONTEXT" #@param {type : "string"}

#@markdown **The logfile should be a JSON style array of UDM entities.**

if upload_log_file:
  print("Please select the file with UDM data (in JSON format)")
  uploaded = files.upload()
  log_file = list(uploaded.keys())[0]
with open(log_file, "r") as log_data:
  log_entries = json.loads(log_data.read())

# delete logfile after upload to prevent naming issues as covered in the notes at the bottom
if upload_log_file:
  os.system(f"rm {log_file}")


payload = {
    "customer_id" : customer_id,
    "log_type": log_type,
    "entities" : log_entries
}

uri = f"https://{region_prefix}malachiteingestion-pa.googleapis.com/v2/entities:batchCreate"

resp = session.post(uri, json=payload)
if resp.status_code > 299:
  pprint(resp.json().get('error').get('message'))
else:
  print(f"Logs uploaded succesfully. Returned body is:\n{pprint(resp.json())}")


In [ ]:
#@title Get Supported log types
#@markdown Use this method to retrieve a list of supported log types.
download_output = False #@param {"type": "boolean"}
file_name = "supported_log_types.json" #@param {"type":"string"}
ENDPOINT_BASE = 'https://{}malachiteingestion-pa.googleapis.com/v2'.format(region_prefix)
LIST_CBN_PARSER_STATUS_URL = '{}/logtypes'.format(ENDPOINT_BASE)
# send the request and process the response
resp = http_client.request(LIST_CBN_PARSER_STATUS_URL, "GET")
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  if download_output:
    with open(file_name, "w") as output_file:
      json.dump(json_resp, output_file, indent=2)
    files.download(file_name)
    file_size = os.path.getsize(file_name)
    print(f"Output written to {file_name}. File is {file_size}B")
  else:
    pprint(json_resp)

# valid_log_types = []
# for log_type in json_resp['logTypes']:
#   valid_log_types.append(log_type["logType"])
# with open("valid_logs.txt", "w") as output_file:
#   output_file.write(str(valid_log_types))

#Feed Schema API
Documentation [link](https://cloud.google.com/chronicle/docs/reference/feed-management-api#feed_schema_api_reference)

[Jump to top](#scrollTo=v0jKbGj8ZZRB)

In [ ]:
#@title GetFeedSchema
#@markdown This method returns a structure representing the entire feed schema.

download_output = True #@param {"type": "boolean"}
file_name = "feedSchema.json" #@param {"type":"string"}

URL = 'https://{}backstory.googleapis.com/v1/feedSchema'.format(region_prefix)
# send the request and process the response
resp = session.get(URL)
json_resp = resp.json()
# show the parser history
if resp.status_code > 299:
  pprint(json_resp['error'])
else:
  if download_output:
    with open(file_name, "w") as output_file:
      json.dump(json_resp, output_file, indent=4)
    files.download(file_name)
    file_size = os.path.getsize(file_name)
    print(f"Output written to {file_name}. File is {file_size}B")
  else:
    pprint(json_resp)

In [ ]:
#@title ListFeedSourceTypeSchemas
#@markdown This method returns information about all feed source types.

download_output = False #@param {"type": "boolean"}
file_name = "feedSourceTypes.json" #@param {"type":"string"}

URL = 'https://{}backstory.googleapis.com/v1/feedSourceTypeSchemas'.format(region_prefix)
# send the request and process the response
resp = session.get(URL)
json_resp = resp.json()
# show the parser history
if resp.status_code > 299:
  pprint(json_resp['error'])
else:
  if download_output:
    with open(file_name, "w") as output_file:
      json.dump(json_resp, output_file, indent=4)
    files.download(file_name)
    file_size = os.path.getsize(file_name)
    print(f"Output written to {file_name}. File is {file_size}B")
  else:
    pprint(json_resp)

In [ ]:
#@title ListLogTypeSchemas
#@markdown This method returns information about all log types compatible with a particular feed source type.

feed_source_type = "API" #@param {type:"string"}
download_output = True #@param {"type": "boolean"}
file_name = "logTypeSchemasHTTP.json" #@param {"type":"string"}

URL = f'https://{region_prefix}backstory.googleapis.com/v1/feedSourceTypeSchemas/{feed_source_type}/logTypeSchemas'
# send the request and process the response
resp = session.get(URL)
json_resp = resp.json()
# show the parser history
if resp.status_code > 299:
  pprint(json_resp['error'])
else:
  if download_output:
    with open(file_name, "w") as output_file:
      json.dump(json_resp, output_file, indent=4)
    files.download(file_name)
    file_size = os.path.getsize(file_name)
    print(f"Output written to {file_name}. File is {file_size}B")
  else:
    pprint(json_resp)

In [ ]:
#@title GetLogTypeSchema
#@markdown This method returns detailed information about all the fields necessary to configure a feed for a particular source type and log type.

feed_source_type = "SFTP" #@param {type:"string"}
log_type = "ONEPASSWORD" #@param {type: "string"}
download_output = False #@param {"type": "boolean"}
file_name = "logTypeSchema.json" #@param {"type":"string"}

URL = f'https://{region_prefix}backstory.googleapis.com/v1/feedSourceTypeSchemas/{feed_source_type}/logTypeSchemas/{log_type}'
# send the request and process the response
resp = session.get(URL)
json_resp = resp.json()
# show the parser history
if resp.status_code > 299:
  pprint(json_resp['error'])
else:
  if download_output:
    with open(file_name, "w") as output_file:
      json.dump(json_resp, output_file, indent=4)
    files.download(file_name)
    file_size = os.path.getsize(file_name)
    print(f"Output written to {file_name}. File is {file_size}B")
  else:
    pprint(json_resp)

#Feed Management API
Documentation [link](https://cloud.google.com/chronicle/docs/reference/feed-management-api#feed_management_api_reference)

[Jump to top](#scrollTo=v0jKbGj8ZZRB)

In [ ]:
#@title Enable Feed
#@markdown Enables an `INACTIVE` feed, which allows it to be executed.
feed_id = "ab26c31e-b1dd-42b7-a40c-a244b6eccaa6" #@param {type: "string"}

download_output = False #@param {type: "boolean"}
file_name = "configured_feeds.json" #@param {type: "string"}

URL = f'https://{region_prefix}backstory.googleapis.com/v1/feeds/{feed_id}:enable'
# send the request and process the response
resp = session.post(URL)
json_resp = resp.json()
# show the parser history
if resp.status_code > 299:
  pprint(json_resp['error'])
else:
  if download_output:
    with open(file_name, "w") as output_file:
      json.dump(json_resp, output_file, indent=4)
    files.download(file_name)
    file_size = os.path.getsize(file_name)
    print(f"Output written to {file_name}. File is {file_size}B")
  else:
    pprint(json_resp)

In [ ]:
#@title Disable Feed
#@markdown Disables a feed. A disabled feed has a status of `INACTIVE`. Disabled feeds will no longer fetch data.
feed_id = "ab26c31e-b1dd-42b7-a40c-a244b6eccaa6" #@param {type: "string"}

download_output = False #@param {type: "boolean"}
file_name = "configured_feeds.json" #@param {type: "string"}

URL = f'https://{region_prefix}backstory.googleapis.com/v1/feeds/{feed_id}:disable'
# send the request and process the response
resp = session.post(URL)
json_resp = resp.json()
# show the parser history
if resp.status_code > 299:
  pprint(json_resp['error'])
else:
  if download_output:
    with open(file_name, "w") as output_file:
      json.dump(json_resp, output_file, indent=4)
    files.download(file_name)
    file_size = os.path.getsize(file_name)
    print(f"Output written to {file_name}. File is {file_size}B")
  else:
    pprint(json_resp)

In [ ]:
#@title GetFeed
#@markdown Gets the details of the feed that was configured.
feed_id = "730245ae-c114-4753-9989-34ba9788a60f" #@param {type: "string"}

download_output = False #@param {type: "boolean"}
file_name = "configured_feeds.json" #@param {type: "string"}

URL = f'https://{region_prefix}backstory.googleapis.com/v1/feeds/{feed_id}'
# send the request and process the response
resp = session.get(URL)
json_resp = resp.json()
# show the parser history
if resp.status_code > 299:
  pprint(json_resp['error'])
else:
  if download_output:
    with open(file_name, "w") as output_file:
      json.dump(json_resp, output_file, indent=4)
    files.download(file_name)
    file_size = os.path.getsize(file_name)
    print(f"Output written to {file_name}. File is {file_size}B")
  else:
    pprint(json_resp)

{'details': {'feedSourceType': 'API',
             'logType': 'OKTA',
             'oktaSettings': {'hostname': 'dev-32032166.okta.com'}},
 'feedState': 'COMPLETED',
 'lastFeedInitiationTime': '2025-04-24T15:17:40.114894002Z',
 'name': 'feeds/730245ae-c114-4753-9989-34ba9788a60f'}


In [ ]:
#@title ListFeeds
#@markdown Retrieves all the feeds configured for a given Chronicle instance.

download_output = False #@param {type: "boolean"}
file_name = "configured_feeds.json" #@param {type: "string"}
URL = f'https://{region_prefix}backstory.googleapis.com/v1/feeds'
# send the request and process the response
resp = session.get(URL)
json_resp = resp.json()
# show the parser history
if resp.status_code > 299:
  pprint(json_resp['error'])
else:
  if download_output:
    with open(file_name, "w") as output_file:
      json.dump(json_resp, output_file, indent=4)
    files.download(file_name)
    file_size = os.path.getsize(file_name)
    print(f"Output written to {file_name}. File is {file_size}B")
  else:
    pprint(json_resp)

In [ ]:
#@title Delete a specific feed
#@markdown Deletes a feed that was configured using the Chronicle Feed Management API.
feed_id = "f625fd1e-ff0f-4065-9f1e-7bb70f8e5e60" #@param {type: "string"}
ENDPOINT_BASE = 'https://{}backstory.googleapis.com/v1'.format(region_prefix)
LIST_FEEDS_URL = '{}/feeds/{}'.format(ENDPOINT_BASE,feed_id)
# send the request and process the response
resp = http_client.request(LIST_FEEDS_URL, "DELETE")
json_resp = json.loads(resp[1])
# show the parser history
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp['error'])
else:
  print("Feed id {} has been succesfully deleted.".format(feed_id))

In [ ]:
#@title Fetch service account
#@markdown Gets a unique service account that Chronicle uses to ingest data. Use this method only if you're setting up a [Cloud Storage feed](https://cloud.google.com/chronicle/docs/reference/feed-management-api#gc-storage).

URL = f'https://{region_prefix}backstory.googleapis.com/v1/fetchFeedServiceAccount'
# send the request and process the response
resp = session.get(URL)
json_resp = resp.json()
# show the parser history
if resp.status_code > 299:
  pprint(json_resp['error'])
else:
  pprint(json_resp)

###Create and Update feed endpoints
These are not currently being added because the payload body varies for each type of feed type. Putting all of them in a notebook is unrealistic and probably unnecessary.

If you would like to try this functionality please use the [CLI tool](#scrollTo=Wx1pfTxOjv57). It will walk you through feed creation step by step.

All details can be found at the official [Feed Management API docs](https://cloud.google.com/chronicle/docs/preview/feed-management-api/feed-management-api)

In [ ]:
#@title CreateFeed
#@markdown Gets the details of the feed that was configured.
body = "{   \"display_name\": \"OCSF Webhook\",   \"details\": {     \"feedSourceType\": \"HTTPS_PUSH_WEBHOOK\",     \"logType\": \"OCSF\"   } }" #@param {type: "string"}
body = json.loads(body)
uri = f'https://{region_prefix}backstory.googleapis.com/v1/feeds'
# send the request and process the response
resp = session.post(uri, json = body)
json_resp = resp.json()
# show the parser history
if resp.status_code > 299:
  pprint(json_resp['error'])
else:
  pprint(json_resp)

#Google Cloud Threat Intelligence API

[Jump to top](#scrollTo=v0jKbGj8ZZRB)

In [ ]:
#@title List Alerts

page_token = "" #@param {type: "string"}
page_size =   1#@param {type: "integer"}

ENDPOINT_BASE = 'https://{}backstory.googleapis.com/v1'.format(region_prefix)
LIST_URL = '{}/uppercaseAlerts?page_size={}&page_token={}'.format(ENDPOINT_BASE, page_size, page_token)
# send the request and process the response
resp = http_client.request(LIST_URL, "GET" )
json_resp = json.loads(resp[1])
# show the parser history
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp['error'])
else:
  pprint(json_resp)

In [ ]:
#@title Get Alert

alert_id = "" #@param {type: "string"}

ENDPOINT_BASE = 'https://{}backstory.googleapis.com/v1'.format(region_prefix)
LIST_URL = '{}/uppercaseAlerts/{}'.format(ENDPOINT_BASE, alert_id)
# send the request and process the response
resp = http_client.request(LIST_URL, "GET" )
json_resp = json.loads(resp[1])
# show the parser history
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp['error'])
else:
  pprint(json_resp)

#CBN Tool
<font color='red' size=+4>Please Note, DEPRECATED</font>:  
 The cbn-tool CLI was deprecated on 29 December, 2023. Developers are requested to use the new [Chronicle CLI](https://github.com/chronicle/cli).

[Jump to top](#scrollTo=v0jKbGj8ZZRB)

In [ ]:
#@title List all Parsers
command_to_run = f"python /content/cbn-tool/cbn_cli.py --region={cbn_region} --credentials_file=/content/{key_filename} list"

result = os.popen(command_to_run)
result = result.read()
print(result)

In [ ]:
#@title Validate a CBN
conf_file = "ede98ebd-2422-4194-acdd-6fac803a7695.conf" #@param {type: "string"}
log_file = "fidelis.log" #@param {type: "string"}
output_file = "validation.txt"  #@param {type: "string"}
save_output = True #@param {type: "boolean"}
print_output = False #@param {type: "boolean"}
command_to_run = f"python /content/cbn-tool/cbn_cli.py --region={cbn_region} --credentials_file=/content/{key_filename} run --conf_file={conf_file} --log_file={log_file}"

result = os.popen(command_to_run)
result = result.read()

if print_output:
  print(result)
if save_output:
  if output_file in os.listdir():
    os.remove(output_file)
  with open(output_file, "w") as file_to_write:
    file_to_write.write(result)
print("Validation complete.")

In [ ]:
#@title Get Sample Logs
log_type = "DNS_SINKHOLE" #@param {type: "string"}
start_date = "2023-01-30" #@param {type: "date"}
end_date = "2023-02-01" #@param {type: "date"}
command_to_run = f"python /content/cbn-tool/cbn_cli.py --region={cbn_region} --credentials_file=/content/{key_filename} gen --log_type={log_type} --start_date={start_date}T00:00:00Z --end_date={end_date}T23:59:59Z"

result = os.popen(command_to_run)
result = result.read()
print(result)

In [ ]:
#@title List errors of a log type between specific days
start_date = "2022-05-23" #@param {type:"date"}
end_date = "2022-06-02" #@param {type:"date"}
log_type = "OKTA" #@param {type:"string"}
output_file = "something.txt" #@param {type:"string"}
write_to_file = False #@param {type: "boolean"}
print_output = True #@param {type: "boolean"}

command_to_run = f"python /content/cbn-tool/cbn_cli.py --region={cbn_region} --credentials_file=/content/{key_filename}  error --start_date={start_date}T00:00:00Z --end_date={end_date}T23:59:59Z --log_type={log_type}"

result = os.popen(command_to_run)
result = result.read()
if print_output:
  print("Result:")
  print(result)

if write_to_file:
  with open(output_file, "w") as file:
    file.write(result)



In [ ]:
#@title Upload Custom CBN

#@markdown Select whether to upload the CBN conf file or specify a local filename. If uploading the cbn_filename variable doesn't matter
upload_conf_file = True #@param {type:"boolean"}
cbn_filename = "blah.blah" #@param {type: "string"}
if upload_conf_file:
  uploaded = files.upload()
  cbn_filename = list(uploaded.keys())[0]
#@markdown Enter the author to be displayed
author = "google.com" #@param {type: "string"}
#@markdown Enter the logtype this CBN will parse
log_type = "ARCSIGHT_CEF" #@param {type: "string"}
status_check_interval = 15

command_to_run = f'python /content/cbn-tool/cbn_cli.py --region={cbn_region} --credentials_file=/content/{key_filename} submit --conf_file={cbn_filename} --log_type={log_type} --author={author}'
print("running command: {}".format(command_to_run))
result = os.popen(command_to_run)
result = result.read()
print("Result:")
print(result)

config_id_regex_result = re.search(r"status --config_id=([\-\w]+)", result)
if config_id_regex_result:
  config_id = config_id_regex_result.groups()[0]
  command_to_run = f"python /content/cbn-tool/cbn_cli.py --region={cbn_region} --credentials_file=/content/{key_filename} status --config_id={config_id}"
  status_check_result = os.popen(command_to_run)
  status_check_result = status_check_result.read()
  status_check_result_json_regex_capture = re.search(r"({.*})",status_check_result, re.DOTALL)
  if status_check_result_json_regex_capture:
    status_check_json = json.loads(status_check_result_json_regex_capture.groups()[0])
    while status_check_json["state"] == "NEW" or status_check_json["state"] == "VALIDATING":
      print("Parser validation in progress. Current state is: {}\nWaiting {} seconds and trying again".format(status_check_json["state"], status_check_interval))
      # pprint(status_check_json)
      sleep(status_check_interval)
      status_check_result = os.popen(command_to_run)
      status_check_result = status_check_result.read()
      status_check_result_json_regex_capture = re.search(r"({.*})",status_check_result, re.DOTALL)
      status_check_json = json.loads(status_check_result_json_regex_capture.groups()[0])
    pprint(status_check_json)




#Customer Management API
[Documentation](https://cloud.google.com/chronicle/docs/preview/customer-management-api/customer-management-api)

[Jump to top](#scrollTo=v0jKbGj8ZZRB)

In [ ]:
#@title Create Customer
#@markdown Creates a customer and associates this customer with the partner. This API fully provisions a customer in Chronicle. The response includes the customer ID.
customer_name = "nogcp" #@param {type: "string"}
customer_code =  "dmrsknogcp" #@param {type: "string"}
customer_subdomains = "dmrsknogcp"  #@param {type: "string"}
#@markdown If specifying the provider_id it should be in the format `locations/global/workforcePools/<pool name>/providers/<provider name>`
provider_id = ""  #@param {type: "string"}
#@markdown If specifying project it should be in the format `projects/<project number>` <br><font color=red>The text based project ID will NOT work. You must use the project number!
gcp_project = "" #@param {type: "string"}
retention_duration = "ONE_YEAR" #@param ["ONE_YEAR", "SIX_MONTHS"]
#@markdown Select whether to upload the SSO file or specify a local filename. If uploading the sso_config_file variable doesn't matter
upload_sso_config = False #@param {type: "boolean"}
# sso_config_filename = "sso_conf.txt"  #@param {type: "string"}
sso_config_filename = "" #@param {type: "string"}

req_body = {
    "customer_name" : customer_name,
    "customer_code" : customer_code,
    "customer_subdomains" : customer_subdomains,
    "retention_duration" : retention_duration
    }

if provider_id:
  req_body["provider_id"] = provider_id

if gcp_project:
  req_body["gcp_project"] = gcp_project

if upload_sso_config:
  print("Please select and upload your SSO config file.")
  uploaded = files.upload()

  sso_config_filename = list(uploaded.keys())[0]

if upload_sso_config and sso_config_filename:
  with open(sso_config_filename, "rb") as file_to_read:
    sso_config_file = base64.b64encode(file_to_read.read()).decode('ascii')

  req_body["sso_config"] = sso_config_file


uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v1/partner/customer/createcustomer"
# send the request and process the response
print("Attempting to create customer. This may take several minutes and there will be no output during this time.")
# extend http timeout to 20 minutes since this is a long call
http_client.timeout = 1200
# pprint(req_body)
resp = http_client.request(uri_to_post, "POST", body=json.dumps(req_body))
# return timeout back to default 1 minute
http_client.timeout = 60
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  pprint(json_resp)


Attempting to create customer. This may take several minutes and there will be no output during this time.
{'customer_code': 'dmrsknogcp',
 'customer_name': 'nogcp',
 'customer_subdomains': 'dmrsknogcp',
 'retention_duration': 'ONE_YEAR'}
'generic::unknown: customer creation failed: INVALID_ARGUMENT'


In [ ]:
#@title Generate IdP Metadata
#@markdown Generates new IdP metadata for a given subdomain. Customers can access Chronicle using <subdomain>.backstory.chronicle.security.
customer_subdomain = "dimarsky-old" #@param {type: "string"}
customer_subdomains = customer_subdomain.lower()
download_output = False #@param {type: "boolean"}
# decode_base64 = True #@param {type: "boolean"}

uri_to_get = "https://{}backstory.googleapis.com/v1/partner/customer/generateidpmetadata".format(region_prefix)

req_body = {
    "customer_subdomain" : customer_subdomain
    }



# send the request and process the response
resp = session.post(uri_to_get, json=req_body)
json_resp = resp.json()
if resp.status_code > 299:
  pprint(json_resp.get('error').get('message'))
else:
  metadata = json_resp['metadata']
  if download_output:
    with open("{}.xml".format(customer_subdomain), "w") as customer_file:
      customer_file.write(metadata)
      # pprint(customer_data, stream=customer_file)
    files.download("{}.xml".format(customer_subdomain))
    print("IdP Metadata has been downloaded to the local system.")
  else:
    print(metadata)


In [ ]:
#@title Get Customer
#@markdown Retrieves the customer details, including the customer name, forwarder IDs, stored credentials, and subdomains for a given customer being managed by the calling partner.
customer_code = "DMRSKnfr" #@param {type: "string"}
customer_code = customer_code.lower()
download_output = False #@param {type: "boolean"}
decode_base64 = True #@param {type: "boolean"}

uri_to_get = "https://{}backstory.googleapis.com/v1/partner/customer/getcustomer?customer_code={}".format(region_prefix ,customer_code)
# send the request and process the response
resp = http_client.request(uri_to_get, "GET")
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  customer_data = json_resp
  if decode_base64:
    for credential_index in range(len(customer_data['credentials'])):
      customer_data['credentials'][credential_index]['credential'] = base64.b64decode(customer_data['credentials'][credential_index]['credential']).decode()
    if "config" in customer_data["ssoConfig"]:
      customer_data["ssoConfig"]["config"] = base64.b64decode(customer_data["ssoConfig"]["config"]).decode()
      sso_config = customer_data['ssoConfig']
    customer_credentials = customer_data['credentials']
    printable_customer_data = customer_data.copy()
    if 'ssoConfig' in printable_customer_data:
      del printable_customer_data['ssoConfig']
    del printable_customer_data['credentials']
    pprint(printable_customer_data)
    if "config" in customer_data["ssoConfig"]:
      print("\n\nSSO Config:\n{}".format(sso_config['config']))
    for credential in customer_credentials:
      print("\n\n{} :\n{}".format(credential["credentialType"], credential["credential"]))

  else:
    pprint(customer_data)
  if download_output:
    with open("{}.json".format(customer_code), "w") as customer_file:
      customer_file.write(json.dumps(customer_data))
      # pprint(customer_data, stream=customer_file)
    files.download("{}.json".format(customer_code))


In [ ]:
#@title Get Customer Forwarder Configs
#@markdown Retrieves the customer's Linux or Windows Forwarder configuration. It first ensures that the partner identified by the service account has authorization to access the customer's forwarder. If there is more than one Chronicle Forwarder for a specific customer, the first active Forwarder is selected and its configuration is returned.
customer_code = "dmrsktst" #@param {type: "string"}
download_output = True #@param {type: "boolean"}
decode_base64 = True #@param {type: "boolean"}

uri_to_get = "https://{}backstory.googleapis.com/v1/partner/customer/forwarder/getconfigs?customer_code={}".format(region_prefix ,customer_code)
# send the request and process the response
resp = http_client.request(uri_to_get, "GET")
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  forwarder_data = json_resp
  if decode_base64:
    for forwarder_index in range(len(forwarder_data['forwarderConfigs'])):
      forwarder_data["forwarderConfigs"][forwarder_index]['config'] = base64.b64decode(forwarder_data["forwarderConfigs"][forwarder_index]['config']).decode()
      print("{}\n{}".format(forwarder_data["forwarderConfigs"][forwarder_index]['platform'], forwarder_data["forwarderConfigs"][forwarder_index]['config']))
  else:
    pprint(forwarder_data)
  if download_output:
    with open("{}_forwarders.json".format(customer_code), "w") as customer_file:
      customer_file.write(json.dumps(forwarder_data))
      # pprint(forwarder_data, stream=customer_file)
    files.download("{}_forwarders.json".format(customer_code))


In [ ]:
#@title List Customers
#@markdown Retrieves a list of the customers associated with the partner. The partner is identified when they authenticate using their service account credentials.
download_output = False #@param {type: "boolean"}
# decode_base64 = False #@param {type: "boolean"}

uri_to_get = f"https://{region_prefix}backstory.googleapis.com/v1/partner/customer/listcustomers"
# send the request and process the response
resp = session.get(uri_to_get)
# json_resp = json.loads(resp[1])
if resp.status_code > 299:
  pprint(resp.json().get('error').get('message'))
else:
  customers = resp.json()
  pprint(customers)
  # if decode_base64:
  #   for forwarder_index in range(len(forwarder_data['forwarderConfigs'])):
  #     forwarder_data["forwarderConfigs"][forwarder_index]['config'] = base64.b64decode(forwarder_data["forwarderConfigs"][forwarder_index]['config']).decode()
  # pprint(forwarder_data)
  if download_output:
    with open("customer_list.json", "w") as customer_file:
      pprint(customers, stream=customer_file)
    files.download("customer_list.json")


In [ ]:
#@title Set Auth Version
#@markdown Enables you to update the authentication version for the specified customer. Complete this step only after confirming that the end-to-end authentication for the tenant they are migrating is working correctly.

# Note: SetAuthVersion uses the POST method and auth_version can only be set to AUTH_VERSION_1 or AUTH_VERSION_2.
customer_code = "dmrsk" #@param {type: "string"}
auth_version = "AUTH_VERSION_3" #@param ["AUTH_VERSION_1","AUTH_VERSION_2", "AUTH_VERSION_3", "AUTH_VERSION_4"]

req_payload = {
    "customer_code" : customer_code,
    "auth_version" : auth_version
}

uri_to_post = "https://{}backstory.googleapis.com/v1/partner/customer/setauthversion".format(region_prefix)
# send the request and process the response
resp = http_client.request(uri_to_post, "POST", body=json.dumps(req_payload))
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  pprint(json_resp)


In [ ]:
#@title Set Compliance Requirements
#@markdown Given a customer code and a list of compliance certifications, SetComplianceRequirements sets compliance requirements for the customer.
customer_code = "dmrsknfr" #@param {type: "string"}
#@markdown Please enter a comma separated list of complaince certifications. Valid values as of 10/18/23 are:
##@markdown * COMPLIANCE_CERTIFICATION_UNSPECIFIED
#@markdown * COMPLIANCE_CERTIFICATION_FEDRAMP_MODERATE
#@markdown * COMPLIANCE_CERTIFICATION_HIPAA
#@markdown * COMPLIANCE_CERTIFICATION_PCI_DSS
#@markdown * COMPLIANCE_CERTIFICATION_FEDRAMP_HIGH
compliance_certifiaction = "" #@param {type : "string"}
compliance_certifiaction = [x.strip() for x in compliance_certifiaction.split(",")]

req_payload = {
    "customer_code" : customer_code,
    "compliance_certifications" : compliance_certifiaction
}

uri_to_post = "https://{}backstory.googleapis.com/v1/partner/customer/setcompliancerequirements".format(region_prefix)
# send the request and process the response
resp = http_client.request(uri_to_post, "POST", body=json.dumps(req_payload))
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  pprint(json_resp)


In [ ]:
#@title Set UI State
#@markdown Enables or disables the web application of a tenant managed by a partner by updating the customer's feature flag.


customer_code = "dmrskbyop" #@param {type: "string"}
UI_state = True #@param {type: "boolean"}

req_payload = {
    "customer_code" : customer_code,
    "state" : UI_state
}

uri_to_post = "https://{}backstory.googleapis.com/v1/partner/customer/setuistate:state".format(region_prefix)
# send the request and process the response
resp = http_client.request(uri_to_post, "POST", body=json.dumps(req_payload))
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  pprint(json_resp)


In [ ]:
#@title Set Workforce Pool Provider
#@markdown Given a customer code and its subdomain, SetWorkforcePoolProvider sets the workforce pool provider ID for the customer and its specified subdomain. Refer to the guide on how to create a workforce pool provider ID.


customer_code = "dmrsknfr" #@param {type: "string"}
customer_code = customer_code.lower()
customer_subdomain = "dimarskynet" #@param {type: "string"}
provider_id = "locations/global/workforcePools/workspace-pool/providers/personal-domain-workspace" #@param {type: "string"}

req_payload = {
    "customer_code" : customer_code,
    "customer_subdomain" : customer_subdomain,
    "provider_id" : provider_id
}

uri_to_post = "https://{}backstory.googleapis.com/v1/partner/customer/setworkforcepoolprovider".format(region_prefix)
# send the request and process the response
resp = http_client.request(uri_to_post, "POST", body=json.dumps(req_payload))
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  pprint(json_resp)


In [ ]:
#@title Update SSO Config
#@markdown Enables you to update the SSO Configuration for the specified customer. You can optionally add a subdomain if there are multiple subdomains for a given customer. If you call this API as part of the migration from AUTH_VERSION_1 to AUTH_VERSION_2, Google recommends setting the update_v2_only field to true.


customer_code = "epamdemo" #@param {type: "string"}
customer_code = customer_code.lower()
customer_subdomain = "epamdemo" #@param {type: "string"}
update_v2_only = True #@param {type: "boolean"}

#@markdown Select whether to upload the SSO file or specify a local filename. If uploading the sso_config_file variable doesn't matter
upload_sso_config = False #@param {type: "boolean"}
sso_config_filename = "/content/okta_metadata.xml"  #@param {type: "string"}

if upload_sso_config:
  print("Please select and upload your SSO config file.")
  uploaded = files.upload()

  sso_config_filename = list(uploaded.keys())[0]

with open(sso_config_filename, "rb") as file_to_read:
  sso_config_file = base64.b64encode(file_to_read.read()).decode('ascii')


req_payload = {
    "customer_code" : customer_code,
    "sso_config" : sso_config_file,
    "customer_subdomain" : customer_subdomain,
    "update_v2_only" : update_v2_only
}

uri_to_post = "https://{}backstory.googleapis.com/v1/partner/customer/updatessoconfig".format(region_prefix)
# send the request and process the response
http_client.timeout = 600
resp = http_client.request(uri_to_post, "PATCH", body=json.dumps(req_payload))
http_client.timeout = 60
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  pprint(json_resp)


In [ ]:
#@title Set GCP Project Link
#@markdown Sets the Google Cloud project for the Chronicle instance. You may provide either the project ID or the project number in the gcp_project field.


customer_code = "dmrsknfr" #@param {type: "string"}
#@markdown The GCP project can either be the Project Number or the Project ID
gcp_project = "chronicle-byop" #@param {type: "string"}

req_payload = {
    "customer_code" : customer_code,
    "gcp_project" : f"projects/{gcp_project}"
}

uri_to_post = "https://{}backstory.googleapis.com/v1/partner/customer/setgcpprojectlink".format(region_prefix)
# send the request and process the response
resp = http_client.request(uri_to_post, "POST", body=json.dumps(req_payload))
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  pprint(json_resp)


In [ ]:
#@title GenerateIamMigrationCommands
#@markdown Provides auto-generated commands that create new IAM policies equivalent to your existing Google SecOps RBAC access control, which is configured in Google SecOps, under the SIEM Settings > Users and Groups page.

customer_code = "dmrskbyop" #@param {type: "string"}

req_payload = {
    "customer_code" : customer_code
}

uri_to_post = "https://{}backstory.googleapis.com/v1/partner/customer/iammigration:generateIamMigrationCommands".format(region_prefix)
# send the request and process the response
resp = http_client.request(uri_to_post, "POST", body=json.dumps(req_payload))
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  pprint(json_resp)


In [ ]:
#@title EnableIAM
#@markdown Enables Feature RBAC controlled using IAM on the Google Security Operations instance of the customer organization specified in the request body.

customer_code = "dmrskbyop" #@param {type: "string"}

req_payload = {
    "customer_code" : customer_code
}

uri_to_post = "https://{}backstory.googleapis.com/v1/partner/customer/iammigration:enableIam".format(region_prefix)
# send the request and process the response
resp = http_client.request(uri_to_post, "POST", body=json.dumps(req_payload))
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  pprint(json_resp)


#Detection Engine
Documentation [link](https://cloud.google.com/chronicle/docs/reference/detection-engine-api#archiverule)

[Jump to top](#scrollTo=v0jKbGj8ZZRB)

In [ ]:
#@title ArchiveRule
#@markdown Archive the specified rule.

ruleId = "ru_4aa9de46-704c-4a96-8aa8-5ee1992302f7" #@param {type: "string"}

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/detect/rules/{ruleId}:archive"
resp = http_client.request(uri_to_post, "POST")
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  pprint(json_resp)


In [ ]:
#@title CancelRetrohunt
#@markdown Cancel a retrohunt for the specified rule.

#@markdown Please specify a rule ID to cancel a retrohunt for the latest version of a rule.

#@markdown **OR**

#@markdown  Please specify a version ID to cancel a retrohunt for a specific version of a rule.
cancel_latest_version = True #@param {type: "boolean"}
ruleId = "" #@param {type: "string"}
versionId = "" #@param {type: "string"}
retrohuntId = "" #@param {type: "string"}

rule_identifier = ruleId if cancel_latest_version else versionId

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/detect/rules/{rule_identifier}/retrohunts/{retrohuntId}:cancelRetrohunt"
resp = http_client.request(uri_to_post, "POST")
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  pprint(json_resp)



In [ ]:
#@title CreateRule
#@markdown Create a new rule without setting the rule to live.

#@markdown Choose whether to upload a text file with the rule or use one already in the filesystem of the colab instance.

upload_rule = False #@param {type: "boolean"}
rule_filename = "rule.txt" #@param {type: "string"}

#@markdown <font color="red">Optional</font>: [add labels for data export API](https://cloud.google.com/chronicle/docs/preview/datatap-config/use-rules-in-datatap)

#@markdown Please enter as a comma seperated list of label=state

#@markdown Example: **label1=ENABLED, label2=DISABLED, label3=ENABLED, etc...**

labels = "label1=ENABLED, label2=DISABLED, label3=ENABLED" #@param {type: "string"}

if upload_rule:
  print("Please select and upload your rule")
  uploaded = files.upload()

  rule_filename = list(uploaded.keys())[0]

with open(rule_filename, "r") as rule:
  rule_text = rule.read()

request_body = {
    "ruleText" : rule_text
}

if labels:
  #create a labels key
  if "labels" not in request_body.keys():
    request_body["labels"] = {"label":[]}

  labels = labels.split(",")
  for label in labels:
    label, state = label.split("=")
    if state.strip().upper() not in ["ENABLED", "DISABLED"]:
      print(f"State must be either \"ENABLED\" or \"DISABLED\". {state.trim().upper()} was found instead. Aborting this execution. Please correct this data.")
      sys.exit(f"Invalid value supplied for state: {state.trim().upper()}")
    request_body["labels"]["label"].append({
        "state": state.strip().upper(),
        "data_tap_label": {
          "sink_name": label.strip(),
        }
      })
# print(json.dumps(request_body, indent=2))
uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/detect/rules"
resp = http_client.request(uri_to_post, "POST", body=json.dumps(request_body))
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  pprint(json_resp)


In [ ]:
#@title CreateRuleVersion
#@markdown Creates a new version of an existing rule. The new version of the rule does not have to be based on the latest version.

ruleId = "ru_4aa9de46-704c-4a96-8aa8-5ee1992302f7" #@param {type: "string"}

#@markdown Choose whether to upload a text file with the rule or use one already in the filesystem of the colab instance.

upload_rule = False #@param {type: "boolean"}
rule_filename = "new_rule.yaral" #@param {type: "string"}

if upload_rule:
  print("Please select and upload your keyfile")
  uploaded = files.upload()

  rule_filename = list(uploaded.keys())[0]

with open(rule_filename, "r") as rule:
  rule_text = rule.read()

request_body = {
    "ruleText" : rule_text
}


uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/detect/rules/{ruleId}:createVersion"
resp = http_client.request(uri_to_post, "POST", body=json.dumps(request_body))
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  pprint(json_resp)


In [ ]:
#@title DeleteRule
#@markdown Delete the specified rule.

ruleId = "ru_f42f44db-4702-483e-9dcd-769023a35119" #@param {type: "string"}

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/detect/rules/{ruleId}"
resp = http_client.request(uri_to_post, "DELETE")
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  pprint(json_resp)


In [ ]:
#@title DisableAlerting
#@markdown Disables alerts for the specified rule.

ruleId = "" #@param {type: "string"}

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/detect/rules/{ruleId}:disableAlerting"
resp = http_client.request(uri_to_post, "POST")
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  pprint(json_resp)


In [ ]:
#@title DisableLiveRule
#@markdown Disable the latest version of the rule.

ruleId = "" #@param {type: "string"}

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/detect/rules/{ruleId}:disableLiveRule"
resp = http_client.request(uri_to_post, "POST")
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  pprint(json_resp)


In [ ]:
#@title EnableAlerting
#@markdown Enables alerts for the specified rule.

ruleId = "" #@param {type: "string"}

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/detect/rules/{ruleId}:enableAlerting"
resp = http_client.request(uri_to_post, "POST")
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  pprint(json_resp)


In [ ]:
#@title EnableLiveRule
#@markdown Set the latest version of a rule to live.

ruleId = "ru_e9b2c796-de5a-4261-8606-88f2c18f78b7" #@param {type: "string"}

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/detect/rules/{ruleId}:enableLiveRule"
resp = http_client.request(uri_to_post, "POST")
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  pprint(json_resp)


In [ ]:
#@title GetDetection
#@markdown Return the specified detection for the specified version of a rule.

#@markdown Please specify a rule ID to get the detection for the latest version of a rule.

#@markdown <font color="red">**OR**</font>

#@markdown  Please specify a version ID to get the detection for a specific version of a rule.
latest_version = True #@param {type: "boolean"}
ruleId = "ru_4160ef00-534f-4f35-880c-367dfc1e30b4" #@param {type: "string"}
versionId = "" #@param {type: "string"}
detectionId = "de_a2e3d7a0-8363-d69e-1079-bdc8a4c5155a" #@param {type: "string"}

rule_identifier = ruleId if latest_version else versionId

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/detect/rules/{rule_identifier}/detections/{detectionId}"
resp = http_client.request(uri_to_post, "GET")
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  pprint(json_resp)



In [ ]:
#@title GetError
#@markdown Return the specified error.

errorID = "" #@param {type: "string"}

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/detect/errors/{errorId}"
resp = http_client.request(uri_to_post, "GET")
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  pprint(json_resp)


In [ ]:
#@title GetRetrohunt
#@markdown Getting the retrohunt for the specified version of a rule.

#@markdown Please specify a rule ID to get the retrohunt for the latest version of a rule.

#@markdown **OR**

#@markdown  Please specify a version ID to get the retrohunt for a specific version of a rule.
latest_version = True #@param {type: "boolean"}
ruleId = "" #@param {type: "string"}
versionId = "" #@param {type: "string"}
retrohuntId = "" #@param {type: "string"}

rule_identifier = ruleId if latest_version else versionId

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/detect/rules/{rule_identifier}/retrohunts/{retrohuntId}"
resp = http_client.request(uri_to_post, "GET")
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  pprint(json_resp)



In [ ]:
#@title GetRule
#@markdown Return the specified rule.

#@markdown Please specify a rule ID to get the latest version of a rule.

#@markdown **OR**

#@markdown  Please specify a version ID to get a specific version of a rule.
latest_version = True #@param {type: "boolean"}
ruleId = "ru_cfbdd40f-1223-4812-8dd4-c6a0dc0e46df" #@param {type: "string"}
versionId = "" #@param {type: "string"}

rule_identifier = ruleId if latest_version else versionId

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/detect/rules/{rule_identifier}"
resp = http_client.request(uri_to_post, "GET")
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  pprint(json_resp)



In [ ]:
#@title ListCuratedRules
#@markdown List Chronicle rules with detections.

#@markdown Page size can range from 1 to 1,000
page_size = 100 #@param {type:"integer"}
page_token = "" #@param {type: "string"}

url_params = {'page_size': page_size}
if page_token:
  url_params["page_token"] = page_token

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/detect/curatedRules"
resp = session.get(uri_to_post, params=url_params)
json_resp = resp.json()
if resp.status_code > 299:
  pprint(json_resp.get('error').get('message'))
else:
  pprint(json_resp)


In [ ]:
#@title ListCuratedRuleDetections
#@markdown Return the detections for the specified Chronicle rule.

#@markdown REQUIRED:
ruleId = "-" #@param {type: "string"}

#@markdown OPTIONAL:
alert_state = "" #@param ["", "ALERTING", "NOT_ALERTING"]
#@markdown Start and End times are optional. If you check the box(es) below the corresponding field will be ignore (unbound start or end time).
no_starting_time = True #@param {type: "boolean"}
no_ending_time = True #@param {type: "boolean"}
startDate = "2025-01-04" #@param {type: "date"}
endDate = "2025-02-04" #@param {type: "date"}
#@markdown Time should be in the format HH:MM:SS including leading zeros
startTime = "12:12:00" #@param {type: "string"}
endTime = "12:12:00" #@param {type: "string"}

#@markdown Sort detections by **DETECTION_TIME** or by **CREATED_TIME**. Defaults to DETECTION_TIME. Detections are returned in descending order of the timestamp.
list_basis = "DETECTION_TIME" #@param ["DETECTION_TIME", "CREATED_TIME"]

startTime = f"{startDate}T{startTime}Z"
endTime = f"{endDate}T{endTime}Z"

#@markdown Page size can range from 1 to 1,000
page_size = 100 #@param {type:"integer"}
page_token = "" #@param {type: "string"}


url_params = {
    'page_size': page_size,
    "list_basis" : list_basis
    }
if page_token:
  url_params["page_token"] = page_token
if not no_starting_time:
  url_params["start_time"] = startTime
if not no_ending_time:
  url_params["end_time"] = endTime
if alert_state:
  url_params["alert_state"] = alert_state

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/detect/curatedRules/{ruleId}/detections"
resp = session.get(uri_to_post, params=url_params)
json_resp = resp.json()
if resp.status_code > 299:
  pprint(json_resp.get('error').get('message'))
else:
  pprint(json_resp)


In [ ]:
#@title ListDetections
#@markdown Return the detections for the specified version of a rule, the latest version of a rule, all versions of a rule, or all versions of all rules.

#@markdown Please specify a rule ID to get detections for the latest version of a rule.

#@markdown **OR**

#@markdown  Please specify a version ID to get detections for a specific version of a rule.

latest_version = False #@param {type: "boolean"}
ruleId = "-" #@param {type: "string"}
#@markdown To get detections for all versions of a specified rule check the below box and make sure the **latest_version** checkbox above is also checked..
all_versions = False #@param {type: "boolean"}
versionId = "" #@param {type: "string"}

#@markdown If you would like to ignore the rule ID and get detections for all versions of all rules then check the below box.
all_detections_for_all_version = True #@param {type: "boolean"}

page_size = 12 #@param {type:"slider", min:1, max:1000, step:1}
page_token = "" #@param {type: "string"}


startTime = "2023-09-01" #@param {type: "date"}
endTime = "2023-09-12" #@param {type: "date"}
startTime = f"{startTime}T00:00:00Z"
endTime = f"{endTime}T23:59:59Z"

write_to_file = True #@param {type: "boolean"}
#@markdown If writing to file should data also be displayed on screen?
also_output_to_screen = False #@param {type: "boolean"}
#@markdown If the write to file option is selected specify the filename to write data to.
filename = "detections.json" #@param {type: "string"}

url_params = {}
url_params["start_time"] = startTime
url_params["end_time"] = endTime

url_params['page_size'] =  page_size
if page_token:
  url_params["page_token"] = page_token
encoded_url_params = urlencode(url_params)

rule_identifier = ruleId if latest_version else versionId

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/detect/rules/{rule_identifier}/detections"
if all_versions:
  uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/detect/rules/{rule_identifier}@-/detections"
if all_detections_for_all_version:
  uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/detect/rules/-/detections"

uri_to_post += f"?{encoded_url_params}"

resp = http_client.request(uri_to_post, "GET")
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  if write_to_file:
    with open(filename, "w") as output_file:
      if "detections" in json_resp.keys():
        output_file.write(json.dumps(json_resp))
        print(f"Wrote {len(json_resp['detections'])} detection to {filename}")
        if also_output_to_screen:
          pprint(json_resp)
      else:
        print("No detections were returned for this query.")
  else:
    pprint(json_resp)


In [ ]:
#@title ListErrors
#@markdown List the latest rule errors.

#@markdown Page size can range from 1 to 1,000
page_size = 101 #@param {type:"integer"}
page_token = "" #@param {type: "string"}

url_params = {'page_size': page_size}
if page_token:
  url_params["page_token"] = page_token

startDate = "2022-11-01" #@param {type: "date"}
endDate = "2022-11-03" #@param {type: "date"}

#@markdown Time should be in the format HH:MM:SS including leading zeros
startTime = "00:00:01" #@param {type: "string"}
endTime = "23:59:59" #@param {type: "string"}

startTime = f"{startDate}T{startTime}Z"
endTime = f"{endDate}T{endTime}Z"

url_params["start_time"] = startTime
url_params["end_time"] = endTime

encoded_url_params = urlencode(url_params)


uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/health/errors?{encoded_url_params}"
resp = http_client.request(uri_to_post, "GET")
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  pprint(json_resp)


In [ ]:
#@title ListRetrohunts
#@markdown Listing the retrohunts for a rule.

#@markdown Please specify a rule ID to get retrohunts for the latest version of a rule.

#@markdown **OR**

#@markdown  Please specify a version ID to get retrohunts for a specific version of a rule.

latest_version = True #@param {type: "boolean"}
ruleId = "" #@param {type: "string"}
#@markdown To get retrohunts for all versions of a specified rule check the below box and make sure the **latest_version** checkbox above is also checked..
all_versions = True #@param {type: "boolean"}
versionId = "" #@param {type: "string"}

#@markdown If you would like to ignore the rule ID and get retrohunts for all versions of all rules then check the below box.
all_retrohunts_for_all_version = True #@param {type: "boolean"}

page_size = 4 #@param {type:"slider", min:1, max:1000, step:1}
page_token = "" #@param {type: "string"}

url_params = {}
url_params['page_size'] = page_size
if page_token:
  url_params["page_token"] = page_token
encoded_url_params = urlencode(url_params)


rule_identifier = ruleId if latest_version else versionId

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/detect/rules/{rule_identifier}/retrohunts"
if all_versions:
  uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/detect/rules/{rule_identifier}@-/retrohunts"
if all_retrohunts_for_all_version:
  uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/detect/rules/-/retrohunts"

uri_to_post += f"?{encoded_url_params}"

resp = http_client.request(uri_to_post, "GET")
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  pprint(json_resp)


In [ ]:
#@title ListRules
#@markdown List the latest versions of all rules.

#@markdown Max page size is 10,000
page_size = 2000 #@param {type:"integer"}
page_token = "" #@param {type: "string"}
download_output = False #@param {type: "boolean"}

url_params = {'page_size': page_size}
if page_token:
  url_params["page_token"] = page_token
encoded_url_params = urlencode(url_params)

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/detect/rules?{encoded_url_params}"
resp = http_client.request(uri_to_post, "GET")
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  if download_output:
    if "listRules.json" in os.listdir():
      os.remove("listRules.json")
    with open("listRules.json", "w") as output_file:
      json.dump(json_resp, output_file, indent=2)
    files.download("listRules.json")
    print("Complete and file downloaded")
  else:
    print(json_resp)
  # pprint(json_resp)


In [ ]:
#@title ListRuleVersions
#@markdown List all versions of a specific rule. The versions are listed in descending order by the rule version creation time.

ruleId = "ru_4aa9de46-704c-4a96-8aa8-5ee1992302f7" #@param {type: "string"}
#@markdown Max page size is 1,000
page_size = 90 #@param {type:"integer"}
page_token = "" #@param {type: "string"}

url_params = {'page_size': page_size}
if page_token:
  url_params["page_token"] = page_token
encoded_url_params = urlencode(url_params)

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/detect/rules/{ruleId}:listVersions?{encoded_url_params}"
resp = http_client.request(uri_to_post, "GET")
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  pprint(json_resp)


In [ ]:
#@title StreamDetectionAlerts
#@markdown Continuously receive Detection Engine results over an HTTP stream as the detections are discovered.

#@markdown The StreamDetectionAlerts API only shows detections created by rules whose alerting status was enabled at the time of detection. Enable alerting using the Chronicle UI by setting the Alerting option to enabled from the Rules Dashboard. Each rule has an alerting status.

specify_continuationTime = True #@param {type:"boolean"}
#@markdown Dates/Times are in UTC


#@markdown Timestamp in RFC 3339 format used to request all detections after the specified time. If omitted, all detections after the current time are transmitted.

continuationDate = "2022-11-01" #@param {type: "date"}
#@markdown Time should be in the format HH:MM:SS including leading zeros
continuationTime = "12:12:00" #@param {type: "string"}

if specify_continuationTime:
  continuationTime = f"{continuationDate}T{continuationTime}Z"
  body = {
    "continuationTime": continuationTime
  }
else:
  body = {}

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/detect/rules:streamDetectionAlerts"
resp = session.post(uri_to_post, json=body, stream=True)
if resp.status_code > 299:
  print(resp.text)
else:
  for line in resp.iter_lines():
    print(line)


In [ ]:
#@title StreamTestRule
#@markdown Test a rule over a specified time range. Return any errors and any detections up to the specified maximum without persistence (detections are not retained).

#@markdown Dates/Times are in UTC

startDate = "2024-04-10" #@param {type: "date"}
endDate = "2024-04-19" #@param {type: "date"}
#@markdown Time should be in the format HH:MM:SS including leading zeros
startTime = "12:12:00" #@param {type: "string"}
endTime = "12:12:00" #@param {type: "string"}

startTime = f"{startDate}T{startTime}Z"
endTime = f"{endDate}T{endTime}Z"

#@markdown Maximum number of results to return. Specify a value between 1 and 10,000. The default is 1,000.
maxResults = 1000 #@param {type: "integer"}

#@markdown ##Note
#@markdown You will be prompted to select a file to upload as the test yara rule.

print("Please select and upload yara rule to be tested")
uploaded = files.upload()

os.system(f"mv '{list(uploaded.keys())[0]}' yara_rule.yara")
with open("yara_rule.yara", "r") as rule_file:
  rule = rule_file.read()

# clean up after ourselves
os.system(f"rm yara_rule.yara")

body = {
  "rule": {
    "ruleText": rule
  },
  "startTime": startTime,
  "endTime": endTime,
  "maxResults": maxResults,
}

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/detect/rules:streamTestRule"
resp = session.post(uri_to_post, json=body, stream=True)
if resp.status_code > 299:
  print(resp.text)
else:
  print("Please wait for the results to be streamed in...")
  for line in resp.iter_lines():
    print(line)


In [ ]:
#@title RunRetrohunt
#@markdown Initiate a retrohunt for the specified rule.

ruleId = "" #@param {type: "string"}

#@markdown Dates/Times are in UTC

startDate = "2022-11-01" #@param {type: "date"}
endDate = "2022-11-03" #@param {type: "date"}

#@markdown Time should be in the format HH:MM:SS including leading zeros
startTime = "00:00:01" #@param {type: "string"}
endTime = "23:59:59" #@param {type: "string"}

startTime = f"{startDate}T{startTime}Z"
endTime = f"{endDate}T{endTime}Z"
request_body = {
    "startTime" : startTime,
    "endTime": endTime
}


uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/detect/rules/{ruleId}:runRetrohunt"
resp = http_client.request(uri_to_post, "POST", body=json.dumps(request_body))
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  pprint(json_resp)


In [ ]:
#@title UnarchiveRule
#@markdown Unarchive the specified rule.

ruleId = "ru_4aa9de46-704c-4a96-8aa8-5ee1992302f7" #@param {type: "string"}

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/detect/rules/{ruleId}:unarchive"
resp = http_client.request(uri_to_post, "POST")
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  pprint(json_resp)


In [ ]:
#@title VerifyRule
#@markdown Verifies that a rule is a valid YARA-L 2.0 rule without creating a new rule or evaluating it over data.

#@markdown Choose whether to upload a text file with the rule or use one already in the filesystem of the colab instance.

upload_rule = True #@param {type: "boolean"}
rule_filename = "" #@param {type: "string"}

if upload_rule:
  print("Please select and upload your keyfile")
  uploaded = files.upload()

  rule_filename = list(uploaded.keys())[0]

with open(rule_filename, "r") as rule:
  rule_text = rule.read()

request_body = {
    "ruleText" : rule_text
}

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/detect/rules:verifyRule"
resp = http_client.request(uri_to_post, "POST", body=json.dumps(request_body))
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  pprint(json_resp)


In [ ]:
#@title WaitRetrohunt
#@markdown Enables you to determine whether or not a retrohunt currently in progress has completed within the time period specified. If the retrohunt completes within the time period, WaitRetrohunt returns the retrohunt data. Otherwise, it returns an error indicating that timeout has been reached without the retrohunt completing.

#@markdown Please specify a rule ID to check a retrohunt wait for the latest version of a rule.

#@markdown **OR**

#@markdown  Please specify a version ID to check a retrohunt wait for a specific version of a rule.
latest_version = True #@param {type: "boolean"}
ruleId = "" #@param {type: "string"}
versionId = "" #@param {type: "string"}
retrohuntId = "" #@param {type: "string"}

#@markdown Time in seconds to wait for the retrohunt specified to complete. If timeout is not specified, the default is 5 minutes (300 seconds). The minimum timeout is 3 seconds and the maximum timeout is 5 minutes.
timeout = 300 #@param {type: "slider", min: 3, max: 300}

rule_identifier = ruleId if latest_version else versionId

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/detect/rules/{rule_identifier}/retrohunts/{retrohuntId}:wait"

req_body = {
    "timeout" : str(timeout)
}

# extend http timeout to may be a long call
http_client.timeout = timeout + 60
resp = http_client.request(uri_to_post, "POST", body=json.dumps(req_body))
# return timeout back to default 1 minute
http_client.timeout = 60

json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  pprint(json_resp)



#Search API
Documentation [link](https://cloud.google.com/chronicle/docs/reference/search-api)

[Jump to top](#scrollTo=v0jKbGj8ZZRB)

In [ ]:
#@title GetLog
#@markdown Returns a single raw log given the UID for the event.

logId = "NDhiMzQ1OThmMDc3OTg4MWNjNGQxY2NiNDVmMmUzN2U=" #@param {type: "string"}

decode_base64 = True #@param {type: "boolean"}
save_to_file = False #@param {type: "boolean"}
#@markdown Save file option must be selected to download the file.
download_file_to_computer = False #@param {type: "boolean"}
#@markdown Specify a filename to write the output to. Can be blank if save_to_file is not checked.
file_name = "testfile.json" #@param {type: "string"}

url_params = {}
url_params["name"] = logId

uri = f"https://{region_prefix}backstory.googleapis.com/v1/events/log:get/"
resp = session.get(uri, params=url_params)
json_resp = resp.json()

if decode_base64 and "log" in json_resp.keys():
  json_resp['log']['decodedLog'] = base64.b64decode(json_resp['log']['logBytes']).decode()


if resp.status_code > 299:
  pprint(json_resp.get('error').get('message'))
else:
  if save_to_file:
    if file_name in os.listdir():
      os.remove(file_name)
      print(f"File named {file_name} already exists. Deleting it before writing new data.")
    with open(file_name, "w") as file_to_write:
      file_to_write.write(json.dumps(json_resp))
      print(f"Saved data to: {file_name}")
    if download_file_to_computer:
      files.download(file_name)
  else:
    pprint(json_resp)

In [ ]:
#@title GetEvent
#@markdown Returns a single event given the UID for the event.

uuid = "AAAAAD2pxUYCmstzWDKckUtDt9EAAAAADgAAAAEAAAA=" #@param {type: "string"}

save_to_file = False #@param {type: "boolean"}
#@markdown Save file option must be selected to download the file.
download_file_to_computer = False #@param {type: "boolean"}
#@markdown Specify a filename to write the output to. Can be blank if save_to_file is not checked.
file_name = "testfile.json" #@param {type: "string"}

uri = f"https://{region_prefix}backstory.googleapis.com/v1/event:get?name={uuid}"
resp = session.get(uri)
json_resp = resp.json()

if resp.status_code > 299:
  pprint(json_resp.get('error').get('message'))
else:
  if save_to_file:
    if file_name in os.listdir():
      os.remove(file_name)
      print(f"File named {file_name} already exists. Deleting it before writing new data.")
    with open(file_name, "w") as file_to_write:
      file_to_write.write(json.dumps(json_resp))
      print(f"Saved data to: {file_name}")
    if download_file_to_computer:
      files.download(file_name)
  else:
    pprint(json_resp)

In [ ]:
#@title ListAlerts
#@markdown Returns information about both asset-based and user-based alerts with event timestamps within the specified time range.

#@markdown Specify the maximum number of alerts to return. You can specify between 1 and 100,000. The default is 10,000.
page_size =  100#@param {type:"integer"}
# page_token = "" #@param {type: "string"}
startDate = "2024-01-01" #@param {type: "date"}
endDate = "2024-03-29" #@param {type: "date"}

#@markdown Time should be in the format HH:MM:SS including leading zeros
startTime = "00:00:01" #@param {type: "string"}
endTime = "23:59:59" #@param {type: "string"}

startTime = f"{startDate}T{startTime}Z"
endTime = f"{endDate}T{endTime}Z"

write_to_file = False #@param {type: "boolean"}
#@markdown If writing to file should data also be displayed on screen?
also_output_to_screen = False #@param {type: "boolean"}
#@markdown If the write to file option is selected specify the filename to write data to.
filename = "alerts.json" #@param {type: "string"}

url_params = {}
url_params["start_time"] = startTime
url_params["end_time"] = endTime

url_params['page_size'] = page_size
# encoded_url_params = urlencode(url_params)

uri = f"https://{region_prefix}backstory.googleapis.com/v1/alert/listalerts"
resp = session.get(uri, params=url_params)
json_resp = resp.json()
if resp.status_code > 299:
  pprint(json_resp.get('error').get('message'))
else:
  # pprint(json_resp)
  if write_to_file:
    with open(filename, "w") as output_file:
      if "alerts" in json_resp.keys() or "userAlerts" in json_resp.keys():
        output_file.write(json.dumps(json_resp))
        print(f"Wrote {len(json_resp['alerts']  if 'alerts' in json_resp.keys() else  [])} alerts and {len(json_resp['userAlerts'] if 'userAlerts' in json_resp.keys() else [])} user alerts to {filename}")
        if also_output_to_screen:
          pprint(json_resp)
      else:
        print("No alerts were returned for this query.")
  else:
    pprint(json_resp)

In [ ]:
#@title ListAssets
#@markdown For your enterprise, given the specified artifact, list all the assets that accessed it within the specified time period, including the first and last time those assets accessed the artifact. This call returns a maximum of 100 assets per request. You can specify a narrower time period to reduce the number of assets returned.

#@markdown Specify the maximum number of assets to return. You can specify between 1 and 10,000. The default is 10,000.
page_size =  100#@param {type:"integer"}
# page_token = "" #@param {type: "string"}
startDate = "2024-10-28" #@param {type: "date"}
endDate = "2024-10-28" #@param {type: "date"}

#@markdown Time should be in the format HH:MM:SS including leading zeros
startTime = "03:00:01" #@param {type: "string"}
endTime = "05:00:00" #@param {type: "string"}

startTime = f"{startDate}T{startTime}Z"
endTime = f"{endDate}T{endTime}Z"

artifact_type = "domain_name" #@param ["domain_name", "destination_ip_address", "hash_md5", "hash_sha1", "hash_sha256"]
artifact_value = 'teamazimut.com' #@param {type : "string"}

write_to_file = False #@param {type: "boolean"}
#@markdown If writing to file should data also be displayed on screen?
also_output_to_screen = True #@param {type: "boolean"}
#@markdown If the write to file option is selected specify the filename to write data to.
filename = "assets.json" #@param {type: "string"}

url_params = {}
url_params[f"artifact.{artifact_type}"] = artifact_value
url_params["start_time"] = startTime
url_params["end_time"] = endTime

url_params['page_size'] = page_size
encoded_url_params = urlencode(url_params)

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v1/artifact/listassets?{encoded_url_params}"
resp = http_client.request(uri_to_post, "GET")
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  # pprint(json_resp)
  if write_to_file:
    with open(filename, "w") as output_file:
      if "assets" in json_resp.keys():
        output_file.write(json.dumps(json_resp))
        print(f"Wrote {len(json_resp['assets'])} assets to {filename}")
        if also_output_to_screen:
          pprint(json_resp)
      else:
        print("No assets were returned for this query.")
  else:
    pprint(json_resp)


In [ ]:
#@title ListAssetAliases
#@markdown Lists all the aliases of an asset in an enterprise for the specified asset identifier and time period.



#@markdown Specify the maximum number of assets to return. You can specify between 1 and 10,000. The default is 10,000.
page_size =  100#@param {type:"integer"}
# page_token = "" #@param {type: "string"}
startDate = "2022-11-01" #@param {type: "date"}
endDate = "2022-11-03" #@param {type: "date"}

#@markdown Time should be in the format HH:MM:SS including leading zeros
startTime = "00:00:01" #@param {type: "string"}
endTime = "23:59:59" #@param {type: "string"}

startTime = f"{startDate}T{startTime}Z"
endTime = f"{endDate}T{endTime}Z"

asset_type = "hostname" #@param ["hostname", "asset_ip_address", "mac", "product_id"]
asset_value = 'google.com' #@param {type : "string"}

write_to_file = False #@param {type: "boolean"}
#@markdown If writing to file should data also be displayed on screen?
also_output_to_screen = False #@param {type: "boolean"}
#@markdown If the write to file option is selected specify the filename to write data to.
filename = "assets.json" #@param {type: "string"}

url_params = {}
url_params[f"asset.{asset_type}"] = asset_value
url_params["start_time"] = startTime
url_params["end_time"] = endTime

url_params['page_size'] = page_size
encoded_url_params = urlencode(url_params)

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v1/alias/listassetaliases?{encoded_url_params}"
resp = http_client.request(uri_to_post, "GET")
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  # pprint(json_resp)
  if write_to_file:
    with open(filename, "w") as output_file:
      if "assets" in json_resp.keys():
        output_file.write(json.dumps(json_resp))
        print(f"Wrote {len(json_resp['assets'])} assets to {filename}")
        if also_output_to_screen:
          pprint(json_resp)
      else:
        print("No assets were returned for this query.")
  else:
    pprint(json_resp)


In [ ]:
#@title ListEvents
#@markdown List all the events discovered within your enterprise on a particular device within the specified time range. If you receive the maximum number of events you specified using the page_size parameter (or 10,000, the default), there might still be more events within your Chronicle account. You can narrow the time range and issue the call again to ensure you have visibility into all possible events.

#@markdown Specify the maximum number of events to return. You can specify between 1 and 10,000. The default is 10,000.
page_size =  2#@param {type:"integer"}
# page_token = "" #@param {type: "string"}

startDate = "2022-11-01" #@param {type: "date"}
endDate = "2022-11-03" #@param {type: "date"}
referenceDate = "2022-11-03" #@param {type: "date"}

#@markdown Time should be in the format HH:MM:SS including leading zeros
startTime = "00:00:01" #@param {type: "string"}
endTime = "23:59:59" #@param {type: "string"}
referenceTime = "00:00:12" #@param {type: "string"}

startTime = f"{startDate}T{startTime}Z"
endTime = f"{endDate}T{endTime}Z"
referenceTime = f"{referenceTime}T{referenceTime}Z"

asset_type = "mac_address" #@param ["hostname", "asset_ip_address", "mac_address", "product_id"]
asset_value = '1c:a0:b8:75:ce:f4' #@param {type : "string"}

write_to_file = False #@param {type: "boolean"}
#@markdown If writing to file should data also be displayed on screen?
also_output_to_screen = False #@param {type: "boolean"}
#@markdown If the write to file option is selected specify the filename to write data to.
filename = "events.json" #@param {type: "string"}

url_params = {}
url_params[f"asset.{asset_type}"] = asset_value
url_params["start_time"] = startTime
url_params["end_time"] = endTime
url_params["reference_time"] = referenceTime

url_params['page_size'] = page_size
encoded_url_params = urlencode(url_params)

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v1/asset/listevents?{encoded_url_params}"
resp = http_client.request(uri_to_post, "GET")
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  # pprint(json_resp)
  if write_to_file:
    with open(filename, "w") as output_file:
      if "events" in json_resp.keys():
        output_file.write(json.dumps(json_resp))
        print(f"Wrote {len(json_resp['events'])} events to {filename}")
        if also_output_to_screen:
          pprint(json_resp)
      else:
        print("No events were returned for this query.")
  else:
    pprint(json_resp)


In [ ]:
#@title ListIoCs
#@markdown List all the IoCs discovered within your enterprise within the specified time range. If you receive the maximum number of IoCs you specified using the page_size parameter (or 10,000, the default), there might still be more IoCs discovered in your Chronicle account. You might want to narrow the time range and issue the call again to ensure you have visibility on all possible IoCs.

#@markdown Specify the maximum number of IoCs to return. You can specify between 1 and 10,000. The default is 10,000.
page_size =  10#@param {type:"integer"}
# page_token = "" #@param {type: "string"}
startDate = "2024-10-28" #@param {type: "date"}

#@markdown Time should be in the format HH:MM:SS including leading zeros
startTime = "03:00:01" #@param {type: "string"}

startTime = f"{startDate}T{startTime}Z"

write_to_file = False #@param {type: "boolean"}
#@markdown If writing to file should data also be displayed on screen?
also_output_to_screen = True #@param {type: "boolean"}
#@markdown If the write to file option is selected specify the filename to write data to.
filename = "iocs.json" #@param {type: "string"}

url_params = {}
url_params["start_time"] = startTime
url_params['page_size'] = page_size
encoded_url_params = urlencode(url_params)

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v1/ioc/listiocs?{encoded_url_params}"
resp = http_client.request(uri_to_post, "GET")
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  # pprint(json_resp)
  if write_to_file:
    with open(filename, "w") as output_file:
      if "response" in json_resp.keys() and "matches" in json_resp["response"].keys():
        output_file.write(json.dumps(json_resp))
        print(f"Wrote {len(json_resp['response']['matches'])} IoCs to {filename}")
        if also_output_to_screen:
          pprint(json_resp)
      else:
        print("No events were returned for this query.")
  else:
    pprint(json_resp)


In [ ]:
#@title ListIoCDetails
#@markdown Use this method to submit an artifact indicator and return any threat intelligence associated with that artifact. The threat intelligence information is drawn from your enterprise security systems and from Google's IoC partners (for example, the DHS threat feed).

artifact_type = "destination_ip_address" #@param ["domain_name", "destination_ip_address"]
artifact_value = '71.6.199.23' #@param {type : "string"}

write_to_file = False #@param {type: "boolean"}
#@markdown If writing to file should data also be displayed on screen?
also_output_to_screen = False #@param {type: "boolean"}
#@markdown If the write to file option is selected specify the filename to write data to.
filename = "IoCdetails.json" #@param {type: "string"}

url_params = {}
url_params[f"artifact.{artifact_type}"] = artifact_value

encoded_url_params = urlencode(url_params)

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v1/artifact/listiocdetails?{encoded_url_params}"
resp = http_client.request(uri_to_post, "GET")
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  # pprint(json_resp)
  if write_to_file:
    with open(filename, "w") as output_file:
      if "sources" in json_resp.keys():
        output_file.write(json.dumps(json_resp))
        print(f"Wrote {len(json_resp['sources'])} IoC detail(s) to {filename}")
        if also_output_to_screen:
          pprint(json_resp)
      else:
        print("No IoC Details were returned for this query.")
  else:
    pprint(json_resp)


In [ ]:
#@title ListUserAliases
#@markdown Lists all the aliases of a user in an enterprise for a specified user identifier and time period.

user_type = "username" #@param ["email", "username", "windows_sid", "employee_id", "product_object_id"]
user_value = 'bob' #@param {type : "string"}

startDate = "2022-11-01" #@param {type: "date"}
endDate = "2022-11-03" #@param {type: "date"}

#@markdown Time should be in the format HH:MM:SS including leading zeros
startTime = "00:00:01" #@param {type: "string"}
endTime = "23:59:59" #@param {type: "string"}

startTime = f"{startDate}T{startTime}Z"
endTime = f"{endDate}T{endTime}Z"

write_to_file = False #@param {type: "boolean"}
#@markdown If writing to file should data also be displayed on screen?
also_output_to_screen = False #@param {type: "boolean"}
#@markdown If the write to file option is selected specify the filename to write data to.
filename = "IoCdetails.json" #@param {type: "string"}

url_params = {}
url_params[f"user.{user_type}"] = user_value
url_params["start_time"] = startTime
url_params["end_time"] = endTime

encoded_url_params = urlencode(url_params)

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v1/alias/listuseraliases?{encoded_url_params}"
resp = http_client.request(uri_to_post, "GET")
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  # pprint(json_resp)
  if write_to_file:
    with open(filename, "w") as output_file:
      if "sources" in json_resp.keys():
        output_file.write(json.dumps(json_resp))
        print(f"Wrote {len(json_resp['sources'])} IoC detail(s) to {filename}")
        if also_output_to_screen:
          pprint(json_resp)
      else:
        print("No IoC Details were returned for this query.")
  else:
    pprint(json_resp)


In [ ]:
#@title udmSearch
#@markdown Preview UDM search API

udm_query = " target.ip != \"\"   match:     principal.ip   outcome:     $sent_bytes = sum(network.sent_bytes)" #@param {type: "string"}
limit =  100#@param {type:"integer"}
# page_token = "" #@param {type: "string"}
startDate = "2024-09-01" #@param {type: "date"}
endDate = "2024-09-04" #@param {type: "date"}

#@markdown Time should be in the format HH:MM:SS including leading zeros
startTime = "00:00:01" #@param {type: "string"}
endTime = "23:59:59" #@param {type: "string"}

startTime = f"{startDate}T{startTime}Z"
endTime = f"{endDate}T{endTime}Z"

write_to_file = False #@param {type: "boolean"}
#@markdown If writing to file should data also be displayed on screen?
also_output_to_screen = False #@param {type: "boolean"}
#@markdown If the write to file option is selected specify the filename to write data to.
filename = "udm_query.json" #@param {type: "string"}

url_params = {}
url_params["time_range.start_time"] = startTime
url_params["time_range.end_time"] = endTime
url_params['query'] = udm_query


url_params['limit'] = limit
# encoded_url_params = urlencode(url_params)

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v1/events:udmSearch"
resp = session.get(uri_to_post,params=url_params, timeout=601)
# json_resp = json.loads(resp[1])
if resp.status_code > 299:
  print(resp.text)
else:
  # pprint(json_resp)
  if write_to_file:
    with open(filename, "w") as output_file:
      output_file.write(resp.text)
    if also_output_to_screen:
      pprint(resp.json())
  else:
    pprint(resp.json())


#Reference List API
Documentation can be found [here](https://cloud.google.com/chronicle/docs/preview/reference-lists/reference-lists-api)

[Jump to top](#scrollTo=v0jKbGj8ZZRB)

In [ ]:
#@title CreateReferenceList
#@markdown Creates a list.

name = "test123" #@param {type : "string"}
description = "I am a" #@param {type : "string"}

#@markdown Please specify a comma separated list of lines to add to this list.
lines = "1,2,3,4,5, sdfdsbsdfds    , 6" #@param {type : "string"}
content_type = "CONTENT_TYPE_DEFAULT_STRING" #@param ["CONTENT_TYPE_DEFAULT_STRING", "REGEX", "CIDR"]

#@markdown If checked the uploaded file will be used instead of the lines variable above. New lines will be used as separators
upload_list = False #@param {type: "boolean"}

if upload_list:
  print("Please select your list to upload")
  uploaded = files.upload()

  list_filename = list(uploaded.keys())[0]

  with open(list_filename, "r") as list_file:
    ref_list = [x.strip() for x in list_file.read().split("\n")]
else:
  ref_list = [x.strip() for x in lines.split(",")]

request_body = {
  "name": name,
  "description": description,
  "lines": ref_list,
  "content_type": content_type
}

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/lists"
resp = http_client.request(uri_to_post, "POST", body=json.dumps(request_body))
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  pprint(json_resp)


In [ ]:
#@title GetReferenceList
#@markdown Returns the specified list.

name = "test123" #@param {type : "string"}

## @markdown How much of each reference list to view. BASIC returns the metadata for the list, but not the full contents. FULL returns everything. FULL is the default.

# uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/lists/{name}"
# resp = http_client.request(uri_to_post, "GET")
# json_resp = json.loads(resp[1])
uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/lists/{name}"
resp = session.get(uri_to_post)
json_resp = resp.json()
if resp.status_code > 299:
  pprint(json_resp.get('error').get('message'))
else:
  pprint(json_resp)


In [ ]:
#@title ListReferenceLists
#@markdown Enumerates lists.

#@markdown How much of each reference list to view. BASIC returns the metadata for the list, but not the full contents. FULL returns everything. FULL is the default.
view = "BASIC" #@param ["FULL", "BASIC"]
#@markdown Maximum number of lists to return in this page of reference lists. The default is 100.
page_size = 1000 #@param {type: "integer"}
#@markdown Page token, copied from a previous response, enabling you paginate through lists, starting from where the previous page left off.
page_token = "" #@param {type: "string"}

url_params = {
  "page_size": page_size,
  "view": view,
}

if page_token:
  url_params["page_token"] = page_token

encoded_url_params = urlencode(url_params)

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/lists?{encoded_url_params}"
# resp = http_client.request(uri_to_post, "GET", body=json.dumps(request_body))
resp = http_client.request(uri_to_post, "GET")
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  pprint(json_resp)


In [ ]:
#@title UpdateReferenceList
#@markdown Updates an existing list.

name = "home_subnest" #@param {type : "string"}
description = "" #@param {type : "string"}
content_type = "CIDR" #@param ["CONTENT_TYPE_DEFAULT_STRING", "REGEX", "CIDR"]

#@markdown Please specify a comma separated list of lines to add to this list.
lines = "10.53.0.0/24,10.53.10.0/23,192.168.0.0/24" #@param {type : "string"}

#@markdown If checked the uploaded file will be used instead of the lines variable above. New lines will be used as separators
upload_list = False #@param {type: "boolean"}

if upload_list:
  print("Please select your list to upload")
  uploaded = files.upload()

  list_filename = list(uploaded.keys())[0]

  with open(list_filename, "r") as list_file:
    ref_list = [x.strip() for x in list_file.read().split("\n")]
else:
  ref_list = [x.strip() for x in lines.split(",")]

request_body = {
  "name": name,
  "lines": ref_list,
  "content_type": content_type
}

update_fields = ["list.lines"]

if description:
  update_fields.append("list.description")
  request_body["description"] = description

url_params = {
  "update_mask": ",".join(update_fields)
}

encoded_url_params = urlencode(url_params)

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/lists?{encoded_url_params}"
resp = http_client.request(uri_to_post, "PATCH", body=json.dumps(request_body))
json_resp = json.loads(resp[1])
if resp[0].status != http.HTTPStatus.OK:
  pprint(json_resp.get('error').get('message'))
else:
  pprint(json_resp)


In [ ]:
#@title VerifyReferenceList
#@markdown Validates list content and returns line errors, if any.

#@markdown Please specify a comma separated list of lines to verify.
lines = "1,2,3,4,5, sdfdsbsdfds   , 6" #@param {type : "string"}
content_type = "REGEX" #@param ["CONTENT_TYPE_DEFAULT_STRING", "REGEX", "CIDR"]

#@markdown If checked the uploaded file will be used instead of the lines variable above. New lines will be used as separators
upload_list = False #@param {type: "boolean"}

if upload_list:
  print("Please select your list to upload")
  uploaded = files.upload()

  list_filename = list(uploaded.keys())[0]

  with open(list_filename, "r") as list_file:
    ref_list = [x.strip() for x in list_file.read().split("\n")]
else:
  ref_list = [x.strip() for x in lines.split(",")]

request_body = {
  "lines": ref_list,
  "content_type": content_type
}

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/lists:verifyReferenceList"
resp = session.post(uri_to_post, json=request_body)
json_resp = resp.json()
if resp.status_code > 299:
  pprint(json_resp.get('error').get('message'))
else:
  pprint(json_resp)


#Data Export API
Documentation [link](https://cloud.google.com/chronicle/docs/preview/data-export-api/data-export-api)

[Jump to top](#scrollTo=v0jKbGj8ZZRB)

In [ ]:
#@title CreateDataExport
#@markdown Creates a new data export.

#@markdown Dates/Times are in UTC

startDate = "2025-01-15" #@param {type: "date"}
endDate = "2025-01-30" #@param {type: "date"}
#@markdown Time should be in the format HH:MM:SS including leading zeros
startTime = "00:00:01" #@param {type: "string"}
endTime = "23:59:59" #@param {type: "string"}

startTime = f"{startDate}T{startTime}Z"
endTime = f"{endDate}T{endTime}Z"

#@markdown An individual log type or 'ALL_TYPES' for all log types
logType = "ALL_TYPES" #@param {type: "string"}

#@markdown Path to the customer-provided Google Cloud Storage bucket in projects/\<project-id\>/buckets/\<bucket-name\>" format
gcsBucket = "projects/learning-sql-363317/buckets/data_export_api_target" #@param {type: "string"}



body = {
  "startTime": startTime,
  "endTime": endTime,
  "logType": logType,
  "gcsBucket": gcsBucket,
}

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v1/tools/dataexport"
resp = session.post(uri_to_post, json=body)
if resp.status_code > 299:
  print(resp.text)
else:
  pprint(resp.json())


In [ ]:
#@title GetDataExport
#@markdown Returns an existing data export.

dataExportId = "73dccbe4-fae2-4208-80a8-dc62b9e1b7a8" #@param {type: "string"}

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v1/tools/dataexport/{dataExportId}"
resp = session.get(uri_to_post)
if resp.status_code > 299:
  print(resp.text)
else:
  pprint(resp.json())


In [ ]:
#@title CancelDataExport
#@markdown Cancels an existing data export request.

#@markdown Note: Only IN_QUEUE data exports can be canceled.

dataExportId = "47d6ff7f-503e-4c31-9a5a-73ddc17bf986" #@param {type: "string"}

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v1/tools/dataexport/{dataExportId}:cancel"
resp = session.post(uri_to_post)
if resp.status_code > 299:
  print(resp.text)
else:
  pprint(resp.json())


In [ ]:
#@title ListAvailableLogTypes
#@markdown List all available log types and their time range.

#@markdown A time range is optional. If not specified, the value is UNIX epoch time starting on January 1st, 1970 at UTC.
include_time_range = True #@param {type: "boolean"}

#@markdown Dates/Times are in UTC

startDate = "2025-01-01" #@param {type: "date"}
endDate = "2025-01-29" #@param {type: "date"}
#@markdown Time should be in the format HH:MM:SS including leading zeros
startTime = "01:00:00" #@param {type: "string"}
endTime = "23:59:59" #@param {type: "string"}

startTime = f"{startDate}T{startTime}Z"
endTime = f"{endDate}T{endTime}Z"


parameters = {
  "startTime": startTime,
  "endTime": endTime
}

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v1/tools/dataexport/listavailablelogtypes"
if include_time_range:
  resp = session.get(uri_to_post, params=parameters)
else:
  resp = session.get(uri_to_post)
if resp.status_code > 299:
  print(resp.text)
else:
  pprint(resp.json())


#DataTap Configuration API
Documentation [link](https://cloud.google.com/chronicle/docs/preview/datatap-config/datatapconfig-api)

[Jump to top](#scrollTo=v0jKbGj8ZZRB)

In [ ]:
#@title Create
#@markdown Creates a DataTap configuration.

displayName = "" #@param {type: "string"}
#@markdown Use the following format for the topic: **projects/\<project_id\>/topics/\<topicId\>**
topic = "" #@param {type: "string"}
filter = "ALERT_UDM_EVENTS" #@param ["ALL_UDM_EVENTS", "ALERT_UDM_EVENTS"]
serializationFormat = "JSON" #@param ["JSON", "MARSHALLED_PROTO"]

body = {
  "displayName": displayName,
  "cloudPubsubSink": {
    "topic": topic,
  },
  "filter": filter,
  "serializationFormat": serializationFormat
}

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v1/dataTaps"
resp = session.post(uri_to_post, json=body)
if resp.status_code > 299:
  print(resp.text)
else:
  pprint(resp.json())


In [ ]:
#@title Update
#@markdown Updates a DataTap configuration.

tapId =  "" #@param {type: "string"}
displayName = "" #@param {type: "string"}
#@markdown Use the following format for the topic: **projects/\<project_id\>/topics/\<topicId\>**
topic = "" #@param {type: "string"}
filter = "ALERT_UDM_EVENTS" #@param ["ALL_UDM_EVENTS", "ALERT_UDM_EVENTS"]
serializationFormat = "JSON" #@param ["JSON", "MARSHALLED_PROTO"]

body = {
  "name" : f"dataTaps/{tapId}",
  "displayName": displayName,
  "cloudPubsubSink": {
    "topic": topic,
  },
  "filter": filter,
  "serializationFormat": serializationFormat
}

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v1/dataTaps/{tapId}"
resp = session.patch(uri_to_post, json=body)
if resp.status_code > 299:
  print(resp.text)
else:
  pprint(resp.json())


In [ ]:
#@title Delete
#@markdown Deletes a DataTap configuration.

tapId =  "" #@param {type: "string"}

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v1/dataTaps/{tapId}"
resp = session.delete(uri_to_post)
if resp.status_code > 299:
  print(resp.text)
else:
  pprint(resp.json())


In [ ]:
#@title Get
#@markdown Get a specific DataTap configuration.

tapId =  "" #@param {type: "string"}

body = {
  "name": f"dataTaps/{tapId}",
}

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v1/dataTaps/{tapId}"
resp = session.get(uri_to_post, json=body)
if resp.status_code > 299:
  print(resp.text)
else:
  pprint(resp.json())


In [ ]:
#@title List
#@markdown List all the DataTap configurations of a customer.

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v1/dataTaps"
resp = session.get(uri_to_post)
if resp.status_code > 299:
  print(resp.text)
else:
  pprint(resp.json())


#Forwarder API
Documentation [link](https://cloud.google.com/chronicle/docs/install/forwarder-management-api)

[Jump to top](#scrollTo=v0jKbGj8ZZRB)

In [ ]:
#@title Create Forwarder
#@markdown Creates a new forwarder in the Chronicle instance. The new forwarder will include any forwarder configuration values provided in the request body. Configuration values for collectors must be specified using Create Collector after using Create Forwarder.

#@markdown For certain settings, configuration values that are missing or zero-valued in the request body are set to default values. For details about forwarder fields and values, see [Forwarder configuration fields](https://cloud.google.com/chronicle/docs/preview/forwarder-management-api/forwarder-management-api#forwarder-config-fields).

displayName = "windows-test" #@param {type: "string"}

#@markdown **[OPTIONAL]** The configuration settings for this forwarder. Please paste a JSON object of the config.

#@markdown A helper is included at the bottom of the forwarder API section to help generate this.

config = "" #@param {type: "string"}

body = {
    "displayName" : displayName
}

if config:
  config = json.loads(config)
  body['config'] = config



uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/forwarders"

# pprint(body)

resp = session.post(uri_to_post, json=body)
if resp.status_code > 299:
  print(resp.text)
else:
  pprint(resp.json())


In [ ]:
#@title Get Forwarder
#@markdown Returns a forwarder.

forwarderID = "67971dbf-2d52-4251-aefd-c329d1d9bf1a" #@param {type : "string"}

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/forwarders/{forwarderID}"

resp = session.get(uri_to_post)
if resp.status_code > 299:
  print(resp.text)
else:
  pprint(resp.json())


In [ ]:
#@title List Forwarders
#@markdown Lists all of the forwarders for a Chronicle instance.

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/forwarders"
resp = session.get(uri_to_post)
if resp.status_code > 299:
  print(resp.text)
else:
  pprint(resp.json())


In [ ]:
#@title Update Forwarder
#@markdown You can update a forwarder by using the updateMask URL query parameter to specify the fields to update.

forwarderID =  "a2ece8f1-78d5-4d0b-81a0-47fc1a8b3b38" #@param {type: "string"}

#@markdown If desired specify an updaetd display name
updated_display_name = "I_like_cheese" #@param {type: "string"}
#@markdown The updated config should look just like the config for generating a forwarder, but only needs the keys/values that need to be updated and no others.
updated_config = "{\"uploadCompression\": true, \"regexFilters\": [{\"description\": \"Just a test of some stuff\", \"regexp\": \".*BLOCKME.*\", \"behavior\": \"BLOCK\"}, {\"description\": \"Just a test of some stuff\", \"regexp\": \".*BLOCKMETOO.*\", \"behavior\": \"BLOCK\"}], \"serverSettings\": {\"state\": \"ACTIVE\", \"httpSettings\": {\"port\": 8081}}}" #@param {type: "string"}

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/forwarders/{forwarderID}"

def mask_array_gen(path,my_dict,path_array = []):
    for k,v in my_dict.items():
        if isinstance(v,dict):
          if path:
            mask_array_gen(path+"."+k,v,path_array)
          else:
            mask_array_gen(k,v,path_array)
        else:
          if path:
            # handle exceptions for repeated fields.
            if path.startswith("config.regexFilters"):
              if "config.regexFilters" not in path_array:
                path_array.append("config.regexFilters")
            elif path.startswith("metadata.labels"):
              if "metadata.labels" not in path_array:
                path_array.append("metadata.labels")
            else:
              path_array.append(path + "." + k)
          else:
            path_array.append(k)
    return {"updateMask" : ",".join(path_array)}

if updated_config:
  updated_config = json.loads(updated_config)
  params = mask_array_gen("config", updated_config)
else:
  params = {
      "updateMask" : ""
  }

if updated_display_name and params["updateMask"]:
  params["updateMask"] += ",displayName"
elif updated_display_name:
  params["updateMask"] = "displayName"

payload_body = {}

if updated_display_name:
  payload_body["display_name"] = updated_display_name

if updated_config:
  payload_body["config"] = updated_config

resp = session.patch(uri_to_post, json=payload_body, params=params)
if resp.status_code > 299:
  print(resp.text)
else:
  pprint(resp.json())

In [ ]:
#@title Delete Forwarder
#@markdown Deletes a forwarder.

forwarderID = "e97a8870-24c9-4bac-aa70-2991ae4a3658" #@param {type : "string"}

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/forwarders/{forwarderID}"

resp = session.delete(uri_to_post)
if resp.status_code > 299:
  print(resp.text)
else:
  pprint(resp.json())


In [ ]:
#@title Generate Forwarder Files
#@markdown Generates and returns the contents of the forwarder's configuration **(.conf)** and authentication **(_auth.conf)** files.

forwarderID = "5e4535c8-651a-49e0-914b-31494df84434" #@param {type : "string"}
download_files = True #@param {type: "boolean"}

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/forwarders/{forwarderID}:generateForwarderFiles"

resp = session.get(uri_to_post)
if resp.status_code > 299:
  print(resp.text)
else:
  if download_files:
    if forwarderID in os.listdir():
      os.system(f"rm -rf {forwarderID}")
    os.mkdir(forwarderID)
    with open(f"./{forwarderID}/config.auth", "w") as auth_file:
      auth_file.write(resp.json()["auth"])
    with open(f"./{forwarderID}/config.conf", "w") as auth_file:
      auth_file.write(resp.json()["config"])
    if f"{forwarderID}.zip" in os.listdir():
      os.remove(f"{forwarderID}.zip")
    os.system(f"zip -r {forwarderID}.zip {forwarderID}")
    files.download(f"{forwarderID}.zip")
    print(f"{forwarderID}.zip has been downloaded with conig data.")
  else:
    pprint(resp.json())


##Forwarder config generation helper

The block is folded under this header beacuse it is long and takes a long time to scroll past.

In [ ]:
#@title Regex Filter Builder
#@markdown This will help build out regex filters for forwarders with multiple regular expressions.


#@markdown If **start_clean** is set to true reset the list of regex rules and start from scratch with a clean list.
start_clean = False #@param {type: "boolean"}
if "regex_array" not in locals() or start_clean:
  regex_array = []

#@markdown Describes what is being filtered and why.

regexFilters_description = "Just a test of some stuff" #@param {type: "string"}

#@markdown The regular expression used to match against each incoming line.

regexFilters_regexp = ".*BLOCKMETOO.*" #@param {type: "string"}

#@markdown Specifies the state of the server functionality. Valid values are: **ALLOW**: This state allows the filtered line to be uploaded. **BLOCK**: This state prevents the filtered line from being uploaded.

regexFilters_behavior = "BLOCK" #@param ["", "ALLOW", "BLOCK"]

regex_array.append({
    "description": regexFilters_description,
    "regexp": regexFilters_regexp,
    "behavior": regexFilters_behavior
})

print(json.dumps(regex_array))


In [ ]:
#@title Generate config
#@markdown This is a helper to build out a config for the create forwarder endpoint (and others)

#@markdown All fields are optional.

#@markdown If true, batches of data are compressed before upload. The default is true.

uploadCompression = True #@param {type: "boolean"}

#@markdown The namespace for identifying logs from this forwarder. **Note:** This is a global setting that applies to the forwarder and the forwarder's collectors, unless it is overridden at the collector level.

assetNamespace = "" #@param {type: "string"}

#@markdown A list of arbitrary key:value pairs that can be specified in the forwarder configuration. **Note:** This is a global setting that applies to the forwarder and the forwarder's collectors, unless it is overridden at the collector level.

#@markdown Please enter them in the form **key1:value1,key2:value2...**

labels =  "windows-server" #@param {type: "string"}

#@markdown Paste a regex filter from the Regex Filter Builder in the cell above

regexFilters_object = "" #@param {type :"string"}

#@markdown Specifies the state of the server functionality. Valid values are: **ACTIVE**: In this state, server settings are applied as specified. **SUSPENDED** In this state, server settings are not applied.

serverSettings_state = "ACTIVE" #@param ["", "ACTIVE", "SUSPENDED"]

#@markdown The number of seconds after which the forwarder returns a bad readiness/health check and still accepts new connections. This is also the time to wait between receiving a signal to stop and actually beginning the shutdown of the server itself. This allows the load balancer time to remove the forwarder from the pool. The default value is **15**.

serverSettings_gracefulTimeout = 0 #@param {type : "integer"}

#@markdown The number of seconds after which the forwarder waits for active connections to successfully close on their own before being closed by the server. The default value is **10**.

serverSettings_drainTimeout =  0 #@param {type : "integer"}

#@markdown The port number that the HTTP server listens on for health checks from the load balancer. Must be between 1024-65535. The default is **8080**.

serverSettings_httpSettings_port = 8081 #@param {type : "integer"}

#@markdown The IP address, or hostname that can be resolved to IP addresses, that the server should listen to. The default value is 0.0.0.0 (the local system).

serverSettings_httpSettings_host = "" #@param {type : "string"}

#@markdown The maximum number of seconds allowed to read entire requests, which includes the header and the body. The default value is **3**.

serverSettings_httpSettings_readTimeout = 0 #@param {type: "integer"}

#@markdown The maximum number of seconds allowed to read request headers. The default value is **3**.

serverSettings_httpSettings_readHeaderTimeout = 0 #@param {type: "integer"}

#@markdown The maximum number of seconds allowed to send a response. The default value is **3**.

serverSettings_httpSettings_writeTimeout = 0 #@param {type: "integer"}

#@markdown The maximum number of seconds to wait for the next request when idle connections are enabled. The default is **3**.

serverSettings_httpSettings_idleTimeout = 0 #@param {type: "integer"}

#@markdown The status code returned when a liveness check is received and the forwarder is available. The default is **204**.

serverSettings_routeSettings_availableStatusCode = 0 #@param {type: "integer"}

#@markdown The status code returned when the forwarder is ready to accept traffic. The default is **204**.

serverSettings_routeSettings_readyStatusCode = 0 #@param {type: "integer"}

#@markdown The status code returned when the forwarder is not ready to accept traffic. The default is **503**.

serverSettings_httpSettings_routeSettings_unreadyStatusCode = 0 #@param {type: "integer"}

config = {}

config["uploadCompression"] = uploadCompression

if assetNamespace:
  if "metadata" not in config.keys():
    config["metadata"] = {}
  config["metadata"]["assetNamespace"] = assetNamespace

if labels:
  if "metadata" not in config.keys():
    config["metadata"] = {}
  config["metadata"]['labels'] = []
  labels = labels.split(",")

  for label in labels:
    label = label.strip()
    label = label.split(":")
    config["metadata"]['labels'].append({"key" : label[0].strip(), "value" : label[1].strip()})

if regexFilters_object:
  config["regexFilters"] = json.loads(regexFilters_object)

# if regexFilters_regexp:
#   config["regexFilters"] = {}
#   config["regexFilters"]["regexp"] = regexFilters_regexp

#   if regexFilters_behavior:
#     config["regexFilters"]["behavior"] = regexFilters_behavior

#   if regexFilters_description:
#     config["regexFilters"]["description"] = regexFilters_description

if serverSettings_state:
  config["serverSettings"] = {}
  config["serverSettings"]["state"] = serverSettings_state

if serverSettings_gracefulTimeout:
  if "serverSettings" not in config.keys():
    config["serverSettings"] = {}
  config["serverSettings"]["gracefulTimeout"] = serverSettings_gracefulTimeout

if serverSettings_drainTimeout:
  if "serverSettings" not in config.keys():
    config["serverSettings"] = {}
  config["serverSettings"]["drainTimeout"] = serverSettings_drainTimeout

if serverSettings_httpSettings_port:
  if "serverSettings" not in config.keys():
    config["serverSettings"] = {}
  if "httpSettings" not in config["serverSettings"].keys():
    config["serverSettings"]["httpSettings"] = {}
  config["serverSettings"]["httpSettings"]["port"] = serverSettings_httpSettings_port

if serverSettings_httpSettings_host:
  if "serverSettings" not in config.keys():
    config["serverSettings"] = {}
  if "httpSettings" not in config["serverSettings"].keys():
    config["serverSettings"]["httpSettings"] = {}
  config["serverSettings"]["httpSettings"]["host"] = serverSettings_httpSettings_host

if serverSettings_httpSettings_readTimeout:
  if "serverSettings" not in config.keys():
    config["serverSettings"] = {}
  if "httpSettings" not in config["serverSettings"].keys():
    config["serverSettings"]["httpSettings"] = {}
  config["serverSettings"]["httpSettings"]["readTimeout"] = serverSettings_httpSettings_readTimeout

if serverSettings_httpSettings_readHeaderTimeout:
  if "serverSettings" not in config.keys():
    config["serverSettings"] = {}
  if "httpSettings" not in config["serverSettings"].keys():
    config["serverSettings"]["httpSettings"] = {}
  config["serverSettings"]["httpSettings"]["readHeaderTimeout"] = serverSettings_httpSettings_readHeaderTimeout

if serverSettings_routeSettings_availableStatusCode:
  if "serverSettings" not in config.keys():
    config["serverSettings"] = {}
  if "httpSettings" not in config["serverSettings"].keys():
    config["serverSettings"]["httpSettings"] = {}
  if "routeSettings" not in config["serverSettings"]["httpSettings"].keys():
    config["serverSettings"]["httpSettings"]["routeSettings"] = {}
  config["serverSettings"]["httpSettings"]["routeSettings"]["availableStatusCode"] = serverSettings_routeSettings_availableStatusCode

if serverSettings_routeSettings_readyStatusCode:
  if "serverSettings" not in config.keys():
    config["serverSettings"] = {}
  if "httpSettings" not in config["serverSettings"].keys():
    config["serverSettings"]["httpSettings"] = {}
  if "routeSettings" not in config["serverSettings"]["httpSettings"].keys():
    config["serverSettings"]["httpSettings"]["routeSettings"] = {}
  config["serverSettings"]["httpSettings"]["routeSettings"]["readyStatusCode"] = serverSettings_routeSettings_readyStatusCode

if serverSettings_httpSettings_routeSettings_unreadyStatusCode:
  if "serverSettings" not in config.keys():
    config["serverSettings"] = {}
  if "httpSettings" not in config["serverSettings"].keys():
    config["serverSettings"]["httpSettings"] = {}
  if "routeSettings" not in config["serverSettings"]["httpSettings"].keys():
    config["serverSettings"]["httpSettings"]["routeSettings"] = {}
  config["serverSettings"]["httpSettings"]["routeSettings"]["unreadyStatusCode"] = serverSettings_httpSettings_routeSettings_unreadyStatusCode


print("Please copy the output and paste it into the config cell in the appropriate call.")
print(json.dumps(config))

#Collector API

Same doc as the forwader API, but [doc link](https://cloud.google.com/chronicle/docs/preview/forwarder-management-api/forwarder-management-api#collector_api_reference) for completness.

[Jump to top](#scrollTo=v0jKbGj8ZZRB)

In [ ]:
#@title Create Collector
#@markdown Creates a new collector in the Chronicle account. Configuration values for collectors must be specified using Create Collector after using Create Forwarder.

#@markdown For certain settings, configuration values that are missing or zero-valued in the request body are set to default values. For details about collector configuration fields and values, see [Collector configuration fields](https://cloud.google.com/chronicle/docs/preview/forwarder-management-api/forwarder-management-api#collector-config-fields).

#@markdown The forawrder ID on which to create the collector.
forwarderID = "5daf55a9-74ee-4c99-816d-1506e5e5f426" #@param {type: "string"}

displayName = "windows_syslog_listener" #@param {type: "string"}

#@markdown **[OPTIONAL]** The configuration settings for this collector. Please paste a JSON object of the config.

#@markdown A helper is included at the bottom of the collector API section to help generate this.

config = "{\"logType\": \"BOX\", \"syslogSettings\": {\"protocol\": \"UDP\", \"address\": \"0.0.0.0\", \"port\": 10514}}" #@param {type: "string"}

#@markdown Specifies the current state of the collector. Valid values are:
state = "ACTIVE" #@param ["", "ACTIVE", "SUSPENDED"]
body = {
    "displayName" : displayName
}

if config:
  config = json.loads(config)
  body['config'] = config



uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/forwarders/{forwarderID}/collectors"

# pprint(body)

resp = session.post(uri_to_post, json=body)
if resp.status_code > 299:
  print(resp.text)
else:
  pprint(resp.json())


In [ ]:
#@title Get Collector
#@markdown Returns a collector.

forwarderID = "b34cab48-2a22-4897-8747-3580eff0dd36" #@param {type : "string"}
collectorID = "" #@param {type : "string"}

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/forwarders/{forwarderID}/collectors/{collectorID}"

resp = session.get(uri_to_post)
if resp.status_code > 299:
  print(resp.text)
else:
  pprint(resp.json())


In [ ]:
#@title List Collectors
#@markdown Lists the existing collectors for the specified forwarder.

forwarderID = "a2ece8f1-78d5-4d0b-81a0-47fc1a8b3b38" #@param {type : "string"}

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/forwarders/{forwarderID}/collectors"

resp = session.get(uri_to_post)
if resp.status_code > 299:
  print(resp.text)
else:
  pprint(resp.json())


In [ ]:
#@title Update Collector

forwarderID =  "a2ece8f1-78d5-4d0b-81a0-47fc1a8b3b38" #@param {type: "string"}
collectorID = "670a1475-fccb-4f30-88e7-9f0951a2b4d9" #@param {type: "string"}

#@markdown The updated config should look just like the config for generating a forwarder, but only needs the keys/values that need to be updated and no others.
updated_display_name = "swiss" #@param {type:"string"}
updated_config = "{\"logType\": \"D3_BANKING\", \"regexFilters\": [{\"description\": \"Just a test of some stuff\", \"regexp\": \".*BLOCKME.*\", \"behavior\": \"BLOCK\"}, {\"description\": \"Just a test of some stuff\", \"regexp\": \".*BLOCKMETOO.*\", \"behavior\": \"BLOCK\"}]}" #@param {type: "string"}

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/forwarders/{forwarderID}/collectors/{collectorID}"

def mask_array_gen(path,my_dict,path_array = []):
    for k,v in my_dict.items():
        if isinstance(v,dict):
          if path:
            mask_array_gen(path+"."+k,v,path_array)
          else:
            mask_array_gen(k,v,path_array)
        else:
          if path:
            # handle exceptions for repeated fields.
            if path.startswith("config.regexFilters"):
              if "config.regexFilters" not in path_array:
                path_array.append("config.regexFilters")
            elif path.startswith("metadata.labels"):
              if "metadata.labels" not in path_array:
                path_array.append("metadata.labels")
            else:
              path_array.append(path + "." + k)
          else:
            path_array.append(k)
    return {"updateMask" : ",".join(path_array)}

if updated_config:
  updated_config = json.loads(updated_config)
  params = mask_array_gen("config", updated_config)
else:
  params = {
      "updateMask" : ""
  }

if updated_display_name and params["updateMask"]:
  params["updateMask"] += ",displayName"
elif updated_display_name:
  params["updateMask"] = "displayName"

payload_body = {}

if updated_display_name:
  payload_body["display_name"] = updated_display_name

if updated_config:
  payload_body["config"] = updated_config


resp = session.patch(uri_to_post, json=payload_body, params=params)
if resp.status_code > 299:
  print(resp.text)
else:
  pprint(resp.json())

In [ ]:
#@title Delete Collector
#@markdown Deletes a collector.

forwarderID = "a2ece8f1-78d5-4d0b-81a0-47fc1a8b3b38" #@param {type : "string"}
collectorID = "5882a551-b8fd-47fc-9d6f-f1526b3f2082" #@param {type : "string"}

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v2/forwarders/{forwarderID}/collectors/{collectorID}"

resp = session.delete(uri_to_post)
if resp.status_code > 299:
  print(resp.text)
else:
  pprint(resp.json())


##Collector config generation helper

The block is folded under this header beacuse it is long and takes a long time to scroll past.

In [ ]:
#@title Regex Filter Builder
#@markdown This will help build out regex filters for collectors with multiple regular expressions.


#@markdown If **start_clean** is set to true reset the list of regex rules and start from scratch with a clean list.
start_clean = False #@param {type: "boolean"}
if "regex_array" not in locals() or start_clean:
  regex_array = []

#@markdown Describes what is being filtered and why.

regexFilters_description = "Just a test of some stuff" #@param {type: "string"}

#@markdown The regular expression used to match against each incoming line.

regexFilters_regexp = ".*BLOCKMETOO.*" #@param {type: "string"}

#@markdown Specifies the state of the server functionality. Valid values are: **ALLOW**: This state allows the filtered line to be uploaded. **BLOCK**: This state prevents the filtered line from being uploaded.

regexFilters_behavior = "BLOCK" #@param ["", "ALLOW", "BLOCK"]

regex_array.append({
    "description": regexFilters_description,
    "regexp": regexFilters_regexp,
    "behavior": regexFilters_behavior
})

print(json.dumps(regex_array))


In [ ]:
#@title Generate Collector config
#@markdown logtype is a required field
logType = "BOX" #@param ["",'ABNORMAL_SECURITY','ABSOLUTE','ACALVIO','ACCELLION','ACTIVE_SYNC','ADAUDIT_PLUS','ADFS','ADMANAGER_PLUS','ADSELFSERVICE_PLUS','AIRDEFENSE','AIRWATCH','AIX_SYSTEM','AI_HUNTER','AKAMAI_CLOUD_MONITOR','AKAMAI_DDOS','AKAMAI_DHCP','AKAMAI_DNS','AKAMAI_EAA','AKAMAI_ETP','AKAMAI_WAF','ALERTLOGIC_NOTIFICATIONS','ALGOSEC','ALLOT_NETENFORCER','ANOMALI_IOC','ANSIBLE_AWX','APACHE','APACHE_KAFKA_AUDIT','APACHE_SPAMASSASSIN','APC_SMART_UPS','APC_STRUXUREWARE','APPIAN_CLOUD','APPOMNI','APTOS_EOM','AQUA_SECURITY','ARBOR_EDGE_DEFENSE','ARCHER_IRM','ARCSIGHT_CEF','AREA1','ARISTA_SWITCH','ARKIME_PCAP','ARMIS','ARMOR_ANYWHERE','ARRAYNETWORKS_VPN','ARRAY_NETWORKS_WAF','ARUBA_AIRWAVE','ARUBA_IPS','ARUBA_WIRELESS','ARXAN_THREAT_ANALYTICS','ASOC_ALERT','ASSET_STATIC_IP','ATLASSIAN_CONFLUENCE','ATLASSIAN_JIRA','ATTIVO','ATT_NETBOND','AUDITD','AUTHENTIC8_SILO','AUTHX','AUTH_ZERO','AUTOMATION_ANYWHERE','AUTOMOX_EPM','AVANAN_EMAIL','AVATIER','AVAYA_IVR','AVAYA_WIRELESS','AWS_AURORA','AWS_CLOUDFRONT','AWS_CLOUDTRAIL','AWS_CLOUDWATCH','AWS_CONFIG','AWS_CONTROL_TOWER','AWS_ELB','AWS_KMS','AWS_MACIE','AWS_REDSHIFT','AWS_ROUTE_53','AWS_S3_SERVER_ACCESS','AWS_SECURITY_HUB','AWS_SESSION_MANAGER','AWS_VPC_FLOW','AWS_WAF','AXONIUS','AZURE','AZURE_ACTIVITY','AZURE_AD','AZURE_AD_AUDIT','AZURE_AD_CONTEXT','AZURE_ATP','AZURE_COSMOS_DB','AZURE_FIREWALL','AZURE_MDM_INTUNE','AZURE_NSG_FLOW','AZURE_RESOURCE_LOGS','AZURE_SECURITY_CENTER','AZURE_SQL','BALABIT','BARRACUDA_CLOUDGEN_ACCESS','BARRACUDA_EMAIL','BARRACUDA_FIREWALL','BARRACUDA_WAF','BEYONDTRUST_BEYONDINSIGHT','BEYONDTRUST_CPB','BEYONDTRUST_ENDPOINT','BEYONDTRUST_REMOTE_ACCESS','BIGSWITCH_BCF','BIND_DNS','BITDEFENDER','BLUECAT_DDI','BLUECAT_EDGE','BLUECOAT_WEBPROXY','BLUE_PRISM','BMC_AMI_DEFENDER','BMC_HELIX_DISCOVERY','BOMGAR','BOX','BRICATA_NDR','BRIVO','BROCADE_SERVERIRON','BROCADE_SWITCH','BRO_DHCP','BRO_HTTP','BRO_JSON','BRO_TSV','BT_IPCONTROL','CAMEYO_BYO_CLOUD','CASSANDRA','CATO_NETWORKS','CATO_SDWAN','CA_ACCESS_CONTROL','CA_ACF2','CA_LDAP','CA_SSO_WEB','CB_APP_CONTROL','CB_EDR','CENTRIFY_SSO','CENTRIPETAL_IOC','CEQUENCE_BOT_DEFENSE','CHECKPOINT_EDR','CHECKPOINT_EMAIL','CHECKPOINT_FIREWALL','CHECKPOINT_HARMONY','CISCO_ACE','CISCO_ACI','CISCO_ACS','CISCO_AMP','CISCO_APIC','CISCO_ASA_FIREWALL','CISCO_CLOUDLOCK_CASB','CISCO_CTS','CISCO_DHCP','CISCO_DNAC','CISCO_DNS','CISCO_EMAIL_SECURITY','CISCO_FIREPOWER_FIREWALL','CISCO_FIRESIGHT','CISCO_IOS','CISCO_ISE','CISCO_MERAKI','CISCO_NX_OS','CISCO_PRIME','CISCO_ROUTER','CISCO_SECURE_MALWARE_ANALYTICS','CISCO_SECURE_WORKLOAD','CISCO_SMA','CISCO_STEALTHWATCH','CISCO_SWITCH','CISCO_TACACS','CISCO_UCM','CISCO_UCS','CISCO_VPN','CISCO_WIPS','CISCO_WIRELESS','CISCO_WSA','CIS_ALBERT_ALERT','CITRIX_MONITOR','CITRIX_NETSCALER','CITRIX_NETSCALER_WEB_LOGS','CITRIX_SDWAN','CITRIX_STOREFRONT','CITRIX_WEB_GATEWAY','CITRIX_WORKSPACE','CLAM_AV','CLEARPASS','CLEARSENSE','CLOUDFLARE','CLOUDFLARE_BOT_MANAGEMENT','CLOUDFLARE_WAF','CLOUDGENIX_SDWAN','CLOUDIAN_HYPERSTORE','CLOUDM','CLOUDPASSAGE_CSM','CLOUDPASSAGE_FIM','CLOUDPASSAGE_LIDS','CLOUDPASSAGE_SVM','CLOUD_IDENTITY_CONTEXT','CLOUD_PASSAGE','CMD','CODE42','CODE42_INCYDR','CODE_WORLDWIDE','COFENSE_TRIAGE','COFENSE_VISION','COHESITY','COMMVAULT','COMODO_AV','CONFLUENT_AUDIT','CORELIGHT','CORTEX_XDR','COVID_CTC_IOC','CRADLEPOINT_NETCLOUD','CROWDSTRIKE_IOC','CSV_CUSTOM_CMDB','CSV_CUSTOM_IOC','CS_CEF_EDR','CS_EDR','CS_STREAM','CUSTOMER_ALERT','CUSTOM_APPLICATION_ACCESS','CUSTOM_DNS','CUSTOM_SECURITY_DATA_ANALYTICS','CYBERARK','CYBEREASON_EDR','CYCODE','CYLANCE','CYLANCE_PROTECT','CYOLO_ZTNA','D3_BANKING','D3_SECURITY','DARKTRACE','DATADOG','DATALOCKER_SAFECONSOLE','DATAWATCH','DB2_DB','DEEPFENCE','DEEP_INSTINCT_EDR','DELL_EMC_AVAMAR','DELL_EMC_CLOUDLINK','DELL_EMC_DATA_DOMAIN','DELL_EMC_NAS','DELL_OPENMANAGE','DELL_SWITCH','DESIGN_PROFIT_CENTRAL_SERVER','DHS_IOC','DIGITALGUARDIAN_EDR','DIGITAL_SHADOWS_IOC','DIGITAL_SHADOWS_SEARCHLIGHT','DMP_ENTRE','DOCKER','DREMIO_DATA_LAKEHOUSE','DROPBOX','DUO_ADMIN','DUO_AUTH','DUO_CASB','DUO_CONTEXT','DUO_NETWORK_GATEWAY','DUO_TELEPHONY','DUO_USER_CONTEXT','DYNATRACE','E2_SOLUTIONS','EATON_UPS','ECAR','ECAR_BRO','EFFICIENTIP_DDI','EIQ_EDR','ELASTIC_AUDITBEAT','ELASTIC_FILEBEAT','ELASTIC_METRICBEAT','ELASTIC_PACKETBEATS','ELASTIC_SEARCH','ELASTIC_WINLOGBEAT','ENDGAME_EDR','ENDPOINT_PROTECTOR_DLP','ENTRUST_NTP_SERVER','EPIC','ESET_AV','ESET_EDR','ESET_IOC','ESTAR','ETQ_RELIANCE','ET_PRO_IOC','EXABEAM_FUSION_XDR','EXCHANGE_MAIL','EXTRAHOP','EXTRAHOP_DHCP','EXTRAHOP_DNS','EXTREME_SWITCH','F5_ASM','F5_BIGIP_LTM','F5_BOT','F5_DNS','F5_SHAPE','F5_VPN','FALCO_IDS','FASTLY_WAF','FIDELIS_ENDPOINT','FIDELIS_NETWORK','FILEZILLA_FTP','FILE_SCANNING_FRAMEWORK','FIREEYE_ALERT','FIREEYE_EMPS','FIREEYE_ETP','FIREEYE_HX','FIREEYE_NX','FIREMON_FIREWALL','FLASHPOINT_IOC','FLUENTD','FORCEPOINT_CASB','FORCEPOINT_DLP','FORCEPOINT_FIREWALL','FORCEPOINT_WEBPROXY','FORESCOUT_NAC','FORSETI','FORTANIX_DSM','FORTINET_DHCP','FORTINET_FIREWALL','FORTINET_FORTIANALYZER','FORTINET_FORTIAUTHENTICATOR','FORTINET_FORTICLIENT','FORTINET_FORTIEDR','FORTINET_FORTINAC','FORTINET_SANDBOX','FORTINET_WEBPROXY','FOX_IT_STIX','FREEIPA','FREERADIUS','FRONTLINE_VM','FUTUREX_HSM','GCP_APIGEE','GCP_CLOUDIDENTITY_DEVICES','GCP_CLOUDIDENTITY_DEVICEUSERS','GCP_CLOUDIOT','GCP_CLOUDSQL','GCP_COMPUTE','GCP_IDS','GCP_LOADBALANCING','GCP_RUN','GCP_THREAT_DETECTION','GCP_VPC_FLOW','GIGAMON','GITHUB','GITLAB','GLOBALSCAPE_SFTP','GLUSTER_FS','GMAIL_LOGS','GMV_CHECKER','GMV_CHECKER_CONTEXT','GREATHORN','GTB_DLP','GUARDDUTY','GUARDICORE_CENTRA','GUARDIUM','HADOOP','HAPROXY','HAPROXY_LOADBALANCER','HASHICORP','HCL_BIGFIX','HCNET_ACCOUNT_ADAPTER','HITACHI_ID_PAM','HONEYD','HPE_ILO','HP_PRINTER','HP_PROCURVE','HYPR_MFA','IBM_AS400','IBM_CICS','IBM_DATAPOWER','IBM_MAAS360','IBM_MQ_FILE_TRANSFER','IBM_SECURITY_VERIFY','IBM_SPECTRUM_PROTECT','IBM_SWITCH','IBM_TIVOLI','IBM_WEBSEAL','IBM_WEBSPHERE_APP_SERVER','IBM_ZOS','IBM_ZSECURE_ALERT','IBOSS_WEBPROXY','IDRAC','IIS','ILLUMIO_CORE','IMANAGE_CLOUD','IMPERVA_DB','IMPERVA_SECURESPHERE','IMPERVA_SONAR','IMPERVA_WAF','IMPRIVATA_CONFIRM_ID','IMPRIVATA_IDG','IMPRIVATA_ONESIGN','INFOBLOX','INFOBLOX_DHCP','INFOBLOX_DNS','INFOBLOX_LOADBALANCER','INFOBLOX_NETMRI','INFOBLOX_RPZ','INFORMIX','IPSWITCH_MOVEIT_AUTOMATION','IPSWITCH_MOVEIT_TRANSFER','IPSWITCH_SFTP','ISC_DHCP','JAMF','JAMF_COMPLIANCE_REPORTER','JAMF_PROTECT','JDE','JENKINS','JUMPCLOUD_DAAS','JUNIPER_FIREWALL','JUNIPER_IPS','JUNIPER_JUNOS','JUNIPER_MX','KAMAILIO','KASEYA','KASPERSKY_AV','KEA_DHCP','KEEPER','KEMP_LOADBALANCER','KIBANA','KISI','KNOWBE4_PHISHER','KONG_GATEWAY','KUBERNETES_AUDIT','KUBERNETES_AUTH_PROXY','KUBERNETES_NODE','KYRIBA','LACEWORK','LASTPASS','LENEL_ONGUARD','LEXMARK_PRINTER','LIAISON_NUBRIDGES','LIMACHARLIE_EDR','LINUX_DHCP','LINUX_SYSMON','LOGICMONITOR','LOOKINGGLASS_IPS','LOOKING_GLASS_IOC','LSI_BMS','MACOS','MAILSCANNER','MALWAREBYTES_EDR','MANAGE_ENGINE_AD360','MANAGE_ENGINE_PASSWORD_MANAGER','MANAGE_ENGINE_REPORTER_PLUS','MANGOAPPS','MARIA_DB','MATERIAL_SECURITY','MATRIX_FRONTIER','MCAFEE_ATD','MCAFEE_DLP','MCAFEE_EDR','MCAFEE_EPO','MCAFEE_ESM','MCAFEE_IPS','MCAFEE_MVISION_CASB','MCAFEE_SKYHIGH_CASB','MCAFEE_UCE','MCAFEE_WEBPROXY','MCAFEE_WEB_PROTECTION','MEDIGATE_CMDB','MEDIGATE_IOT','MENANDMICE_DNS','MENLO_SECURITY','MICROSEMI_NTP','MICROSOFT_ATA','MICROSOFT_CASB','MICROSOFT_DEFENDER_ENDPOINT','MICROSOFT_DEFENDER_IDENTITY','MICROSOFT_GRAPH_ALERT','MICROSOFT_NETLOGON','MICROSOFT_SCEP','MICROSOFT_SENTINEL','MICROSOFT_SQL','MICROSOFT_SSTP','MIMECAST_MAIL','MIMECAST_WEBPROXY','MINERVA_AV','MISP_IOC','MOBILEIRON','MONGO_DB','MULESOFT','MYSQL','NAGIOS','NASUNI_FILE_SERVICES','NCC_SCOUTSUITE','NCR_DIGITAL_INSIGHT_FSG','NCR_DIGITAL_INSIGHT_GL','NETAPP_ONTAP','NETAPP_SAN','NETDISCO','NETFILTER_IPTABLES','NETMOTION','NETSKOPE_ALERT','NETSKOPE_WEBPROXY','NETSURION_PROTECTWISE','NGINX','NIMBLE_OS','NIST_NVD','NIX_SYSTEM','NUCLEUS_ASSET','NUCLEUS_VULNERABILITY','NUCLEUS_VULNERABILITY_DELTA','NUTANIX_FRAME','NUTANIX_PRISM','NXLOG_MANAGER','OBSERVEIT','OBSIDIAN','OFFICE_365','OKTA','OKTA_ACCESS_GATEWAY','OKTA_RADIUS','OKTA_USER_CONTEXT','ONBASE_CMS','ONEIDENTITY_ARS','ONEIDENTITY_CHANGE_AUDITOR','ONEIDENTITY_DEFENDER','ONEIDENTITY_TPAM','ONELOGIN_SSO','ONELOGIN_USER_CONTEXT','ONEPASSWORD','OPENAM','OPENDJ','OPENGEAR','OPENLDAP','OPENPATH','OPENSSH','OPENTEXT_FAX2MAIL','OPEN_VPN','ORACLE_CLOUD_AUDIT','ORACLE_DB','ORCA','ORDR_IOT','OSCAR_CLAIMS','OSINT_IOC','OSQUERY_EDR','OSSEC','PAN_CASB','PAN_EDR','PAN_FIREWALL','PAN_GLOBAL_PROTECT','PAN_IOC','PAN_PRISMA_CLOUD','PASSIVE_DNS','PCAP_SSL_CLIENT_HELLO','PEOPLESOFT','PEPLINK_LOADBALANCER','PEPLINK_ROUTER','PEPLINK_SWITCH','PERIMETERX_BOT_PROTECTION','PFSENSE','PHISHEYE_ALERT','PING','PIVOTAL','PLASO','PLIXER_SCRUTINIZER','POSTFIX_MAIL','POWERSHELL','POWER_DNS','PREEMPT','PREEMPT_AUTH','PREVEIL_ENTERPRISE','PROOFID','PROOFPOINT_CASB','PROOFPOINT_MAIL','PROOFPOINT_MAIL_FILTER','PROOFPOINT_ON_DEMAND','PROOFPOINT_TRAP','PROOFPOINT_WEB_BROWSER_ISOLATION','PROTEGRITY_DEFIANCE','PROWATCH','PULSE_SECURE_VPN','PUPPET','PURE_STORAGE','QUALYS_CONTINUOUS_MONITORING','QUALYS_VM','QUEST_AD','RADIUS','RADWARE_DDOS','RADWARE_FIREWALL','RAPID7_INSIGHT','RAPID7_NEXPOSE','RECORDED_FUTURE_IOC','REDCANARY_CLOUD_PROTECTION_RAW','REDCANARY_EDR','REDHAT_DIRECTORY_SERVER','REDHAT_IM','REDHAT_KEYCLOAK','REDHAT_OPENSHIFT','REDHAT_STACKROX','REMEDIANT_SECUREONE','RH_ISAC_IOC','RIBBON_ANALYTICS_PLATFORM','RIBBON_SBC','RING_CENTRAL','RISKIQ_DIGITAL_FOOTPRINT','RSA_AUTH_MANAGER','RSA_NETWITNESS','RUBRIK','RUBRIK_POLARIS','RUCKUS_WIRELESS','SAFECONNECT_NAC','SAILPOINT_IAM','SALESFORCE','SALESFORCE_CONTEXT','SAP_HANA','SAP_INSURANCE','SECBERUS','SECUREAUTH_SSO','SECURELINK','SEMPERIS_ADFR','SEMPERIS_DSP','SENDMAIL','SENTINEL_DV','SENTINEL_EDR','SEP','SERVICENOW_AUDIT','SERVICENOW_CMDB','SERVICENOW_ROLES','SERVICENOW_SECURITY','SEVCO_CMDB','SHAREPOINT','SHIBBOLETH_IDP','SHODAN_IO','SIEMENS_SIPASS','SIGNAL_SCIENCES_WAF','SILVERFORT','SILVERPEAK_FIREWALL','SITEMINDER_SSO','SLACK_AUDIT','SNARE_SOLUTIONS','SNIPE_IT','SNOOPY_LOGGER','SNORT_IDS','SNOWFLAKE','SOFTWARE_HOUSE_ACS','SOLARIS_SYSTEM','SOLARWINDS_SERV_U','SONARQUBE','SONIC_FIREWALL','SOPHOS_AV','SOPHOS_CAPSULE8','SOPHOS_CENTRAL','SOPHOS_DHCP','SOPHOS_EDR','SOPHOS_FIREWALL','SOPHOS_UTM','SOURCEFIRE_IDS','SPLUNK','SPLUNK_DNS','SPLUNK_PHANTOM','SPYCLOUD','SQUID_WEBPROXY','STEALTHBITS_AUDIT','STEALTHBITS_DEFEND','STEELHEAD','STREAMALERT','STRONGSWAN_VPN','SUPERNA_EYEGLASS','SURICATA_EVE','SURICATA_IDS','SWIFT_AMH','SWIMLANE','SYMANTEC_CASB','SYMANTEC_DLP','SYMANTEC_EDR','SYMANTEC_EVENT_EXPORT','SYMANTEC_MAIL','SYMANTEC_VIP','SYMANTEC_WEB_ISOLATION','SYMANTEC_WSS','SYSDIG','TANIUM_ASSET','TANIUM_AUDIT','TANIUM_COMPLY','TANIUM_DEPLOY','TANIUM_DISCOVER','TANIUM_INSIGHT','TANIUM_INTEGRITY_MONITOR','TANIUM_PATCH','TANIUM_QUESTION','TANIUM_REVEAL','TANIUM_TH','TANIUM_THREAT_RESPONSE','TEAMVIEWER','TENABLE_IO','TENABLE_SC','TGDETECT','THALES_DIS','THALES_LUNA_HSM','THALES_MFA','THINKST_CANARY','THREATCONNECT_IOC','THYCOTIC','THYCOTIC_DEVOPS_SECRETVAULT','TIPPING_POINT','TOMCAT','TRENDMICRO_AV','TRENDMICRO_CLOUDAPPSECURITY','TRENDMICRO_DEEP_SECURITY','TRENDMICRO_EDR','TRENDMICRO_WEBPROXY','TRIPWIRE_FIM','UBIQUITI_SWITCH','UDM','ULTRADNS','UMBRELLA_DNS','UMBRELLA_FIREWALL','UMBRELLA_IP','UMBRELLA_WEBPROXY','UNBOUND_DNS','UNIFI_AP','UNIFI_SWITCH','UPTYCS_EDR','VANDYKE_SFTP','VARONIS','VECTRA_DETECT','VECTRA_STREAM','VEEAM','VENAFI','VERIZON_NDR','VIRUSTOTAL_THREAT_HUNTER','VITALQIP','VMRAY_FLOG_XML','VMWARE_AVINETWORKS_IWAF','VMWARE_AVI_VANTAGE','VMWARE_ESX','VMWARE_HCX','VMWARE_HORIZON','VMWARE_NSX','VMWARE_TANZU','VMWARE_VCENTER','VMWARE_VREALIZE','VMWARE_VSHIELD','VMWARE_WORKSPACE_ONE','VOLTAGE','VORMETRIC','VSFTPD_AUDIT','WATCHGUARD','WAZUH','WHITECLOUD_EDR','WINDOWS_AD','WINDOWS_APPLOCKER','WINDOWS_DEFENDER_ATP','WINDOWS_DEFENDER_AV','WINDOWS_DHCP','WINDOWS_DNS','WINDOWS_FIREWALL','WINDOWS_NET_POLICY_SERVER','WINDOWS_SYSMON','WINEVTLOG','WINEVTLOG_XML','WIZ_IO','WORDPRESS_CMS','WORKDAY','WORKDAY_AUDIT','WORKSPACE_ACTIVITY','WORKSPACE_ALERTS','WORKSPACE_CHROMEOS','WORKSPACE_GROUPS','WORKSPACE_MOBILE','WORKSPACE_PRIVILEGES','WORKSPACE_USERS','WORKSPOT_CONTROL','WP_ENGINE','WTI_CONSOLE_SERVERS','YUBICO_OTP','ZEROFOX_PLATFORM','ZIMPERIUM','ZIX_EMAIL_ENCRYPTION','ZOOM_OPERATION_LOGS','ZSCALER_CASB','ZSCALER_DNS','ZSCALER_FIREWALL','ZSCALER_VPN','ZSCALER_WEBPROXY'] {type: "string"}

#@markdown All subsequent fields are optional

#@markdown The namespace for identifying logs from this collector.

assetNamespace = "" #@param {type : "string"}

#@markdown A list of arbitrary key:value pairs that can be specified in the forwarder configuration. **Note:** This is a global setting that applies to the forwarder and the forwarder's collectors, unless it is overridden at the collector level.

#@markdown Please enter them in the form **key1:value1,key2:value2...**
labels =  "" #@param {type: "string"}

#@markdown Paste a regex filter from the Regex Filter Builder in the cell above

regexFilters_object = "" #@param {type :"string"}

#@markdown Specifies the disk buffering state for the collector
diskBuffer_state = "" #@param ["", "ACTIVE", "SUSPENDED"]

#@markdown The directory path for files written.
diskBuffer_directoryPath = "" #@param {type: "string"}

#@markdown The maximum buffered file size.
diskBuffer_maxFileBufferBytes = 0 #@param {type: "integer"}

#@markdown The number of seconds between batches. The default is 10.
maxSecondsPerBatch = 0 #@param {type: "integer"}

#@markdown The number of bytes queued before forwarder batch upload. The default is 1048576.
maxBytesPerBatch = 0 #@param {type: "integer"}

#@markdown The below section allows you to specify the type of ingestion settings to use for this collector. One type is allowed for a collector configuration.

#@markdown ##File

fileSettings_filePath = "" #@param {type: "string"}

#@markdown ##Kafka

#@markdown The username of an identity used for authentication.
kafkaSettings_authentication_username = "" #@param {type: "string"}

#@markdown The password of the account identified by the username.
kafkaSettings_authentication_password = "" #@param {type: "string"}

#@markdown The Kafka topic from which to ingest data. For details, see [Collect data from Kafka topics](https://cloud.google.com/chronicle/docs/install/forwarder-linux#collect_data_from_kafka_topic).
kafkaSettings_topic = "" #@param {type: "string"}

#@markdown A group ID.
kafkaSettings_groupId = "" #@param {type: "string"}

#@markdown The maximum number of seconds a dial will wait for a connect to complete. The default is 60.
kafkaSettings_timeout = 0 #@param {type : "integer"}

#@markdown A repeated string listing Kafka brokers. For example: **"broker-1:9092", "broker-2:9093"**

#@markdown **Note:** All values are replaced during an update operation. Therefore, to update a list of brokers to add a new broker, specify all existing brokers and the new broker.
kafkaSettings_brokers = "" #@param {type : "string"}

#@markdown The path and certificate filename. For example: **/<wbr>path/to/cert.pem**
kafkaSettings_tlsSettings_certificate = "" #@param {type: "string"}

#@markdown The path and certificate key filename. For example: **/<wbr>path/to/cert.key**
kafkaSettings_tlsSettings_certificateKey = "" #@param {type : "string"}

#@markdown The minimum TLS version.
kafkaSettings_minimumTlsVersion = "" #@param {type : "string"}

#@markdown If true, enables SSL certification verification (for kafka connections). The default is **false**.
kafkaSettings_tlsSettings_insecureSkipVerify = False #@param {type : "boolean"}

#@markdown ##PCAP
#@markdown The interface to listen to for PCAP data.
pcapSettings_networkInterface = "" #@param {type : "string"}

#@markdown The Berkeley Packet Filter (BPF) for pcap.
pcapSettings_bpf = "" #@param {type : "string"}

#@markdown ##Splunk
#@markdown The username of an identity used for authentication.
splunkSettings_authentication_username = "" #@param {type : "string"}

#@markdown The password of the account identified by the username.
splunkSettings_authentication_password = "" #@param {type : "string"}

#@markdown The host or IP address for the Splunk REST API.
splunkSettings_host = "" #@param {type : "string"}

#@markdown The port for the Splunk REST API.
splunkSettings_port = 0 #@param {type : "integer"}

#@markdown 	The minimum time range in seconds for a given Splunk search. For details, see Collect Splunk data. The default is **10**.
splunkSettings_minimumWindowSize = 0 #@param {type : "integer"}

#@markdown The maximum time range in seconds for a given Splunk search. For details, see Collect Splunk data. The default is **30**.
splunkSettings_maximumWindowSize = 0 #@param {type : "integer"}

#@markdown The query used to filter records within Splunk. For example: **search index=* sourcetype=dns**
splunkSettings_queryString = "" #@param {type : "string"}

#@markdown The query mode for Splunk. For example: **realtime**
splunkSettings_queryMode = "" #@param {type : "string"}

#@markdown If true, the certificate is ignored.
splunkSettings_certIgnored = False #@param {type : "boolean"}

#@markdown ##Syslog
#@markdown Specifies the protocol the collector will use to listen for syslog data.
syslogSettings_protocol = "UDP" #@param ["", "UDP", "TCP"]

#@markdown The target IP address or hostname where the collector resides and listens for syslog data.
syslogSettings_address = "0.0.0.0" #@param {type : "string"}

#@markdown The target port where the collector resides and listens for syslog data.
syslogSettings_port = 10514 #@param {type : "integer"}

#@markdown The size in bytes of the TCP socket's buffer. The default for TCP is **65536**. The default for UDP is **8192**.
syslogSettings_bufferSize = 0 #@param {type : "integer"}

#@markdown The number of seconds of inactivity after which the TCP connection is dropped. The default is **60**.
syslogSettings_connectonTimeout = 0 #@param {type : "integer"}

collector_config = {}

if logType:
  collector_config["logType"] = logType

if assetNamespace:
  if "metadata" not in collector_config.keys():
    collector_config["metadata"] = {}
  collector_config["metadata"]["assetNamespace"] = assetNamespace

if labels:
  if "metadata" not in collector_config.keys():
    collector_config["metadata"] = {}
  collector_config["metadata"]['labels'] = []
  labels = labels.split(",")

  for label in labels:
    label = label.strip()
    label = label.split(":")
    collector_config["metadata"]['labels'].append({"key" : label[0].strip(), "value" : label[1].strip()})

if regexFilters_object:
  collector_config["regexFilters"] = json.loads(regexFilters_object)

if diskBuffer_state:
  if "diskBuffer" not in collector_config.keys():
    collector_config["diskBuffer"] = {}
  collector_config["diskBuffer"]["state"] = diskBuffer_state

if diskBuffer_directoryPath:
  if "diskBuffer" not in collector_config.keys():
    collector_config["diskBuffer"] = {}
  collector_config["diskBuffer"]["directoryPath"] = diskBuffer_directoryPath

if diskBuffer_maxFileBufferBytes:
  if "diskBuffer" not in collector_config.keys():
    collector_config["diskBuffer"] = {}
  collector_config["diskBuffer"]["maxFileBufferBytes"] = diskBuffer_maxFileBufferBytes

if maxSecondsPerBatch:
  collector_config["maxSecondsPerBatch"] = maxSecondsPerBatch

if maxBytesPerBatch:
  collector_config["maxBytesPerBatch"] = maxBytesPerBatch

if fileSettings_filePath:
  collector_config["fileSettings"] = {}
  collector_config["fileSettings"]["filePath"] = fileSettings_filePath

if kafkaSettings_authentication_username:
  if "kafkaSettings" not in collector_config.keys():
    collector_config["kafkaSettings"] = {}
  if "authentication" not in collector_config["kafkaSettings"].keys():
    collector_config["kafkaSettings"]["authentication"] = {}
  collector_config["kafkaSettings"]["authentication"]["username"] = kafkaSettings_authentication_username

if kafkaSettings_authentication_password:
  if "kafkaSettings" not in collector_config.keys():
    collector_config["kafkaSettings"] = {}
  if "authentication" not in collector_config["kafkaSettings"].keys():
    collector_config["kafkaSettings"]["authentication"] = {}
  collector_config["kafkaSettings"]["authentication"]["password"] = kafkaSettings_authentication_password

if kafkaSettings_topic:
  if "kafkaSettings" not in collector_config.keys():
    collector_config["kafkaSettings"] = {}
  collector_config["kafkaSettings"]["topic"] = kafkaSettings_topic

if kafkaSettings_groupId:
  if "kafkaSettings" not in collector_config.keys():
    collector_config["kafkaSettings"] = {}
  collector_config["kafkaSettings"]["groupId"] = kafkaSettings_groupId

if kafkaSettings_timeout:
  if "kafkaSettings" not in collector_config.keys():
    collector_config["kafkaSettings"] = {}
  collector_config["kafkaSettings"]["timeout"] = kafkaSettings_timeout

if kafkaSettings_brokers:
  if "kafkaSettings" not in collector_config.keys():
    collector_config["kafkaSettings"] = {}
  collector_config["kafkaSettings"]["brokers"] = kafkaSettings_brokers

if kafkaSettings_tlsSettings_certificate:
  if "kafkaSettings" not in collector_config.keys():
    collector_config["kafkaSettings"] = {}
  if "tlsSettings" not in collector_config["kafkaSettings"].keys():
    collector_config["kafkaSettings"]["tlsSettings"] = {}
  collector_config["kafkaSettings"]["tlsSettings"]["certificate"] = kafkaSettings_tlsSettings_certificate

if kafkaSettings_tlsSettings_certificateKey:
  if "kafkaSettings" not in collector_config.keys():
    collector_config["kafkaSettings"] = {}
  if "tlsSettings" not in collector_config["kafkaSettings"].keys():
    collector_config["kafkaSettings"]["tlsSettings"] = {}
  collector_config["kafkaSettings"]["tlsSettings"]["certificateKey"] = kafkaSettings_tlsSettings_certificateKey

if kafkaSettings_tlsSettings_insecureSkipVerify:
  if "kafkaSettings" not in collector_config.keys():
    collector_config["kafkaSettings"] = {}
  if "tlsSettings" not in collector_config["kafkaSettings"].keys():
    collector_config["kafkaSettings"]["tlsSettings"] = {}
    collector_config["kafkaSettings"]["tlsSettings"]["insecureSkipVerify"] = kafkaSettings_tlsSettings_insecureSkipVerify

if pcapSettings_networkInterface:
  if "pcapSettings" not in collector_config.keys():
    collector_config["pcapSettings"] = {}
  collector_config["pcapSettings"]["networkInterface"] = pcapSettings_networkInterface

if pcapSettings_bpf:
  if "pcapSettings" not in collector_config.keys():
    collector_config["pcapSettings"] = {}
  collector_config["pcapSettings"]["bpf"] = pcapSettings_bpf

if splunkSettings_authentication_username:
  if "splunkSettings" not in collector_config.keys():
    collector_config["splunkSettings"] = {}
  if "authentication" not in collector_config["splunkSettings"].keys():
    collector_config["splunkSettings"]["authentication"] = {}
  collector_config["splunkSettings"]["authentication"]["username"] = splunkSettings_authentication_username

if splunkSettings_authentication_password:
  if "splunkSettings" not in collector_config.keys():
    collector_config["splunkSettings"] = {}
  if "authentication" not in collector_config["splunkSettings"].keys():
    collector_config["splunkSettings"]["authentication"] = {}
  collector_config["splunkSettings"]["authentication"]["password"] = splunkSettings_authentication_password

if splunkSettings_host:
  if "splunkSettings" not in collector_config.keys():
    collector_config["splunkSettings"] = {}
  collector_config["splunkSettings"]["host"] = splunkSettings_host

if splunkSettings_port:
  if "splunkSettings" not in collector_config.keys():
    collector_config["splunkSettings"] = {}
  collector_config["splunkSettings"]["port"] = splunkSettings_port

if splunkSettings_minimumWindowSize:
  if "splunkSettings" not in collector_config.keys():
    collector_config["splunkSettings"] = {}
  collector_config["splunkSettings"]["minimumWindowSize"] = splunkSettings_minimumWindowSize

if splunkSettings_maximumWindowSize:
  if "splunkSettings" not in collector_config.keys():
    collector_config["splunkSettings"] = {}
  collector_config["splunkSettings"]["maximumWindowSize"] = splunkSettings_maximumWindowSize

if splunkSettings_queryString:
  if "splunkSettings" not in collector_config.keys():
    collector_config["splunkSettings"] = {}
  collector_config["splunkSettings"]["queryString"] = splunkSettings_queryString

if splunkSettings_queryMode:
  if "splunkSettings" not in collector_config.keys():
    collector_config["splunkSettings"] = {}
  collector_config["splunkSettings"]["queryMode"] = splunkSettings_queryMode

if splunkSettings_certIgnored:
  if "splunkSettings" not in collector_config.keys():
    collector_config["splunkSettings"] = {}
  collector_config["splunkSettings"]["certIgnored"] = splunkSettings_certIgnored

if syslogSettings_protocol:
  if "syslogSettings" not in collector_config.keys():
    collector_config["syslogSettings"] = {}
  collector_config["syslogSettings"]["protocol"] = syslogSettings_protocol

if syslogSettings_address:
  if "syslogSettings" not in collector_config.keys():
    collector_config["syslogSettings"] = {}
  collector_config["syslogSettings"]["address"] = syslogSettings_address

if syslogSettings_port:
  if "syslogSettings" not in collector_config.keys():
    collector_config["syslogSettings"] = {}
  collector_config["syslogSettings"]["port"] = syslogSettings_port

if syslogSettings_bufferSize:
  if "syslogSettings" not in collector_config.keys():
    collector_config["syslogSettings"] = {}
  collector_config["syslogSettings"]["bufferSize"] = syslogSettings_bufferSize

if syslogSettings_connectonTimeout:
  if "syslogSettings" not in collector_config.keys():
    collector_config["syslogSettings"] = {}
  collector_config["syslogSettings"]["connectonTimeout"] = syslogSettings_connectonTimeout

# There can only be one type of colletor per call
valid_colletors = [
    "syslogSettings",
    "splunkSettings",
    "pcapSettings",
    "kafkaSettings",
    "fileSettings"
]

collectors_present = list(set(valid_colletors) & set(collector_config.keys()))
if len(collectors_present) > 1:
  print(f"At most one type of collector can be present. There are {len(collectors_present)}\nThey are: {', '.join(collectors_present)}")
else:
  print("Please copy the output and paste it into the config cell in the appropriate call.")
  print(json.dumps(collector_config))

#BigQuery Access API
Documentation for this API can be found [here](https://cloud.google.com/chronicle/docs/reference/bigquery-access-api).

[Jump to top](#scrollTo=v0jKbGj8ZZRB)

In [ ]:
#@title Update BigQuery Access
#@markdown Chronicle supports self-service access to Chronicle data in BigQuery. You can use this endpoint to grant Identity and Access Management (IAM) roles that give the following permissions for a user email:

#@markdown The email must be a Google Accounts and ID Administration (GAIA) user email address of a Chronicle Security customer.
email = "" #@param {type: "string"}

body = {
    "email" : email
}

uri_to_post = f"https://{region_prefix}backstory.googleapis.com/v1/tools/bigqueryAccess:update"
resp = session.patch(uri_to_post, json=body)
if resp.status_code > 299:
  print(resp.text)
else:
  pprint(resp.json())


#Chronicle CLI Tool
Documentation for this tool can be found [here](https://cloud.google.com/chronicle/docs/administration/cli-user-guide#data_access_workflows).

The tool itself can be found [here](https://github.com/chronicle/cli).

[Jump to top](#scrollTo=v0jKbGj8ZZRB)

In [ ]:
#@title Forwarders
operation = "create" #@param ["create", "delete", "generate_files", "get", "list", "update"]
!chronicle_cli forwarders $operation --region $cli_region

In [ ]:
#@title Collectors
operation = "create" #@param ["create", "delete", "get", "list", "update"]
!chronicle_cli forwarders collectors $operation --region $cli_region

In [ ]:
#@title Feeds
operation = "create" #@param ["create", "delete", "disable", "enable", "get", "list", "update"]
!chronicle_cli feeds $operation --region $cli_region

In [ ]:
#@title Parsers
operation = "list" #@param ["archive", "download", "generate", "history", "list", "list_errors", "run", "status", "submit"]
!chronicle_cli parsers $operation --region $cli_region

In [ ]:
#@title Parsers V2
#@markdown This functionality requires two things

#@markdown 1) A Chronicle instance with BYOP setup<br>2) A service account key from the GCP project tied to the Chronicle instance with the appropriate IAM roles granted

PROJECT_ID = "chronicle-byop" #@param {type: "string"}
CUSTOMER_ID = "34871035-2752-4075-aed6-2e7f261be67d" #@param {type: "string"}

operation = "get_parser" #@param ["list_parsers","list_extensions","run_parser","submit_parser","submit_extension","delete_parser","delete_extension","deactivate_parser","activate_parser","get_parser","get_extension","get_validation_report"]

#@markdown Optional parameters, please refer to [documentation](https://cloud.google.com/chronicle/docs/administration/cli-user-guide#parser_management_v2_user_workflows) for more information.
AUTHOR  = "" #@param [""] {allow-input: true}
CONFIG_FILE = "" #@param [""] {allow-input: true}
LOG_FILE = "" #@param [""] {allow-input: true}
LOG_TYPE = "ADFS" #@param [""] {allow-input: true}
PARSER_ID = "" #@param [""] {allow-input: true}
PARSEREXTENSION_ID = "" #@param [""] {allow-input: true}
state = "" #@param  ["ALL","ACTIVE","INACTIVE",""]
export = "" #@param [""] {allow-input: true}
file_format = "" #@param ["", "TXT", "JSON"]
env = "" #@param ["","prod","test"]
verbose = False #@param {type:"boolean"}

if state:
  state = f"--state {state}"
else:
  state = ""

if file_format:
  file_format = f"--file-format {file_format}"
else:
  file_format = ""

if export:
  export = f"--export {export}"
else:
  export = ""

if env:
  env = f"--env {env}"
else:
  env = ""

if verbose:
  verbose = f"--verbose"
else:
  verbose = ""

print(f"Running:\nchronicle_cli parsers {operation} --v2 {state} {file_format} {export} {env} {verbose} {PROJECT_ID} {CUSTOMER_ID} --region {cli_region} {LOG_TYPE} {CONFIG_FILE} {LOG_FILE} {PARSER_ID} {PARSEREXTENSION_ID}\n")
!chronicle_cli parsers $operation --v2 $state $file_format $export $env $verbose $PROJECT_ID $CUSTOMER_ID --region $cli_region $LOG_TYPE $CONFIG_FILE $LOG_FILE $PARSER_ID $PARSEREXTENSION_ID

In [ ]:
#@title Provide Big Query email access
!chronicle_cli bigquery provide_access --region $cli_region

# Notes

## File uploads

- It turns out that the colab `files.upload()` call ([docs](https://colab.sandbox.google.com/notebooks/io.ipynb#scrollTo=BaCkyg5CV5jF))returns the name of the file selected, not the resulting name saved to the filesystem. If the same file is uploaded multiple times it wil be saved as:
  - `filename.ext`
  - `filename (1).ext`
  - `filename (2).ext`
  - etc...
- This means that the original file (`filename.ext`) will continue to get referenced and may cause confusion.
- Please try to keep this in mind if re-uploading the same file mulitple times.
- Files can be deleted from the files view on the left hand side (folder icon)

[Jump to top](#scrollTo=v0jKbGj8ZZRB)
